<a href="https://colab.research.google.com/github/batust/presidio-db-json/blob/main/pre-prcoessor-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Trying out batch processing
from typing import List, Optional, Dict, Union, Iterator, Iterable
import collections
from dataclasses import dataclass
import pprint

import pandas as pd

from presidio_analyzer import AnalyzerEngine, BatchAnalyzerEngine, RecognizerResult, DictAnalyzerResult
from presidio_anonymizer import AnonymizerEngine, BatchAnonymizerEngine
from presidio_anonymizer.entities import EngineResult

In [ ]:
# Authenicating and accessing Llama
from os import environ
from huggingface_hub import login

HUGGING_FACE_TOKEN = environ.get("HUGGING_FACE_TOKEN")
login(token=HUGGING_FACE_TOKEN)

In [ ]:
# Keeps crashing session
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
llama_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def classify_sensitive_columns(columns):
    prompt = f"Given the column names {columns}, identify which may contain sensitive information (e.g., names, IDs, financial data). Return only a list of sensitive column names."

    response = llama_pipeline(prompt, max_length=100, do_sample=True)

    return response[0]['generated_text']

columns = ["Account", "Project ID", "Program Name", "Employee", "Cost Category",
           "Entity", "COA ID", "COA", "Day", "Rate", "QUANTITY", "Actuals",
           "Posting Category", "Description"]

print(classify_sensitive_columns(columns))

In [ ]:
!pip install flair
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0].strip().replace(' ', '-')

Saving testing-csv-ff.csv to testing-csv-ff.csv


In [ ]:
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger

EXCLUDED_HEADERS = ["description", "comment", "comments", "query", "notes", "email"]

# ner_tagger = SequenceTagger.load("flair/ner-english")
tagger = SequenceTagger.load('flair/pos-english')
if file_name.endswith('.xlsx'):
  df = pd.read_excel(file_name)
elif file_name.endswith('.csv'):
  df = pd.read_csv(file_name)

df

2025-03-19 11:28:24,203 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
2025-03-19 11:28:25,113 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


,id,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960
...,...,...,...,...,...,...,...,...,...,...,...
29995,1201877,hans anders,hansanders.nl,1982.0,retail,1001 - 5000,"giethoorn, overijssel, netherlands",netherlands,linkedin.com/company/hans-anders,390,959
29996,2828041,real canadian superstore,superstore.ca,NaN,retail,1001 - 5000,"madrid, madrid, spain",spain,linkedin.com/company/real-canadian-superstore,390,1320
29997,835865,"shumaker, loop & kendrick, llp",slk-law.com,1925.0,law practice,501 - 1000,"toledo, ohio, united states",united states,linkedin.com/company/shumaker-loop-&-kendrick-llp,390,688
29998,1611480,business wire,businesswire.com,1961.0,public relations and communications,501 - 1000,"san francisco, california, united states",united states,linkedin.com/company/business-wire,390,998


In [ ]:
# Not batch processing
def classify_cell_as_noun(texts):
    sentences = [Sentence(text) for text in texts]
    tagger.predict(sentences)

    results = []
    noun_count = 0

    for sentence, original_text in zip(sentences, texts):
        all_nouns = any("NN" in token.tag for token in sentence)
        classification = "Noun" if all_nouns else "Not a noun"

        results.append({"Original Text": original_text, "Classification": classification})
        noun_count += all_nouns

    is_noun_column = noun_count > (len(results) // 1.5)
    return results, is_noun_column

sensitive = set()

for column in df.columns:
    values = df[column].astype(str).tolist()
    results, is_noun = classify_cell_as_noun(values)

    if is_noun: sensitive.add(column)

print(sensitive)


In [ ]:
# Batch processing
import pandas as pd
import concurrent.futures
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/pos-english")

EXCLUDED_HEADERS = ["description", "comment", "comments", "query", "notes", "email"]
FILE_NAME = "companies_sorted.csv"
BATCH_SIZE = 10000
FLAIR_BATCH_SIZE = 100
MAX_THREADS = 4

sensitive = set()

def classify_cells_as_nouns(texts):
    """Batch processes text to check if columns are noun-heavy."""
    texts = ['' if pd.isna(text) else text for text in texts]

    results = []
    noun_count = 0

    for i in range(0, len(texts), FLAIR_BATCH_SIZE):
        batch_text = texts[i : i + FLAIR_BATCH_SIZE]
        sentences = [Sentence(text) for text in batch_text]

        tagger.predict(sentences)

        for sentence, original_text in zip(sentences, batch_text):
            all_nouns = any("NN" in token.tag for token in sentence)
            classification = "Noun" if all_nouns else "Not a noun"

            results.append({"Original Text": original_text, "Classification": classification})

            noun_count += all_nouns

    is_noun_column = noun_count > (len(results) * 0.66)

    return is_noun_column

def process_column(col, chunk):
    """Runs noun classification on a single column."""
    values = chunk[col].astype(str).fillna("").tolist()
    return col if classify_cells_as_nouns(values) else None

FILE_NAME = 'smaller_companies.csv'
if FILE_NAME.endswith(".xlsx"):
    df_iterator = pd.read_excel(FILE_NAME, chunksize=BATCH_SIZE)
elif FILE_NAME.endswith(".csv"):
    df_iterator = pd.read_csv(FILE_NAME, chunksize=BATCH_SIZE)

for chunk in df_iterator:
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        futures = {executor.submit(process_column, col, chunk): col for col in chunk.columns}

        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                sensitive.add(result)

print("Sensitive Columns:", sensitive)

2025-03-19 11:51:26,893 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, pos

Sensitive Columns: {'linkedin url', 'name', 'country', 'industry', 'locality'}


In [ ]:
masked_df = df_iterator.copy()

for col in masked_df:
  if col in sensitive:
    masked_df[col] = [f'{col}_{i}' for i in range(len(masked_df[col]))]

masked_df

In [ ]:
# Trying spacy and flair on large dataset using Threading
import re
import json
import spacy
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger
from concurrent.futures import ThreadPoolExecutor
from threading import Lock

# Load models
flair_tagger = SequenceTagger.load("flair/pos-english")
nlp = spacy.load("en_core_web_sm")

# Constants
BATCH_SIZE = 10000  # Process in chunks
FLAIR_SAMPLE_SIZE = 10  # Sample size for noun detection
THREADS = 4  # Number of parallel threads
PHONE_REGEX = r"\b(\+\d{1,4}\s?)?(\(?\d{1,4}\)?[-.]?\s?)?\d{3,4}[-.]?\d{3,4}[-.]?\d{3,4}\b"

DESCRIPTIVE_COLUMNS = {"description", "comment", "comments", "query", "notes"}
SENSITIVE_COLUMNS = set()
MAPPINGS = {}
VALUE_TO_MASK = {}
ENTITY_COUNTER = 1
mask_lock = Lock()  # Thread-safe lock for dictionary updates

def is_noun_column(texts):
    """Identify if a column contains mostly nouns based on a sample."""
    texts = ["" if pd.isna(text) else str(text) for text in texts[:FLAIR_SAMPLE_SIZE]]
    sentences = [Sentence(text) for text in texts]
    flair_tagger.predict(sentences)
    noun_count = sum(any("NN" in token.tag for token in sent) for sent in sentences)
    return noun_count > (len(texts) * 0.6)

def mask_text(text, col_name, row_idx):
    """Mask the input text and store the mapping, ensuring thread-safe access."""
    global ENTITY_COUNTER
    if pd.isna(text) or str(text).strip() == "":
        return text
    with mask_lock:  # Thread-safe updates
        if text in VALUE_TO_MASK:
            return VALUE_TO_MASK[text]
        masked_value = f"{col_name}_{row_idx}"
        VALUE_TO_MASK[text] = masked_value
        MAPPINGS[masked_value] = text
        ENTITY_COUNTER += 1
    return masked_value

def process_chunk(chunk):
    """Process a chunk of the CSV, masking sensitive data, and return a new masked DataFrame."""
    new_chunk = chunk.copy()  # Ensure original DataFrame remains unchanged

    for row_idx in new_chunk.index:
        # Mask sensitive columns
        for col in SENSITIVE_COLUMNS:
            new_chunk.at[row_idx, col] = mask_text(new_chunk.at[row_idx, col], col, row_idx)

        # Mask phone numbers across all columns
        for col in new_chunk.columns:
            new_chunk.at[row_idx, col] = re.sub(PHONE_REGEX, lambda m: mask_text(m.group(0), "phone", row_idx), str(new_chunk.at[row_idx, col]))

        # Mask descriptive text columns
        descriptive_cols = [col for col in new_chunk.columns if col.lower() in DESCRIPTIVE_COLUMNS]
        for col in descriptive_cols:
            doc = nlp(str(new_chunk.at[row_idx, col]))
            for ent in doc.ents:
                new_chunk.at[row_idx, col] = new_chunk.at[row_idx, col].replace(ent.text, mask_text(ent.text, col, row_idx))

    return new_chunk  # Return masked DataFrame

def main(file_path):
    """Process the entire CSV file in chunks and write masked data to a new file."""
    global SENSITIVE_COLUMNS

    # Identify sensitive columns from a sample
    df_sample = pd.read_csv(file_path, nrows=FLAIR_SAMPLE_SIZE)
    SENSITIVE_COLUMNS = {col for col in df_sample.columns if is_noun_column(df_sample[col].astype(str).tolist())}

    # Process CSV in chunks with threading
    reader = pd.read_csv(file_path, chunksize=BATCH_SIZE)
    first_chunk = True  # Ensure header is written only once

    with ThreadPoolExecutor(max_workers=THREADS) as executor:
        for masked_chunk in executor.map(process_chunk, reader):
            # Append masked chunk to file (efficient memory usage)
            masked_chunk.to_csv("masked_output.csv", index=False, mode="a", header=first_chunk)
            first_chunk = False  # Disable header for subsequent chunks

    # Save mappings separately for possible de-anonymization
    with open("mappings.json", "w") as f:
        json.dump(MAPPINGS, f)

    print("Processing complete. Masked data saved.")

if __name__ == "__main__":
    main('smaller_companies.csv')

2025-03-20 08:47:18,085 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


<ipython-input-24-72a88127c458>:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5872184' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_chunk.at[row_idx, col] = re.sub(PHONE_REGEX, lambda m: mask_text(m.group(0), "phone", row_idx), str(new_chunk.at[row_idx, col]))
<ipython-input-24-72a88127c458>:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1911.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_chunk.at[row_idx, col] = re.sub(PHONE_REGEX, lambda m: mask_text(m.group(0), "phone", row_idx), str(new_chunk.at[row_idx, col]))
<ipython-input-24-72a88127c458>:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '274047' has dtype incompatible with int64,

Processing complete. Masked data saved.


In [ ]:
!pip install flair
!pip install presidio_analyzer
!pip install presidio_anonymizer
# !python -m spacy download en_core_web_lg
!pip install --upgrade --force-reinstall "numpy==1.26.4"
!pip install --upgrade --force-reinstall "Cython" "spacy"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Testing on smaller dataset
import re
import json
import spacy
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger
from threading import Lock

# Load models
flair_tagger = SequenceTagger.load("flair/pos-english")
nlp = spacy.load("en_core_web_sm")

# Constants
PHONE_REGEX = r"\b(\+\d{1,4}\s?)?(\(?\d{1,4}\)?[-.]?\s?)?\d{3,4}[-.]?\d{3,4}[-.]?\d{3,4}\b"
DESCRIPTIVE_COLUMNS = {"description", "comment", "comments", "query", "notes"}
SENSITIVE_COLUMNS = set()
MAPPINGS = {}
VALUE_TO_MASK = {}
mask_lock = Lock()

def is_noun_column(texts):
    """Identify if a column contains mostly nouns based on a sample."""
    texts = ["" if pd.isna(text) else str(text) for text in texts]
    sentences = [Sentence(text) for text in texts]
    flair_tagger.predict(sentences)
    noun_count = sum(any("NN" in token.tag for token in sent) for sent in sentences)
    return noun_count > (len(texts) * 0.6)

def mask_text(text, col_name, row_idx):
    """Mask the input text and store the mapping."""
    if pd.isna(text) or str(text).strip() == "":
        return text
    with mask_lock:
        if text in VALUE_TO_MASK:
            return VALUE_TO_MASK[text]
        masked_value = f"{col_name}_{row_idx}"
        VALUE_TO_MASK[text] = masked_value
        MAPPINGS[masked_value] = text
    return masked_value

def process_chunk(chunk):
    """Process a dataframe chunk (30 rows) for masking and return a new masked DataFrame."""
    # Create a copy of the DataFrame to avoid modifying the original
    new_chunk = chunk.copy()

    for row_idx in new_chunk.index:
        # Mask sensitive columns
        for col in SENSITIVE_COLUMNS:
            new_chunk.loc[row_idx, col] = mask_text(new_chunk.loc[row_idx, col], col, row_idx)

        # Mask phone numbers
        for col in new_chunk.columns:
            new_chunk.loc[row_idx, col] = re.sub(PHONE_REGEX, lambda m: mask_text(m.group(0), "phone", row_idx), str(new_chunk.loc[row_idx, col]))

        # Mask descriptive text columns
        for col in set(DESCRIPTIVE_COLUMNS) & set(new_chunk.columns):
            doc = nlp(str(new_chunk.loc[row_idx, col]))
            for ent in doc.ents:
                new_chunk.loc[row_idx, col] = new_chunk.loc[row_idx, col].replace(ent.text, mask_text(ent.text, col, row_idx))

    return new_chunk  # Returning the new modified DataFrame


def main(file_path):
    """Process only 30 rows of the CSV file."""
    global SENSITIVE_COLUMNS
    # Read first 30 rows
    df = pd.read_csv(file_path, nrows=30)

    # Identify sensitive columns (case-insensitive)
    SENSITIVE_COLUMNS = {col for col in df.columns if is_noun_column(df[col].astype(str).tolist())}

    # Process and save output
    processed_df = process_chunk(df)
    print("\nProcessed Data:\n", processed_df.head(10))  # Display first 10 rows for verification
    processed_df.to_csv("masked_output.csv", index=False)

    with open("mappings.json", "w") as f:
        json.dump(MAPPINGS, f)

    print("Processing complete. Masked data saved.")

if __name__ == "__main__":
    main('smaller_companies.csv')


# **TESTING CODE!**

In [ ]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")

# Regex
EMAIL_REGEX = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
PHONE_REGEX = r"\b(\+\d{1,4}\s?)?(\(?\d{1,4}\)?[-.]?\s?)?\d{3,4}[-.]?\d{3,4}[-.]?\d{3,4}\b"

# def detect_noun_desc(text):
#     doc = nlp(text)
#     modified_text = []

#     for token in doc:
#         token_text = token.text

#         if re.fullmatch(EMAIL_REGEX, token_text):
#             modified_text.append("<email>")
#         elif re.fullmatch(PHONE_REGEX, token_text):
#             modified_text.append("<phone>")
#         elif token.pos_ == "PROPN":
#             modified_text.append("<sensitive>")
#         else:
#             modified_text.append(token_text)

#     return " ".join(modified_text)

def detect_noun(file_path):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            continue  # Skip purely numeric columns

        text_samples = df[col].astype(str).head(5)  # Take first 5 values
        noun_count = 0
        total_count = 0

        for value in text_samples:
            if "%" in value or value.replace(".", "").isdigit():
                continue  # Skip percentage or number-like values

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['NOUN', 'PROPN']:
                    noun_count += 1
                total_count += 1

        # Mark column as sensitive only if a significant portion are nouns
        if total_count > 0 and (noun_count / total_count) > 0.2:
            sensitive.append(col)

    return sensitive

In [ ]:
import pandas as pd
import re

def descriptive_cols(file_path):
    """
    Reads a CSV or Excel file, removes column names similar to 'description', 'remarks', etc.,
    and returns a filtered list of sensitive columns.
    """
    # Define keywords to filter out
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation",
    "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience",
    "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report",
    "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text"]

    # Ensure columns are properly loaded from CSV/Excel
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, nrows=1)  # Read only header
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path, nrows=1, engine="openpyxl")  # Read only header
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    # Get actual column names
    all_columns = df.columns.tolist()

    # Filter out columns that match the keywords
    filtered_columns = [col for col in all_columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]

    return filtered_columns




In [ ]:
import pandas as pd
import spacy

# nlp = spacy.load("en_core_web_sm")

def detect_noun(file_path):
    # Detect file type and read accordingly
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []
    column=[]
    for col in df.columns:
        column.append(col)
        # **Skip numeric columns**
        if pd.api.types.is_numeric_dtype(df[col]): continue

        # **Process only non-empty, valid string values**
        valid_values = df[col].dropna().astype(str)  # Remove NaN
        valid_values = [val for val in valid_values.head(3) if val.lower() not in ["nan", "none", "","na","null","n/a"]]

        for value in valid_values:
            doc = nlp(value)
            noun_count = sum(1 for token in doc if token.pos_ in ['NOUN', 'PROPN'])
            word_count = len(doc)

            # **Mark column as sensitive only if a high % of words are nouns**
            if word_count > 0 and (noun_count / word_count) > 0.2:
                if col not in sensitive:
                    sensitive.append(col)
    print(column)
    print(sensitive)
    return sensitive



In [ ]:
# Testing code!
# Main code
import json
import pandas as pd

mapping = {}

def csv_an(input_file, output_file):
    df = pd.read_csv(input_file, engine="python")

    sensitive_old = detect_noun(input_file)
    descrp = descriptive_cols(input_file)
    sensitive = list(set(sensitive_old) - set(descrp))
    column_counters = {col: 1 for col in df.columns if col in sensitive_old}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column

    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)
        df[col] = new_values

    for col in descrp:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                df.at[idx, col] = detect_noun_desc(val)

    df.to_csv(output_file, index=False)
    print(f"Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)


csv_an('testing-csv-ff.csv', 'masked_csv_output.csv')


['Reporting Month', 'Cluster Name', 'Input Account Name', 'Account Name', 'New Assets Issuance', 'Allocations', 'Welcome Email', 'Onboarding Trainings', 'BGV', 'NDA', 'Granting Access', 'Client Onboarding', 'Deallocation', 'Assets Return', 'Access Removal', 'Compliance % on mandatory trainings', 'Compliance % on mandatory audits', 'Timesheet Compliance %']
['Reporting Month', 'Cluster Name', 'Input Account Name', 'Account Name', 'Client Onboarding', 'Compliance % on mandatory trainings', 'Compliance % on mandatory audits', 'Timesheet Compliance %']
Anonymized file saved as masked_csv_output.csv


# **CONSOLITATING CODE**

In [ ]:
!pip install flair
!pip install faker
!pip install presidio_analyzer
!pip install presidio_anonymizer
# !python -m spacy download en_core_web_lg
!pip install --upgrade --force-reinstall "numpy==1.26.4"
!pip install --upgrade --force-reinstall "Cython" "spacy"

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Using cached Cython-3.0.12-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.3 kB)
  Using cached spacy-3.8.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy

In [ ]:
import re
import json
import time
import spacy
import openpyxl
import pandas as pd
from google.colab import files
mapping={}
nlp = spacy.load("en_core_web_sm")

def time_it(func):
  def wrapper(*args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    end = time.time()
    print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
    return result
  return wrapper

#-------------------------------------------------------------------------------------------------------------------------------
#function to find out the noun values dominating columns considering it will be a sensitive data if there is so many nouns
#------------------------------------------------------------------------------------------------------------------------------
def detect_noun(file_path):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            continue  # Skip purely numeric columns

        text_samples = df[col].astype(str).head(5)  # Take first 5 values
        noun_count = 0
        total_count = 0

        for value in text_samples:
            if "%" in value or value.replace(".", "").isdigit():
                continue  # Skip percentage or number-like values

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['NOUN', 'PROPN']:
                    noun_count += 1
                total_count += 1

        # Mark column as sensitive only if a significant portion are nouns
        if total_count > 0 and (noun_count / total_count) > 0.2:
            sensitive.append(col)

    return sensitive


#------------------------------------------------------------------------------------------
#finding out the descriptive data columns which may have sensitive data in the form of text
#------------------------------------------------------------------------------------------
def descriptive_columns(file_path):
    # Define keywords to filter out
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation",
    "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience",
    "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report",
    "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text"]

    # Ensure columns are properly loaded from CSV/Excel
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, nrows=1)  # Read only header
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path, nrows=1, engine="openpyxl")  # Read only header
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    # Get actual column names
    all_columns = df.columns.tolist()


    des=[col for col in all_columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]
    return des

#-----------------------------------------------------------------
#anonymizing descriptive values
#-----------------------------------------------------------------
def detect_noun_desc(text):
    doc = nlp(text)
    modified_text=[]
    for token in doc:
        if token.pos_ in ['PROPN']:
            modified_text.append("<sensitive>")
        else:
            modified_text.append(token.text)
    text=" ".join(modified_text)
    return text


#-------------------------------------------------------------------
#function to anonymize a excel file
#-------------------------------------------------------------------
def excel_an(input_file, output_file):
    df = pd.read_excel(input_file)  # Read as string for safety

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))

    column_counters = {col: 1 for col in df.columns if col in sensitive_old}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column
    mapping = {}

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)

        df[col] = new_values

    # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_excel(output_file, index=False, sheet_name="Anonymized Data")
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

#------------------------------------------------------
#function for de-anonymizing excel data
#------------------------------------------------------

def excel_dean(input_file, output_file, mapping_file):
    print("🔄 Loading data...")

    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    df = pd.read_excel(input_file)  # Read as string for safety
    mapping_keys = set(mapping.keys())
    df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)
    df.to_excel(output_file, index=False, sheet_name="De-anonymized Data")

    print(f"✅ De-anonymized file saved as {output_file}")

#----------------------------------------------------------
#function for anonymizing csv data
#----------------------------------------------------------
def csv_an(input_file, output_file):
    df = pd.read_csv(input_file, engine="python")

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))

    column_counters = {col: 1 for col in df.columns if col in sensitive_old}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)

        df[col] = new_values

    # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_csv(output_file, index=False)
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

#------------------------------------------------------------
#function for de anonymizing csv data
#------------------------------------------------------------
def csv_dean(input_file, output_file, mapping_file):
    print("🔄 Loading data...")

    with open(mapping_file, "r") as f:
        mapping = json.load(f)
    df = pd.read_csv(input_file, engine="python", dtype=str)  # Read as string for safety
    mapping_keys = set(mapping.keys())
    df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)
    df.to_csv(output_file, index=False)

    print(f"✅ De-anonymized file saved as {output_file}")


#--------------------------------------------------------------
#function to determine whcih type of data is need to perform
#--------------------------------------------------------------
@time_it
def anonymization(input_file):
    if input_file.endswith(".csv"):
        csv_an(input_file,"intermediate.csv")
        csv_dean("intermediate.csv","deanonymized.csv","mappings.json")
    elif input_file.endswith(".xlsx"):
        excel_an(input_file,"intermediate.xlsx")
        excel_dean("intermediate.xlsx","deanonymized.xlsx","mappings.json")

uploaded = files.upload()
file_name = list(uploaded.keys())[0].strip().replace(' ', '-')
anonymization(file_name)

Saving testing-csv-ff.csv to testing-csv-ff.csv
✅ Anonymized file saved as intermediate.csv
🔄 Loading data...
✅ De-anonymized file saved as deanonymized.csv


Execution time anonymization: 0.533583 seconds


<ipython-input-8-c92c682ea52b>:220: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)


In [ ]:
# Testing simplified consolidated code
import re
import json
import time
import spacy
import pandas as pd
from google.colab import files
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

mapping = {}
nlp = spacy.load("en_core_web_sm")

analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

def read_file(file_path):
    """Read CSV or Excel file and return DataFrame."""
    if file_path.endswith(".csv"):
        return pd.read_csv(file_path, engine="python"), "csv"
    elif file_path.endswith(".xlsx"):
        return pd.read_excel(file_path, engine="openpyxl"), "excel"
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

def detect_noun(df):
    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64'] or re.search(r'\b(id|rate|amount|price|cost|value|score|percentage)\b', col, re.IGNORECASE):
            continue  # Skip numeric and financial data

        text_samples = df[col].astype(str).head(10)
        noun_count, total_count = 0, 0

        for value in text_samples:
            if value.replace(".", "").isdigit() or '%' in value:
                continue

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['PROPN', 'NOUN']:
                    noun_count += 1
                total_count += 1

        if total_count > 0 and (noun_count / total_count) > 0.5:
            sensitive.append(col)

    print('\n\nSensitive cols:', sensitive)
    return sensitive

def detect_id_columns(df):
    return [col for col in df.columns if re.search(r'\b(id)$\b', col, re.IGNORECASE)]

def descriptive_columns(df):
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation", "reviews", "feedback", "history"]
    return [col for col in df.columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]

def detect_pii(text):
    results = analyzer.analyze(text=text, entities=["PHONE_NUMBER", "CREDIT_CARD"], language="en")
    return anonymizer.anonymize(text=text, analyzer_results=results).text if results else text

def detect_pii_columns(df):
    """Detect columns that contain PII data (phone, credit card, ID)."""
    pii_columns = []

    for col in df.columns:
        sample_texts = df[col].astype(str).head(10).dropna().tolist()
        for text in sample_texts:
            if analyzer.analyze(text=text, entities=["PHONE_NUMBER", "CREDIT_CARD"], language="en"):
                pii_columns.append(col)
                break

    print("\n\nPII Columns:", pii_columns)
    return pii_columns

def anonymize_data(df):
    sensitive = detect_noun(df)
    id_cols = detect_id_columns(df)
    desc = descriptive_columns(df)
    pii_cols = detect_pii_columns(df)
    all_sensitive_cols = set(sensitive + id_cols + pii_cols)  # Combine all detected sensitive columns

    column_counters = {col: 1 for col in all_sensitive_cols}
    column_mappings = {col: {} for col in all_sensitive_cols}

    for col in all_sensitive_cols:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value
                    column_counters[col] += 1
                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)
        df[col] = new_values

    for col in desc:
        df[col] = df[col].astype(str).apply(detect_pii)  # PII masking for descriptive columns

    return df

def save_file(df, file_type, output_file):
    if file_type == "csv":
        df.to_csv(output_file, index=False)
    else:
        df.to_excel(output_file, index=False, sheet_name="Anonymized Data")

@time_it
def anonymization(input_file):
    df, file_type = read_file(input_file)
    df = anonymize_data(df)
    output_file = f"anonymized.{file_type}"
    save_file(df, file_type, output_file)
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)
    print(f"✅ Anonymized file saved as {output_file}")

@time_it
def de_anonymization(input_file, mapping_file):
    df, file_type = read_file(input_file)
    with open(mapping_file, "r") as f:
        mapping = json.load(f)
    df = df.applymap(lambda x: mapping.get(x, x))
    output_file = f"deanonymized.{file_type}"
    save_file(df, file_type, output_file)
    print(f"✅ De-anonymized file saved as {output_file}")

# uploaded = files.upload()
# file_name = list(uploaded.keys())[0].strip().replace(' ', '-')
# anonymization(file_name)
anonymization(input_file = 'testing-csv-ff.csv')
de_anonymization(file_name, "mappings.json")




Sensitive cols: ['Reporting Month', 'Cluster Name', 'Input Account Name', 'Account Name']


PII Columns: ['Mobile Number', 'Credit Card Number']
✅ Anonymized file saved as anonymized.csv


Execution time anonymization: 4.649121 seconds


<ipython-input-38-7c67c9f901aa>:138: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping.get(x, x))


✅ De-anonymized file saved as deanonymized.csv


Execution time de_anonymization: 0.777546 seconds


In [ ]:
# Masking prompt from values in mappings.json
import json

def mask_prompt(prompt, mapping_file="mappings.json"):
    """Replaces original values in the prompt with masked values from mappings.json."""
    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    reversed_mapping = {v: k for k, v in mapping.items()}  # Reverse mapping for lookup
    for original, masked in reversed_mapping.items():
        if original in prompt:
            prompt = prompt.replace(original, masked)

    return prompt

def unmask_response(response, mapping_file="mappings.json"):
    """Replaces masked values in the response with original values from mappings.json."""
    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    for masked, original in mapping.items():
        if masked in response:
            response = response.replace(masked, original)

    return response

prompt_1 = "Please analyze the sales performance for Gamma Inc in Mar-24. The key account manager is Alex Brown, and their contact number is 7654321098. Employee ID 35678 was responsible for managing this account. Identify any trends or anomalies."
prompt_2 = "For the North cluster, examine the transactions linked to Liam Johnson in Apr-24. His registered mobile number is 5432109876, and the associated credit card number is 4539876543210987. Provide insights on spending patterns and any unusual activities."
prompt_3 = "Check the financial records for Theta Corp in Jul-24. The primary contact is William Young, and his phone number is 7654321096. Employee ID 55667 handled this account. Assess if there were any fraudulent transactions."

response_1 = mask_prompt(prompt_1)

print(response_1)
print(unmask_response(response_1))

Please analyze the sales performance for Input Account Name3 in Reporting Month3. The key account manager is Account Name3, and their contact number is Mobile Number3. Employee ID Employee ID3 was responsible for managing this account. Identify any trends or anomalies.
Please analyze the sales performance for Gamma Inc in Mar-24. The key account manager is Alex Brown, and their contact number is 7654321098. Employee ID 35678 was responsible for managing this account. Identify any trends or anomalies.


In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.6 MB/s eta 0:00:00


In [ ]:
# Implementing using Faker
import re
import json
import time
import spacy
import pandas as pd
from faker import Faker
from google.colab import files
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

# Initialize dependencies
fake = Faker()
nlp = spacy.load("en_core_web_sm")

analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

mapping = {}  # Stores original-to-anonymized values for de-anonymization

def time_it(func):
    """Decorator to measure execution time."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

def read_file(file_path):
    """Read CSV or Excel file and return DataFrame."""
    if file_path.endswith(".csv"):
        return pd.read_csv(file_path, engine="python"), "csv"
    elif file_path.endswith(".xlsx"):
        return pd.read_excel(file_path, engine="openpyxl"), "excel"
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

def detect_noun(df):
    """Detect columns that contain mostly noun-based text data."""
    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64'] or re.search(r'\b(id|rate|amount|price|cost|value|score|percentage)\b', col, re.IGNORECASE):
            continue  # Skip numeric and financial data

        text_samples = df[col].astype(str).head(10)
        noun_count, total_count = 0, 0

        for value in text_samples:
            if value.replace(".", "").isdigit() or '%' in value:
                continue

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['PROPN', 'NOUN']:
                    noun_count += 1
                total_count += 1

        if total_count > 0 and (noun_count / total_count) > 0.5:
            sensitive.append(col)

    print('\n\nSensitive cols:', sensitive)
    return sensitive

def detect_id_columns(df):
    """Detect columns that contain ID values."""
    return [col for col in df.columns if re.search(r'\b(id)$\b', col, re.IGNORECASE)]

def descriptive_columns(df):
    """Detect columns that contain descriptive or free-text data."""
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation", "reviews", "feedback", "history"]
    return [col for col in df.columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]

def detect_pii(text):
    """Detect and anonymize PII data in text fields."""
    results = analyzer.analyze(text=text, entities=["PHONE_NUMBER", "CREDIT_CARD"], language="en")
    return anonymizer.anonymize(text=text, analyzer_results=results).text if results else text

def detect_pii_columns(df):
    """Detect columns that contain PII data (phone, credit card, etc.)."""
    pii_columns = []

    for col in df.columns:
        sample_texts = df[col].astype(str).head(10).dropna().tolist()
        for text in sample_texts:
            if analyzer.analyze(text=text, entities=["PHONE_NUMBER", "CREDIT_CARD"], language="en"):
                pii_columns.append(col)
                break

    print("\n\nPII Columns:", pii_columns)
    return pii_columns

# Mapping Faker functions to relevant column types
faker_mapping = {
    "name": fake.name,
    "email": fake.email,
    "phone": fake.phone_number,
    "address": fake.address,
    "uuid": fake.uuid4,
    "credit_card": fake.credit_card_number,
}

def get_faker_value(column_name):
    """Return a Faker-generated value based on column type."""
    if re.search(r'\b(name)\b', column_name, re.IGNORECASE):
        return faker_mapping["name"]()
    if re.search(r'\b(email)\b', column_name, re.IGNORECASE):
        return faker_mapping["email"]()
    if re.search(r'\b(phone|mobile|contact)\b', column_name, re.IGNORECASE):
        return faker_mapping["phone"]()
    if re.search(r'\b(address|location)\b', column_name, re.IGNORECASE):
        return faker_mapping["address"]()
    if re.search(r'\b(id)\b', column_name, re.IGNORECASE):
        return faker_mapping["uuid"]()
    if re.search(r'\b(credit_card|card_number|credit card)\b', column_name, re.IGNORECASE):
        return faker_mapping["credit_card"]()
    return fake.word()  # Default fallback

def anonymize_data(df):
    """Anonymize sensitive columns using Faker."""
    sensitive = detect_noun(df)
    id_cols = detect_id_columns(df)
    desc = descriptive_columns(df)
    pii_cols = detect_pii_columns(df)
    all_sensitive_cols = set(sensitive + id_cols + pii_cols)

    column_mappings = {col: {} for col in all_sensitive_cols}

    for col in all_sensitive_cols:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]
                else:
                    anonymized_value = get_faker_value(col)
                    column_mappings[col][val] = anonymized_value
                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)
        df[col] = new_values

    for col in desc:
        df[col] = df[col].astype(str).apply(detect_pii)  # Mask PII in descriptive text

    return df

def save_file(df, file_type, output_file):
    """Save the DataFrame to a file."""
    if file_type == "csv":
        df.to_csv(output_file, index=False)
    else:
        df.to_excel(output_file, index=False, sheet_name="Anonymized Data")

@time_it
def anonymization(input_file):
    """Perform anonymization on a file."""
    df, file_type = read_file(input_file)
    df = anonymize_data(df)
    output_file = f"anonymized.{file_type}"
    save_file(df, file_type, output_file)

    # Save mapping for de-anonymization
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

    print(f"✅ Anonymized file saved as {output_file}")

@time_it
def de_anonymization(input_file, mapping_file):
    """Revert anonymized data to its original values using mappings."""
    df, file_type = read_file(input_file)

    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    df = df.applymap(lambda x: mapping.get(x, x))
    output_file = f"deanonymized.{file_type}"
    save_file(df, file_type, output_file)

    print(f"✅ De-anonymized file saved as {output_file}")

uploaded = files.upload()
file_name = list(uploaded.keys())[0].strip().replace(' ', '-')
anonymization(file_name)
# anonymization("testing-csv-ff.csv")
de_anonymization("anonymized.csv", "mappings.json")


Saving testing-csv-ff.csv to testing-csv-ff.csv


Sensitive cols: ['Reporting Month', 'Cluster Name', 'Input Account Name', 'Account Name']


PII Columns: ['Mobile Number', 'Credit Card Number']
✅ Anonymized file saved as anonymized.csv


Execution time anonymization: 2.413634 seconds
✅ De-anonymized file saved as deanonymized.csv


Execution time de_anonymization: 0.007664 seconds


<ipython-input-3-322bf1fc318e>:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping.get(x, x))


# **TESTING FAKER CODE**

In [ ]:
!pip install flair
!pip install faker
!pip install presidio_analyzer
!pip install presidio_anonymizer
# !python -m spacy download en_core_web_lg
!pip install --upgrade --force-reinstall "numpy==1.26.4"
!pip install --upgrade --force-reinstall "Cython" "spacy"

In [ ]:
# Testing Faker code (1)
import pandas as pd
from presidio_analyzer import AnalyzerEngine
from faker import Faker

def time_it(func):
    """Decorator to measure execution time."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
  entity_columns = {}  # Initialize as a dictionary
  for col in df.columns:
    unique_values = df[col].dropna().astype(str).unique()[:25]
    entity_counts = {}

    for value in unique_values:
      results = analyzer.analyze(text=value, language="en")
      for result in results:
        entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1
    if entity_counts:
      predominant_entity = max(entity_counts, key=entity_counts.get)
      if predominant_entity not in entity_columns:
        entity_columns[predominant_entity] = []  # Assign a list to the entity key
      entity_columns[predominant_entity].append(col)
  return entity_columns  # Return the dictionary

def load_and_analyze(file_path):
  if file_path.endswith(".csv"):
    df = pd.read_csv(file_path, engine="python")
  elif file_path.endswith((".xls", ".xlsx")):
    df = pd.read_excel(file_path)
  else:
    raise ValueError("Unsupported file format. Provide a CSV or Excel file.")

  df_sample = df.drop_duplicates().head()
  classified_columns = analyze_column(df_sample)
  return classified_columns  # Return the dictionary

In [ ]:
# Testing Faker code (2)
file="smaller_companies.csv"
classified_columns=load_and_analyze(file)
df=pd.read_csv(file)
print(classified_columns)



Execution time analyze_column: 0.468736 seconds
{'DATE_TIME': ['id', 'size range'], 'LOCATION': ['name', 'locality', 'country'], 'URL': ['domain', 'linkedin url'], 'US_DRIVER_LICENSE': ['current employee estimate', 'total employee estimate']}


In [ ]:
# Testing Faker code (3)
import os
import json
from faker import Faker

fake = Faker()
Faker.seed(42)

# data_mapping.json
MAPPING_FILE = "output/mappings.json"
forward_mapping = {}
reverse_mapping = {}

@time_it
def load_mapping():
    """Loads existing mappings from a file if available."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                # Convert keys back to tuples if needed
                forward_mapping = {tuple(eval(k)): v for k, v in data.get("forward_mapping", {}).items()}
                reverse_mapping = {tuple(eval(k)): v for k, v in data.get("reverse_mapping", {}).items()}
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            # If the file is invalid, reset the mappings
            forward_mapping = {}
            reverse_mapping = {}

def save_mapping():
    """Saves the mapping to a JSON file."""
    os.makedirs(os.path.dirname(MAPPING_FILE), exist_ok=True)
    # Convert tuple keys to strings before saving
    modified_forward_mapping = {str(k): v for k, v in forward_mapping.items()}
    modified_reverse_mapping = {str(k): v for k, v in reverse_mapping.items()}
    with open(MAPPING_FILE, "w") as f:
        json.dump({"forward_mapping": modified_forward_mapping, "reverse_mapping": modified_reverse_mapping}, f, indent=4)

@time_it
def generate_fake_value(original: str, entity_type: str) -> str:
    """Generates fake values while ensuring uniqueness."""
    if original in forward_mapping:
        return forward_mapping[original]

    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": fake.credit_card_number,
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        # "DATE": fake.date_of_birth,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
    }

    fake_value = faker_mapping.get(entity_type, lambda: original)()
    forward_mapping[original] = fake_value
    reverse_mapping[fake_value] = original
    return fake_value

@time_it
def mask_sensitive_data(df: pd.DataFrame, classified_columns: dict) -> pd.DataFrame:
    """Masks sensitive columns using Faker based on classified entity types."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df[column] = df[column].astype(str).apply(lambda x: generate_fake_value(x, entity_type) if x.strip() else x)
    return df

# testing-csv-ff.csv
# smaller_companies.csv
if __name__ == "__main__":
    load_mapping()
    input_file = "smaller_companies.csv"
    output_file = "output/anonymized.csv"

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze("smaller_companies.csv")

    df = mask_sensitive_data(df, classified_columns)
    df.to_csv(output_file, index=False)
    save_mapping()

In [ ]:
# Hashing file so as to reuse it (FAKER)
import os
import json
import pandas as pd
import hashlib
from faker import Faker

fake = Faker()
Faker.seed(42)

MAPPING_FILE = "mappings.json"  # Store mappings directly
N_LINES_FOR_HASH = 100  # Number of lines to consider for hashing

# Load existing mappings if available
if os.path.exists(MAPPING_FILE):
    with open(MAPPING_FILE, "r") as f:
        try:
            mappings_data = json.load(f)
        except json.JSONDecodeError:
            mappings_data = {}
else:
    mappings_data = {}

def compute_file_hash(file_path, num_lines=N_LINES_FOR_HASH):
    """Compute a quick hash based on the first `num_lines` of the file."""
    hasher = hashlib.sha256()
    with open(file_path, "r", encoding="utf-8") as f:
        for _ in range(num_lines):
            line = f.readline()
            if not line:
                break
            hasher.update(line.encode("utf-8"))
    return hasher.hexdigest()

def save_mapping(filename, file_hash, forward_mapping, reverse_mapping):
    """Save mappings along with filename and file hash."""
    mappings_data[filename] = {
        "file_hash": file_hash,
        "forward_mapping": forward_mapping,
        "reverse_mapping": reverse_mapping
    }
    with open(MAPPING_FILE, "w") as f:
        json.dump(mappings_data, f, indent=4)

def load_mapping(filename, file_hash):
    """Load mapping if filename and file hash match."""
    if filename in mappings_data and mappings_data[filename]["file_hash"] == file_hash:
        return mappings_data[filename]["forward_mapping"], mappings_data[filename]["reverse_mapping"]
    return {}, {}

def generate_fake_value(original: str, entity_type: str, forward_mapping, reverse_mapping):
    """Generates fake values while ensuring uniqueness."""
    if original in forward_mapping:
        return forward_mapping[original]

    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": fake.credit_card_number,
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
    }

    fake_value = faker_mapping.get(entity_type, lambda: original)()
    forward_mapping[original] = fake_value
    reverse_mapping[fake_value] = original
    return fake_value

def mask_sensitive_data(df: pd.DataFrame, classified_columns: dict, forward_mapping, reverse_mapping):
    """Masks sensitive columns using Faker based on classified entity types."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df[column] = df[column].astype(str).apply(lambda x: generate_fake_value(x, entity_type, forward_mapping, reverse_mapping) if x.strip() else x)
    return df

# Run masking process
if __name__ == "__main__":
    input_file = "smaller_companies.csv"
    output_file = "anonymized.csv"

    # Compute file hash
    file_hash = compute_file_hash(input_file)

    # Load or initialize mappings
    forward_mapping, reverse_mapping = load_mapping(input_file, file_hash)

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze(input_file)  # Ensure this function is defined

    df = mask_sensitive_data(df, classified_columns, forward_mapping, reverse_mapping)
    df.to_csv(output_file, index=False)

    # Save updated mapping
    save_mapping(input_file, file_hash, forward_mapping, reverse_mapping)


In [ ]:
# Tried out finding total unique names
from faker import Faker
from faker.exceptions import UniquenessException

fake = Faker()
fake.unique.clear()
unique_names = set()

try:
  while True:
    unique_names.add(fake.unique.name())
except UniquenessException:
  print(f'Unique names length:', len(unique_names))

In [ ]:
# Masking ID in column headers

import os
import re
import time
import json
import pandas as pd
from faker import Faker
from presidio_analyzer import AnalyzerEngine

# Presidio Analyzer
analyzer = AnalyzerEngine()

# Faker initalization
fake = Faker()
Faker.seed(42)

# mappings.json
MAPPING_FILE = "mappings.json"
forward_mapping = {}
reverse_mapping = {}

def time_it(func):
    """Decorator to measure execution time."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
  entity_columns = {}  # Initialize as a dictionary
  for col in df.columns:
    unique_values = df[col].dropna().astype(str).unique()[:15]
    entity_counts = {}

    for value in unique_values:
      results = analyzer.analyze(text=value, language="en")
      for result in results:
        entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1
    if entity_counts:
      predominant_entity = max(entity_counts, key=entity_counts.get)
      if predominant_entity not in entity_columns:
        entity_columns[predominant_entity] = []  # Assign a list to the entity key
      entity_columns[predominant_entity].append(col)
  return entity_columns  # Return the dictionary

def load_and_analyze(file_path):
  if file_path.endswith(".csv"):
    df = pd.read_csv(file_path, engine="python")
  elif file_path.endswith((".xls", ".xlsx")):
    df = pd.read_excel(file_path)
  else:
    raise ValueError("Unsupported file format. Provide a CSV or Excel file.")

  df_sample = df.drop_duplicates().head()
  classified_columns = analyze_column(df_sample)
  return classified_columns

def load_mapping():
    """Loads existing mappings from a file if available."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            forward_mapping = {}
            reverse_mapping = {}

def save_mapping():
    """Saves the mapping to a JSON file."""
    with open(MAPPING_FILE, "w") as f:
        json.dump({"forward_mapping": forward_mapping, "reverse_mapping": reverse_mapping}, f, indent=4)

def generate_fake_value(original: str, entity_type: str) -> str:
    """Generates fake values while ensuring uniqueness."""
    if original in forward_mapping:
        return forward_mapping[original]

    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": fake.credit_card_number,
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
        "ID": fake.uuid4,  # Generating fake values for ID columns
    }

    fake_value = faker_mapping.get(entity_type, lambda: original)()
    forward_mapping[original] = fake_value
    reverse_mapping[fake_value] = original
    return fake_value

@time_it
def mask_sensitive_data(df: pd.DataFrame, classified_columns: dict) -> pd.DataFrame:
    """Masks sensitive columns using Faker based on classified entity types."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df[column] = df[column].astype(str).apply(lambda x: generate_fake_value(x, entity_type) if x.strip() else x)

    # Find columns with 'id' in the name using regex and mask them
    id_columns = [col for col in df.columns if re.search(r'\bid\b', col, re.IGNORECASE)]
    print('ID cols:', id_columns)
    for col in id_columns:
        df[col] = df[col].astype(str).apply(lambda x: generate_fake_value(x, "ID") if x.strip() else x)

    return df


# smaller_companies.csv
# testing-csv-ff.csv
if __name__ == "__main__":
    load_mapping()
    input_file = "smaller_companies.csv"
    output_file = "anonymized.csv"

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze(input_file)

    df = mask_sensitive_data(df, classified_columns)
    df.to_csv(output_file, index=False)
    save_mapping()


In [ ]:
# Testing columnwise replacing using Faker
import os
import re
import time
import json
import pandas as pd
from faker import Faker
from presidio_analyzer import AnalyzerEngine

# Initialize Faker & Presidio Analyzer
fake = Faker()
Faker.seed(42)
analyzer = AnalyzerEngine()

# Mapping file for storing forward/reverse mappings
MAPPING_FILE = "mappings.json"
forward_mapping = {}
reverse_mapping = {}

def time_it(func):
    """Decorator to measure execution time."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

def load_mapping():
    """Loads existing mappings from a file if available."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            forward_mapping = {}
            reverse_mapping = {}

def save_mapping():
    """Saves the mapping to a JSON file."""
    with open(MAPPING_FILE, "w") as f:
        json.dump({"forward_mapping": forward_mapping, "reverse_mapping": reverse_mapping}, f, indent=4)

@time_it
def analyze_column(df):
    """Detects sensitive columns using Presidio."""
    entity_columns = {}
    for col in df.columns:
        unique_values = df[col].dropna().astype(str).unique()[:10]
        entity_counts = {}

        for value in unique_values:
            results = analyzer.analyze(text=value, language="en")
            for result in results:
                entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

        if entity_counts:
            predominant_entity = max(entity_counts, key=entity_counts.get)
            if predominant_entity not in entity_columns:
                entity_columns[predominant_entity] = []
            entity_columns[predominant_entity].append(col)

    return entity_columns

def load_and_analyze(file_path):
    """Loads CSV/Excel and analyzes sensitive columns."""
    df = pd.read_csv(file_path) if file_path.endswith(".csv") else pd.read_excel(file_path)
    return analyze_column(df)

def generate_fake_values(df, entity_type, column):
    """Generates fake values for an entire column while preserving uniqueness."""
    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": lambda: fake.credit_card_number(card_type=None),
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
        "ID": fake.uuid4,
        "US_BANK_NUMBER": lambda: fake.credit_card_number(card_type="visa")
    }

    if entity_type not in faker_mapping:
        return df  # Skip if no Faker function exists

    faker_func = faker_mapping[entity_type]

    # Convert numeric columns to strings before applying Faker mapping
    if df[column].dtype in [int, float]:
        df[column] = df[column].astype(str)

    original_values = df[column].dropna().astype(str).unique()
    fake_values = [forward_mapping.get(value, faker_func()) for value in original_values]

    # Store mappings
    for original, fake_val in zip(original_values, fake_values):
        forward_mapping[original] = str(fake_val)  # Ensure JSON serializable
        reverse_mapping[str(fake_val)] = original

    df[column] = df[column].map(forward_mapping).fillna(df[column])

    return df



def mask_id_columns(df):
    """Finds columns with 'id' in the name and replaces them with fake UUIDs."""
    id_columns = [col for col in df.columns if re.search(r'\bid\b', col, re.IGNORECASE)]

    for column in id_columns:
        unique_ids = df[column].astype(str).dropna().unique()
        fake_ids = [forward_mapping.get(value, str(fake.uuid4())) for value in unique_ids]

        # Store mappings for ID replacements
        for original, fake_id in zip(unique_ids, fake_ids):
            forward_mapping[original] = fake_id
            reverse_mapping[fake_id] = original

        # 🔹 Forcefully update df[column] with mapped values
        df[column] = df[column].astype(str).apply(lambda x: forward_mapping.get(x, x))

    return df


@time_it
def mask_sensitive_data(df, classified_columns):
    """Masks all detected sensitive columns using Faker."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df = generate_fake_values(df, entity_type, column)

    # Mask all ID columns separately
    df = mask_id_columns(df)

    return df

# testing-csv-ff.csv
# smaller_companies.csv
if __name__ == "__main__":
    load_mapping()
    input_file = "smaller_100k_companies.csv"
    output_file = "anonymized.csv"

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze(input_file)

    df = mask_sensitive_data(df, classified_columns)
    df.to_csv(output_file, index=False)

    save_mapping()



Execution time analyze_column: 6.428666 seconds

Execution time mask_sensitive_data: 67.272321 seconds


In [ ]:
# Thejus code fast!
output_folder = "output_files"
class MaskingModule:
    def __init__(self, mapping_file: str = f"{output_folder}/data_mapping.json", seed: int = 42):
        self.fake = Faker()
        Faker.seed(seed)
        self.mapping_file = mapping_file
        self.forward_mapping = {}
        self.reverse_mapping = {}
        self._load_mapping_if_exists()
        print("✅ MaskingModule initialized.")

    def _generate_fake_column(self, column: pd.Series, column_name: str) -> pd.Series:
        """Generate fake data for a whole column at once and update mappings."""
        column_lower = column_name.lower()
        original_values = column.dropna().unique()
        fake_values = []

        if (
            "employee" in column_lower
            or "company" in column_lower
            or "account" in column_lower
        ):
            fake_values = [
                f"{self.fake.first_name()}{self.fake.random_number(digits=12)}{self.fake.first_name()}"
                for _ in original_values
            ]
        elif "project" in column_lower:
            fake_values = [
                f"PRJ-{self.fake.random_number(digits=12)}" for _ in original_values
            ]
        elif "entity" in column_lower:
            fake_values = [
                f"ENT-{self.fake.random_number(digits=12)}" for _ in original_values
            ]
        elif "program" in column_lower:
            fake_values = [
                f"PROG-{self.fake.random_number(digits=12)}" for _ in original_values
            ]
        else:
            fake_values = [
                f"{self.fake.first_name()}-{self.fake.random_number(digits=20)}-{self.fake.first_name()}"
                for _ in original_values
            ]

        mapping = dict(zip(original_values, fake_values))
        self.forward_mapping.update(mapping)
        self.reverse_mapping.update({v: k for k, v in mapping.items()})

        return column.map(mapping).fillna(column)

    def _save_mapping(self):
        """Save mappings to JSON with string keys and serializable values."""

        def convert_to_serializable(obj):
            if isinstance(
                obj, (np.integer, np.int64, np.int32)
            ):  # Convert numpy int to Python int
                return int(obj)
            elif isinstance(
                obj, (np.floating, np.float64, np.float32)
            ):  # Convert numpy float to Python float
                return float(obj)
            elif isinstance(obj, np.ndarray):  # Convert numpy array to list
                return obj.tolist()
            else:
                return obj  # Return as-is if already serializable

        mapping_data = {
            "forward_mapping": {
                str(k): convert_to_serializable(v) for k, v in self.forward_mapping.items()
            },
            "reverse_mapping": {
                str(k): convert_to_serializable(v) for k, v in self.reverse_mapping.items()
            },
            "metadata": {
                "updated_at": datetime.now().isoformat(),
                "record_count": len(self.forward_mapping),
            },
        }

        with open(self.mapping_file, "w") as f:
            json.dump(mapping_data, f, indent=4)

        print("💾 Mapping saved successfully.")

    def _load_mapping_if_exists(self):
        """Load existing mappings from JSON."""
        if os.path.exists(self.mapping_file):
            with open(self.mapping_file, "r") as f:
                data = json.load(f)
                self.forward_mapping = data["forward_mapping"]
                self.reverse_mapping = data["reverse_mapping"]
            print("📂 Existing mapping loaded.")

    def process_file(
        self,
        input_file: str,
        output_file: str,
        mode: str = "anonymize",
        include_columns: List[str] = None,
    ):
        """Process CSV and Excel files for anonymization and de-anonymization."""
        if mode not in ["anonymize", "deanonymize"]:
            raise ValueError("Mode must be either 'anonymize' or 'deanonymize'")

        file_ext = os.path.splitext(input_file)[-1].lower()
        df = (
            pd.read_csv(input_file) if file_ext == ".csv" else pd.read_excel(input_file)
        )
        print("📊 Data loaded successfully.")

        include_set = {"id", "name", "domain", "year", "founded", "industry", "size"}
        if include_columns:
            include_set.update(include_columns)

        for column in df.columns:
            if column in include_set:
                if mode == "anonymize":
                    df[column] = self._generate_fake_column(df[column], column)
                else:
                    df[column] = (
                        df[column]
                        .astype(str)
                        .map(self.reverse_mapping)
                        .where(df[column].notna(), df[column])
                    )

        df.to_csv(output_file, index=False) if file_ext == ".csv" else df.to_excel(
            output_file, index=False
        )
        if mode == "anonymize":
            self._save_mapping()
        print(f"✅ Data {mode}d and saved to {output_file}")

    def check_if_two_files_are_same(self, input_file: str, restored_file: str):
        """Check if the original and restored files match."""
        file_ext = os.path.splitext(input_file)[-1].lower()
        original_df = (
            pd.read_csv(input_file) if file_ext == ".csv" else pd.read_excel(input_file)
        )
        restored_df = (
            pd.read_csv(restored_file)
            if file_ext == ".csv"
            else pd.read_excel(restored_file)
        )

        test = original_df.equals(restored_df)
        print(f"📊 Are files identical? {test}")
        if not test:
            print("⚠️ Files are not identical!")
            print("🔍 Sample from original:")
            print(original_df.head(2))
            print("🔍 Sample from restored:")
            print(restored_df.head(2))
        else:
            print("✅ Files match perfectly!")

    def print_mapping_sample(self, n: int = 5):
        """Print sample mappings."""
        print("📌 Forward Mapping (Original → Anonymized):")
        for k, v in list(self.forward_mapping.items())[:n]:
            print(f"{k} → {v}")
        print("📌 Reverse Mapping (Anonymized → Original):")
        for k, v in list(self.reverse_mapping.items())[:n]:
            print(f"{k} → {v}")

    def mask_text(self, text, replace_dict):
        """Mask text using forward mapping."""
        for key, value in replace_dict.items():
            text = text.replace(key, value)
        return text



In [ ]:
# Optimizing Thejus's logic
import os
import re
import json
import time
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime
from google.colab import files
from presidio_analyzer import AnalyzerEngine

# Initialize Faker & Presidio Analyzer
fake = Faker()
Faker.seed(42)
analyzer = AnalyzerEngine()

# Mapping file for storing forward/reverse mappings
MAPPING_FILE = "mappings.json"
forward_mapping = {}
reverse_mapping = {}

NUMERIC_PATTERN = re.compile(r"^[<>]?[\d,.%]+$")
SENSITIVE_HEADERS = {"phone", "mobile", "credit", "card", "id"}
EXCLUDED_ENTITIES = {"DATE_TIME"}  # Exclude unwanted entities


def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper


def load_mapping():
    """Loads existing mapping from file, ensuring proper data handling."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
            print("📂 Existing mapping loaded.")
        except json.JSONDecodeError as e:
            print(f"⚠️ Error decoding JSON file: {e}")


def save_mapping():
    """Saves the mapping to a JSON file with string keys."""
    mapping_data = {
        "forward_mapping": {str(k): str(v) for k, v in forward_mapping.items()},
        "reverse_mapping": {str(k): str(v) for k, v in reverse_mapping.items()},
        "metadata": {
            "updated_at": datetime.now().isoformat(),
            "record_count": len(forward_mapping),
        },
    }
    with open(MAPPING_FILE, "w") as f:
        json.dump(mapping_data, f, indent=4)
    print("💾 Mapping saved successfully.")



@time_it
def analyze_column(df):
    """Uses Presidio to classify columns based on detected entity types."""
    entity_columns = {}
    for col in df.columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in SENSITIVE_HEADERS):
            entity_columns.setdefault("SENSITIVE", []).append(col)
            continue

        unique_values = df[col].dropna().astype(str).unique()[:10]
        entity_counts = {}

        for value in unique_values:
            if NUMERIC_PATTERN.match(value):
                continue  # Skip purely numeric values unless it's a phone/credit card

            results = analyzer.analyze(text=value, language="en")
            for result in results:
                if result.entity_type in EXCLUDED_ENTITIES:
                    continue  # Skip unwanted entities
                entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

        if entity_counts:
            predominant_entity = max(entity_counts, key=entity_counts.get)
            if predominant_entity not in entity_columns:
                entity_columns[predominant_entity] = []
            entity_columns[predominant_entity].append(col)

    return entity_columns


def generate_fake_values(column):
    """Generates consistent fake values for a column while preserving uniqueness."""
    col_name = column.name.lower()
    original_values = column.dropna().unique()

    if col_name in forward_mapping:
        mapping = forward_mapping[col_name]
    else:
        mapping = {}

    fake_values = []
    if re.search(r"\b(phone|mobile)\b", col_name, re.IGNORECASE):
        fake_values = [fake.phone_number() for _ in original_values]
    elif re.search(r"\b(credit|card)\b", col_name, re.IGNORECASE):
        fake_values = [fake.credit_card_number() for _ in original_values]
    elif re.search(r"\b(id|identifier)\b", col_name, re.IGNORECASE):
        fake_values = [fake.bothify(text="ID##########????") for _ in original_values]
    elif re.search(r"\b(name|full[_\s]?name|first[_\s]?name|last[_\s]?name)\b", col_name, re.IGNORECASE):
        fake_values = [f'{fake.name()--{fake.bothify(text="??????????")}}' for _ in original_values]
    else:
        fake_values = [fake.bothify(text="????########") for _ in original_values]


    mapping.update(dict(zip(original_values, fake_values)))
    forward_mapping[col_name] = mapping
    reverse_mapping[col_name] = {v: k for k, v in mapping.items()}

    return column.map(mapping).fillna(column)


@time_it
def mask_columnwise(df):
    """Replaces sensitive data only in columns detected as sensitive by Presidio."""
    sensitive_columns = analyze_column(df)

    for entity, cols in sensitive_columns.items():
        for column in cols:
            print(f"🔒 Masking sensitive column: {column} (Detected as {entity})")
            df[column] = generate_fake_values(df[column])

    return df


def restore_data(df):
    """Restores original values using stored mappings."""
    for column in df.columns:
        col_name = column.lower()
        if col_name in reverse_mapping:
            df[column] = df[column].astype(str).map(reverse_mapping[col_name]).fillna(df[column])
    return df


def check_if_files_match(original_file, restored_file):
    """Checks if the original and restored files match."""
    file_ext = os.path.splitext(original_file)[-1].lower()
    original_df = pd.read_csv(original_file) if file_ext == ".csv" else pd.read_excel(original_file)
    restored_df = pd.read_csv(restored_file) if file_ext == ".csv" else pd.read_excel(restored_file)
    test = original_df.equals(restored_df)
    print(f"📊 Are files identical? {test}")
    if not test:
        print("⚠️ Files are not identical!")
    else:
        print("✅ Files match perfectly!")


if __name__ == "__main__":
    load_mapping()
    uploaded = files.upload()
    input_file = list(uploaded.keys())[0].strip().replace(' ', '-')
    # input_file = "smaller_companies.csv" if not input_file else input_file

    anonymized_file = "anonymized.csv"
    restored_file = "restored.csv"

    df = pd.read_csv(input_file)
    df = mask_columnwise(df)  # Only masks detected sensitive columns
    df.to_csv(anonymized_file, index=False)
    save_mapping()

    df = restore_data(pd.read_csv(anonymized_file))
    df.to_csv(restored_file, index=False)

    check_if_files_match(input_file, restored_file)


In [ ]:
# V2
import os
import re
import json
import time
import pickle
import numpy as np
import pandas as pd
from faker import Faker
from datetime import datetime
from google.colab import files
from presidio_analyzer import AnalyzerEngine
# Initialize Faker & Presidio Analyzer
fake = Faker()
Faker.seed(42)
analyzer = AnalyzerEngine()

# Mapping file for storing forward/reverse mappings
MAPPING_FILE = "mappings.pkl"
forward_mapping = {}
reverse_mapping = {}

NUMERIC_PATTERN = re.compile(r"^[<>]?[\d,.%]+$")
SENSITIVE_HEADERS = {"phone", "mobile", "credit", "card", "id", "name"}
EXCLUDED_ENTITIES = {"DATE_TIME"}

def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def load_merged_pickle():
    """Loads fake data from fake_dataset.pkl"""
    merged_file = "fake_dataset.pkl"

    if os.path.exists(merged_file):
        try:
            with open(merged_file, "rb") as f:
                return pickle.load(f)
        except (pickle.UnpicklingError, EOFError) as e:
            print(f"⚠️ Error loading fake_dataset.pkl: {e}")

    return {}

def load_mapping():
    """Loads existing mapping from Pickle file, ensuring proper data handling."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "rb") as f:
                data = pickle.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
            print("📂 Existing mapping loaded.")
        except (pickle.UnpicklingError, EOFError) as e:
            print(f"⚠️ Error loading Pickle file: {e}")

def save_mapping(filename):
    """Saves the mapping to a Pickle file with filename tracking."""
    mapping_data = {
        "file_name": filename,
        "forward_mapping": forward_mapping,
        "reverse_mapping": reverse_mapping,
        "metadata": {
            "updated_at": datetime.now().isoformat(),
            "record_count": len(forward_mapping),
        },
    }
    with open(MAPPING_FILE, "wb") as f:
        pickle.dump(mapping_data, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("💾 Mapping saved successfully.")

def load_and_analyze(file_path):
    """Loads the file (CSV or Excel) and returns a DataFrame."""
    if file_path.endswith(".csv"):
        return pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        return pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Provide a CSV or Excel file.")

@time_it
def check_existing_mappings(input_file):
    """Checks if existing mappings match the input file and regenerates anonymized data if needed."""
    if os.path.exists(MAPPING_FILE):
        with open(MAPPING_FILE, "r") as f:
            data = json.load(f)
            stored_filename = data.get("file_name")

        if stored_filename and stored_filename == input_file:
            if not os.path.exists("anonymized.csv"):
                print("📌 Regenerating anonymized.csv from existing mappings...")
                df = load_and_analyze(input_file)
                df = apply_existing_mappings(df)
                write_file(df, "anonymized.csv")
                return True
        else:
            print("⚠️ Input file does not match stored filename. Overwriting mapping...")
    return False

def write_file(df, file_path):
    """Writes DataFrame to CSV or Excel."""
    file_ext = os.path.splitext(file_path)[-1].lower()
    if file_ext == ".csv":
        df.to_csv(file_path, index=False)
    elif file_ext in [".xls", ".xlsx"]:
        df.to_excel(file_path, index=False)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Excel.")

@time_it
def analyze_column(df):
    """Uses Presidio to classify columns based on detected entity types."""
    entity_columns = {}
    for col in df.columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in SENSITIVE_HEADERS):
            entity_columns.setdefault("SENSITIVE", []).append(col)
            continue

        unique_values = df[col].dropna().astype(str).unique()[:10]
        entity_counts = {}

        for value in unique_values:
            if NUMERIC_PATTERN.match(value):
                continue  # Skip purely numeric values unless it's a phone/credit card

            results = analyzer.analyze(text=value, language="en")
            for result in results:
                if result.entity_type in EXCLUDED_ENTITIES:
                    continue  # Skip unwanted entities
                entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

        if entity_counts:
            predominant_entity = max(entity_counts, key=entity_counts.get)
            if predominant_entity not in entity_columns:
                entity_columns[predominant_entity] = []
            entity_columns[predominant_entity].append(col)

    return entity_columns

def generate_fake_values(column):
    """Generates fake values while preserving uniqueness, using bothify and merged.json data."""
    col_name = column.name.lower()
    original_values = column.dropna().unique()

    match_key = None
    for key in merged_data.keys():
        if re.search(fr"\b{key}\b", col_name, re.IGNORECASE):
            match_key = key
            break

    if column.dtype.kind in 'biufc':  # Skip numeric types except IDs
        if re.search(r'\bid\b', col_name, re.IGNORECASE):
            fake_values = [f"ID-{fake.bothify(text='???##########')}" for _ in original_values]
            mapping = dict(zip(original_values, fake_values))
            forward_mapping[col_name] = mapping
            reverse_mapping[col_name] = {v: k for k, v in mapping.items()}
            return column.map(mapping).fillna(column)

        return column

    fake_values = []

    # ✅ Handle different column types
    if re.search(r'\b(phone|mobile)\b', col_name, re.IGNORECASE):
        fake_values = [fake.bothify(text='+## ##########') for _ in original_values]
    elif re.search(r'\b(card|credit)\b', col_name, re.IGNORECASE):
        fake_values = [fake.bothify(text='####-####-####-####') for _ in original_values]
    elif re.search(r'\b(domain|url|link)\b', col_name, re.IGNORECASE):
        fake_values = [
            f"www.{fake.bothify(text='?????')}/{fake.bothify(text='##??#?#?#?#?###')}/{fake.bothify(text='####')}.com"
            for _ in original_values
        ]
    elif re.search(r'\bname\b', col_name, re.IGNORECASE) and match_key:
        fake_values_list = merged_data[match_key]
        fake_values = [
            f"{name.split()[0]} {name.split()[-1]} {fake.bothify(text='????')}" if ' ' in name else f"{name} {fake.bothify(text='????')}"
            for name in fake_values_list[:len(original_values)]
        ]
    elif match_key:
        fake_values_list = merged_data[match_key]
        fake_values = [
            fake_values_list[i] if i < len(fake_values_list) else fake.bothify(text='??????????')
            for i in range(len(original_values))
        ]
    else: fake_values = [fake.bothify(text="?????##########") for _ in original_values]

    mapping = dict(zip(original_values, fake_values))
    forward_mapping[col_name] = mapping
    reverse_mapping[col_name] = {v: k for k, v in mapping.items()}

    return column.map(mapping).fillna(column)

@time_it
def mask_columnwise(df):
    """Replaces sensitive data only in columns detected as sensitive by Presidio."""
    sensitive_columns = analyze_column(df)

    for entity, cols in sensitive_columns.items():
        for column in cols:
            print(f"🔒 Masking sensitive column: {column} (Detected as {entity})")
            df[column] = generate_fake_values(df[column])

    return df

def apply_existing_mappings(df):
    """Applies stored mappings to re-anonymize a DataFrame if mappings.json exists."""
    for column in df.columns:
        col_name = column.lower()
        if col_name in forward_mapping:
            df[column] = df[column].astype(str).map(forward_mapping[col_name]).fillna(df[column])
    return df


def restore_data(df):
    """Restores original values using stored mappings."""
    for column in df.columns:
        col_name = column.lower()
        if col_name in reverse_mapping:
            df[column] = df[column].astype(str).map(reverse_mapping[col_name]).fillna(df[column])
    return df


def check_if_files_match(original_file, restored_file):
    """Checks if the original and restored files match."""
    file_ext = os.path.splitext(original_file)[-1].lower()
    original_df = pd.read_csv(original_file) if file_ext == ".csv" else pd.read_excel(original_file)
    restored_df = pd.read_csv(restored_file) if file_ext == ".csv" else pd.read_excel(restored_file)

    test = original_df.equals(restored_df)
    print(f"📊 Are files identical? {test}")
    if not test:
        print("⚠️ Files are not identical!")
    else:
        print("✅ Files match perfectly!")

if __name__ == "__main__":
    merged_data = load_merged_pickle()
    load_mapping()
    # uploaded = files.upload()
    # input_file = list(uploaded.keys())[0].strip().replace(' ', '-')
    input_file = 'smaller_100k_companies.csv'

    if not check_existing_mappings(input_file):
        df = load_and_analyze(input_file)
        df = mask_columnwise(df)
        write_file(df, "anonymized.csv")
        save_mapping(input_file)

    restored_df = restore_data(load_and_analyze("anonymized.csv"))
    write_file(restored_df, "restored.csv")
    check_if_files_match(input_file, "restored.csv")

    print("✅ Processing complete!")


⏳ Execution time check_existing_mappings: 0.000007 seconds

⏳ Execution time analyze_column: 0.789008 seconds
🔒 Masking sensitive column: id (Detected as SENSITIVE)
🔒 Masking sensitive column: name (Detected as SENSITIVE)
🔒 Masking sensitive column: domain (Detected as URL)
🔒 Masking sensitive column: linkedin url (Detected as URL)
🔒 Masking sensitive column: locality (Detected as LOCATION)
🔒 Masking sensitive column: country (Detected as LOCATION)

⏳ Execution time mask_columnwise: 15.929399 seconds
💾 Mapping saved successfully.
📊 Are files identical? True
✅ Files match perfectly!
✅ Processing complete!


In [ ]:
# Converted list of dictionary of lists into dictionary
# Converting json to pickle
import os
import json
import pickle

def convert_merged_json(input_file="merged.json", output_file="converted_merged.json"):
    """Converts a list of dictionaries into a single dictionary and saves it."""
    if not os.path.exists(input_file):
        print(f"⚠️ File {input_file} not found.")
        return

    try:
        with open(input_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        if isinstance(data, list):
            merged_dict = {}
            for entry in data:
                if isinstance(entry, dict):
                    merged_dict.update(entry)

            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(merged_dict, f, indent=4)

            print(f"✅ Successfully converted and saved as {output_file}")
        else:
            print("⚠️ The input JSON is already in the correct format.")

    except json.JSONDecodeError as e:
        print(f"❌ Error parsing JSON: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

def converting_json_to_pickle(input_file="fake_dataset.json", output_file="fake_dataset.pkl"):
  with open(input_file, 'r') as fp:
    data = json.load(fp)

  with open(output_file, 'wb') as fp:
    pickle.dump(data, fp)

# converting_json_to_pickle()

In [ ]:
# Testing - classify orgainization
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import pandas as pd
from faker import Faker
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine

# Variable defined
ID = {}
fake_data={}
used_urls = set()

url_extensions = [
    ".com", ".net", ".org", ".edu", ".gov", ".co", ".us", ".uk", ".in", ".ru",
    ".jp", ".cn", ".de", ".fr", ".it", ".nl", ".es", ".br", ".au", ".ca",
    ".ch", ".se", ".no", ".za", ".mx", ".ar", ".be", ".kr", ".pl", ".tr",
    ".ua", ".ir", ".sa", ".ae", ".my", ".sg", ".hk", ".tw", ".nz", ".id",
    ".th", ".ph", ".vn", ".bd", ".lk", ".np", ".pk", ".cz", ".gr", ".hu",
    ".fi", ".dk", ".il", ".ie", ".pt", ".sk", ".si", ".ro", ".bg", ".rs",
    ".lt", ".lv", ".ee", ".hr", ".ba", ".md", ".ge", ".kz", ".by", ".tm",
    ".uz", ".af", ".qa", ".om", ".kw", ".bh", ".ye", ".jo", ".lb", ".sy",
    ".iq", ".ps", ".az", ".am", ".kg", ".mn", ".bt", ".mv", ".mm", ".kh",
    ".la", ".tl", ".sb", ".fj", ".pg", ".to", ".tv", ".ws", ".fm", ".ki"
]
with gzip.open("faker_data_v2.json.gz", "rt", encoding = 'utf-8') as f: fake_data_list = json.load(f)
# Objects
nlp = spacy.load("en_core_web_lg")
fake=Faker()
analyzer = AnalyzerEngine()

for data in fake_data_list:
    for key,value in data.items(): fake_data[key]=deque(value)
domain_pool = list(fake_data.get('url', deque()))

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
    entity_columns = {}

    # Step 1: Use Presidio to analyze all columns
    for col in df.columns:
        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns


def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value} {string.ascii_uppercase[counter % 26]}."
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category in {"location", "country"}: return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url":
        # Check if the URL already exists, if so, append a country extension
        fake_value = base_fake_value
        while fake_value in used_urls:
            ext = random.choice(url_extensions)
            if not fake_value.endswith(ext):
                fake_value += ext
        used_urls.add(fake_value)
        return fake_value
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 100_000_000 + 1}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value):
    global ID
    fake_value = None
    original_value = original_value.strip()
    if original_value in forward_mapping[category]: return forward_mapping[category][original_value]

    # Special case for ID
    if category == 'id':
        length = 6
        while True:
            fake_value = fake.bothify(text=f'ID-{"#"*length}')
            if fake_value not in ID:
                ID[fake_value] = True
                break
            if len(ID) >= 10 ** length: length += 1
    elif category == 'url':
        domain1, domain2 = random.sample(domain_pool, 2)
        base_fake_value = f"https://{domain1.lower()}/{domain2.lower()}.co"

        if base_fake_value not in reverse_mapping["url"]:
            fake_value = base_fake_value
        else:
            counter = len(reverse_mapping["url"])
            fake_value = modify_fake_value("url", base_fake_value, counter)

    elif fake_data.get(category):
      for _ in range(len(fake_data[category])):
          candidate = fake_data[category].popleft()
          fake_data[category].append(candidate)  # Reinsert at end (rotation)
          if candidate not in reverse_mapping[category]:
              fake_value = candidate
              break
      else:
          counter = len(reverse_mapping[category])
          base_fake_value = random.choice(list(reverse_mapping[category])) if reverse_mapping[category] else f"{category}_"
          fake_value = modify_fake_value(category, base_fake_value, counter)

    if not fake_value or fake_value in reverse_mapping[category]:
        counter = len(reverse_mapping[category])
        base = fake_value if fake_value else f"{category}_"
        fake_value = modify_fake_value(category, base, counter)

    # Store forward & reverse mappings
    forward_mapping[category][original_value] = fake_value
    reverse_mapping[category][fake_value] = original_value

    return fake_value


@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            df[col] = df[col].astype(str).apply(lambda x: get_fake_value(matching_keys[0], str(x)) if x else str(x))
    return df

def restore_original_value(category, fake_value):
    return reverse_mapping[category].get(fake_value, fake_value)

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(lambda x: restore_original_value(category, str(x)) if x else str(x))

    return df
def compare_files(original_file, restored_file):
    """Check if the original and restored files are identical."""
    file_ext = os.path.splitext(original_file)[-1].lower()
    original_df = pd.read_excel(original_file) if file_ext == ".xlsx" else pd.read_csv(original_file)
    restored_df = pd.read_excel(restored_file) if file_ext == ".xlsx" else pd.read_csv(restored_file)

    is_identical = original_df.equals(restored_df)
    print(f"📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")
    if not is_identical:
        print("⚠️ The restored file does not match the original. There may be an issue with the mapping.")

    return is_identical

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":forward_mapping,
        "reverse_mapping":reverse_mapping
    }
    with open(mapping_file, "w") as f:
        json.dump(mapping_data, f, indent=4)


if __name__=="__main__":
    input_file="smaller_100k_companies.csv"
    file_ext=os.path.splitext(input_file)[-1].lower()

    df = pd.read_excel(input_file, dtype=str) if file_ext == ".xlsx" else pd.read_csv(input_file, dtype=str, low_memory=False)
    entity_columns=analyze_column(df)
    print(entity_columns)

    anonymized_df=mask_dataframe(df)
    output_file="anonymized.xlsx" if file_ext==".xlsx" else "anonymized.csv"
    anonymized_df.to_excel(output_file,index=False) if file_ext==".xlsx" else anonymized_df.to_csv(output_file,index=False)
    print(f"✅ Anonymized data saved as {output_file}")

    save_mapping(input_file)
    restored_df=unmask_dataframe(pd.read_excel(output_file) if file_ext==".xlsx" else pd.read_csv(output_file))
    restored_file="restored.xlsx" if file_ext==".xlsx" else "restored.csv"
    restored_df.to_excel(restored_file,index=False) if file_ext==".xlsx" else restored_df.to_csv(restored_file,index=False)
    print(f"✅ Restored data saved as {restored_file}")
    compare_files(input_file, restored_file)


⏳ Execution time analyze_column: 3.669720 seconds
{'id': 'ID', 'name': 'ORG', 'domain': 'URL', 'size range': 'DATE_TIME', 'locality': 'LOCATION', 'country': 'COUNTRY', 'linkedin url': 'URL', 'current employee estimate': 'US_DRIVER_LICENSE', 'total employee estimate': 'US_DRIVER_LICENSE'}

⏳ Execution time mask_dataframe: 2.481203 seconds
✅ Anonymized data saved as anonymized.csv

⏳ Execution time unmask_dataframe: 0.367132 seconds
✅ Restored data saved as restored.csv
📊 Are files identical? ✅ Yes


In [ ]:
# Main Code - Optimizing for excel :- Main Code
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import tempfile
import pandas as pd
from faker import Faker
from pathlib import Path
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine

# Variable defined
ID, fake_data, used_urls, entity_columns  = {}, {}, set(), {}
COMMENT_KEYWORDS = ['comments', 'description', 'remarks', 'note', 'feedback', 'observation']

url_extensions = [
    ".com", ".net", ".org", ".edu", ".gov", ".co", ".us", ".uk", ".in", ".ru",
    ".jp", ".cn", ".de", ".fr", ".it", ".nl", ".es", ".br", ".au", ".ca",
    ".ch", ".se", ".no", ".za", ".mx", ".ar", ".be", ".kr", ".pl", ".tr",
    ".ua", ".ir", ".sa", ".ae", ".my", ".sg", ".hk", ".tw", ".nz", ".id",
    ".th", ".ph", ".vn", ".bd", ".lk", ".np", ".pk", ".cz", ".gr", ".hu",
    ".fi", ".dk", ".il", ".ie", ".pt", ".sk", ".si", ".ro", ".bg", ".rs",
    ".lt", ".lv", ".ee", ".hr", ".ba", ".md", ".ge", ".kz", ".by", ".tm",
    ".uz", ".af", ".qa", ".om", ".kw", ".bh", ".ye", ".jo", ".lb", ".sy",
    ".iq", ".ps", ".az", ".am", ".kg", ".mn", ".bt", ".mv", ".mm", ".kh",
    ".la", ".tl", ".sb", ".fj", ".pg", ".to", ".tv", ".ws", ".fm", ".ki"
]
with open("test.json", "r", ) as f: fake_data_list = json.load(f)
# Objects
nlp = spacy.load("en_core_web_sm")
fake=Faker()
analyzer = AnalyzerEngine()

for data in fake_data_list:
    for key,value in data.items(): fake_data[key]=deque(value)
domain_pool = list(fake_data.get('url', deque()))

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))
# For comments logic
def get_comment_columns(df):
    return [col for col in df.columns if any(kw in col.lower() for kw in COMMENT_KEYWORDS)]

def mask_comment_columns(df, mapping):
    comment_cols = get_comment_columns(df)
    print('Inside mask comment cols:', comment_cols)
    for col in comment_cols:
        df[col] = df[col].astype(str).apply(lambda x: mask_comment_text(x, mapping))
    return df

def mask_comment_text(text, mapping):
    if not text.strip():
        return text
    doc = nlp(text)
    modified_text = text
    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            mapped_value = mapping['names'].get(token.text, None)
            if mapped_value:
              modified_text = re.sub(
                rf"\b{re.escape(token.text)}(?=[\W_]|$)", mapped_value, modified_text
            )

    return modified_text

def unmask_comment_text(text, reverse_mapping):
    if not text.strip(): return text
    doc = nlp(text)
    modified_text = text
    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            original_value = reverse_mapping['names'].get(token.text, None)
            if original_value:
                modified_text = re.sub(
                    rf"\b{re.escape(token.text)}(?=[\W_]|$)", original_value, modified_text
                )
    return modified_text

# To track time
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
    # Step 1: Use Presidio to analyze all columns
    comment_cols = get_comment_columns(df)

    for col in df.columns:
        if col in comment_cols: continue

        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns

def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value}{string.ascii_lowercase[counter % 26]}"
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category in {"location", "country"}: return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url":
        # Check if the URL already exists, if so, append a country extension
        fake_value = base_fake_value
        while fake_value in used_urls:
            ext = random.choice(url_extensions)
            if not fake_value.endswith(ext):
                fake_value += ext
        used_urls.add(fake_value)
        return fake_value
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 100_000_000 + 1}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value):
    global ID
    fake_value = None
    if category == "names":
        original_tokens = str(original_value).split()
        fake_tokens = []
        used_fake_names = set(reverse_mapping[category])

        for token in original_tokens:
            if token in forward_mapping[category]:
                fake_token = forward_mapping[category][token]
            else:
                fake_token = None

                if fake_data.get(category):
                    for _ in range(len(fake_data[category])):
                        candidate = fake_data[category].popleft()
                        fake_data[category].append(candidate)

                        if candidate not in used_fake_names and candidate != token:
                            fake_token = candidate
                            break
                        else:
                            # Modify it if already used
                            counter = 0
                            base = candidate
                            while True:
                                modified = modify_fake_value(category, base, counter)
                                if modified not in used_fake_names and modified != token:
                                    fake_token = modified
                                    break
                                counter += 1
                            break  # once we found a modified one, skip the rest

                # Absolute fallback: no fake_data or nothing worked
                if not fake_token:
                    counter = 0
                    base_pool = list(used_fake_names)
                    random.shuffle(base_pool)

                    for base in base_pool[:10]:  # Try 10
                        fake_token = modify_fake_value(category, base, counter)
                        if fake_token not in used_fake_names and fake_token != token:
                            break
                        counter += 1
                    else:
                        # Final fallback: just keep modifying the original token
                        while True:
                            fake_token = modify_fake_value(category, token, counter)
                            if fake_token not in used_fake_names:
                                break
                            counter += 1

                # Save mappings
                forward_mapping[category][token] = fake_token
                reverse_mapping[category][fake_token] = token
                used_fake_names.add(fake_token)

            fake_tokens.append(fake_token)

        return " ".join(fake_tokens)



    if original_value in forward_mapping[category]: return forward_mapping[category][original_value]
    # Special case for ID
    if category == 'id':
        length = 6
        while True:
            fake_value = fake.bothify(text=f'ID-{"#"*length}')
            if fake_value not in ID:
                ID[fake_value] = True
                break
            if len(ID) >= 10 ** length: length += 1
    elif category == 'url':
        domain1, domain2 = random.sample(domain_pool, 2)
        base_fake_value = f"https://{domain1.lower()}/{domain2.lower()}.co"

        if base_fake_value not in reverse_mapping["url"]:
            fake_value = base_fake_value
        else:
            counter = len(reverse_mapping["url"])
            fake_value = modify_fake_value("url", base_fake_value, counter)

    elif fake_data.get(category):
      length = len(fake_data[category])
      for _ in range(length):
            candidate = fake_data[category].popleft()
            fake_data[category].append(candidate)  # Reinsert at end (rotation)
            if candidate not in reverse_mapping[category]:
                fake_value = candidate
                break
      else:
          counter = len(reverse_mapping[category])
          base_fake_value = (
              random.choice(list(reverse_mapping[category]))
              if reverse_mapping[category]
              else f"{category}_"
          )
          fake_value = modify_fake_value(category, base_fake_value, counter)

    if not fake_value or fake_value in reverse_mapping[category]:
        counter = len(reverse_mapping[category])
        base = fake_value if fake_value else f"{category}_"
        fake_value = modify_fake_value(category, base, counter)

    # Store forward & reverse mappings
    forward_mapping[category][original_value] = fake_value
    reverse_mapping[category][fake_value] = original_value

    return fake_value


@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [
            key for key, value in entity_mapping.items() if value == entity
        ]
        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(
                lambda x: get_fake_value(category, x) if x else x
            )
    return df


def restore_original_value(category, fake_value):
    return reverse_mapping[category].get(fake_value, fake_value)

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            category = matching_keys[0]

            if category == "names":
                def reverse_name(val):
                    tokens = str(val).split()
                    original_tokens = [reverse_mapping[category].get(tok, tok) for tok in tokens]
                    return " ".join(original_tokens)
                df[col] = df[col].astype(str).apply(lambda x: reverse_name(x) if pd.notna(x) else x)

            else:
                df[col] = df[col].astype(str).apply(
                    lambda x: restore_original_value(category, str(x)) if pd.notna(str(x)) else x
                )
    comment_cols = get_comment_columns(df)
    for col in comment_cols:
        df[col] = df[col].astype(str).apply(lambda x: unmask_comment_text(x, reverse_mapping))
    return df

@time_it
def compare_files(original_csv, restored_csv):
    """Compare two CSV files and return if they are identical."""
    original_df = pd.read_csv(original_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)
    restored_df = pd.read_csv(restored_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)

    is_identical = original_df.equals(restored_df)
    print(f"\n📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")

    if not is_identical:
        print("⚠️ The restored file does not match the original. Investigate the mapping or masking logic.")

    return is_identical

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":forward_mapping,
        "reverse_mapping":reverse_mapping
    }
    with open(mapping_file, "a") as f:
        json.dump(mapping_data, f, indent=4)

def process_file(input_file, file_ext):
    input_base = os.path.splitext(os.path.basename(input_file))[0]
    output_dir = os.path.join(".", input_base)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if file_ext == ".xlsx":
        xl = pd.read_excel(input_file, sheet_name=None, dtype=str, engine='openpyxl')

        for sheet_name, df in xl.items():
            print(f"\n🔍 Processing sheet: {sheet_name}")

            original_csv = os.path.join(output_dir, f"{sheet_name}_original.csv")
            masked_csv = os.path.join(output_dir, f"{sheet_name}_masked.csv")
            restored_csv = os.path.join(output_dir, f"{sheet_name}_restored.csv")

            df.to_csv(original_csv, index=False)

            entity_columns = analyze_column(df)
            print(f"Detected entities: {entity_columns}")

            masked_df = mask_dataframe(df.copy())
            # masked_df = mask_comment_columns(masked_df, forward_mapping)
            restored_df = unmask_dataframe(masked_df.copy())

            masked_df.to_csv(masked_csv, index=False)
            restored_df.to_csv(restored_csv, index=False)

            compare_files(original_csv, restored_csv)

            save_mapping(f'{input_file}-->{sheet_name}')
            entity_columns.clear()


    elif file_ext == ".csv":
        df = pd.read_csv(input_file, dtype=str, low_memory=False)
        print(f"\n🔍 Processing CSV: {input_file}")

        original_csv = os.path.join(output_dir, "original.csv")
        masked_csv = os.path.join(output_dir, "anonymized.csv")
        restored_csv = os.path.join(output_dir, "restored.csv")

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        df.to_csv(original_csv, index=False)

        entity_columns = analyze_column(df)
        print(f"Detected entities: {entity_columns}")

        masked_df = mask_dataframe(df.copy())
        # masked_df = mask_comment_columns(masked_df, forward_mapping)
        restored_df = unmask_dataframe(masked_df.copy())

        masked_df.to_csv(masked_csv, index=False)
        restored_df.to_csv(restored_csv, index=False)

        compare_files(original_csv, restored_csv)

        save_mapping(input_file)
    else:
        raise ValueError("Unsupported file type. Use .csv or .xlsx only.")


if __name__ == "__main__":
    input_file = "for_colab_test.csv"
    file_ext = os.path.splitext(input_file)[-1].lower()
    process_file(input_file, file_ext)


🔍 Processing CSV: for_colab_test.csv

⏳ Execution time analyze_column: 2.157893 seconds
Detected entities: {'names': 'PERSON', 'emails': 'URL', 'phone': 'PHONE_NUMBER', 'credit': 'US_BANK_NUMBER', 'url': 'URL', 'location': 'LOCATION', 'company': 'ORG'}

⏳ Execution time mask_dataframe: 0.005189 seconds

⏳ Execution time unmask_dataframe: 0.004125 seconds

📊 Are files identical? ❌ No
⚠️ The restored file does not match the original. Investigate the mapping or masking logic.

⏳ Execution time compare_files: 0.007497 seconds


In [ ]:
# Main Code - Testing comments :- Main Code
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import tempfile
import pandas as pd
from faker import Faker
from pathlib import Path
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine

# Variable defined
ID, fake_data, used_urls, entity_columns  = {}, {}, set(), {}
COMMENT_KEYWORDS = ['comments', 'description', 'remarks', 'note', 'feedback', 'observation']

url_extensions = [
    ".com", ".net", ".org", ".edu", ".gov", ".co", ".us", ".uk", ".in", ".ru",
    ".jp", ".cn", ".de", ".fr", ".it", ".nl", ".es", ".br", ".au", ".ca",
    ".ch", ".se", ".no", ".za", ".mx", ".ar", ".be", ".kr", ".pl", ".tr",
    ".ua", ".ir", ".sa", ".ae", ".my", ".sg", ".hk", ".tw", ".nz", ".id",
    ".th", ".ph", ".vn", ".bd", ".lk", ".np", ".pk", ".cz", ".gr", ".hu",
    ".fi", ".dk", ".il", ".ie", ".pt", ".sk", ".si", ".ro", ".bg", ".rs",
    ".lt", ".lv", ".ee", ".hr", ".ba", ".md", ".ge", ".kz", ".by", ".tm",
    ".uz", ".af", ".qa", ".om", ".kw", ".bh", ".ye", ".jo", ".lb", ".sy",
    ".iq", ".ps", ".az", ".am", ".kg", ".mn", ".bt", ".mv", ".mm", ".kh",
    ".la", ".tl", ".sb", ".fj", ".pg", ".to", ".tv", ".ws", ".fm", ".ki"
]
with open("test.json", "r", ) as f: fake_data_list = json.load(f)
# Objects
nlp = spacy.load("en_core_web_sm")
fake=Faker()
analyzer = AnalyzerEngine()

for data in fake_data_list:
    for key,value in data.items(): fake_data[key]=deque(value)
domain_pool = list(fake_data.get('url', deque()))

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))

# To track time
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

# For comments logic
def get_comment_columns(df):
    return [col for col in df.columns if any(kw in col.lower() for kw in COMMENT_KEYWORDS)]

@time_it
def mask_comment_columns(df, mapping):
    comment_cols = get_comment_columns(df)
    print('Inside mask comment cols:', comment_cols)
    for col in comment_cols:
        df[col] = df[col].astype(str).apply(lambda x: mask_comment_text(x, mapping))
    return df

def mask_comment_text(text, mapping):
    if not text.strip():
        return text
    doc = nlp(text)
    modified_text = text
    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            mapped_value = mapping['names'].get(token.text, token.text)
            if mapped_value:
              modified_text = re.sub(
                rf"\b{re.escape(token.text)}(?=[\W_]|$)", mapped_value, modified_text
            )

    return modified_text

def unmask_comment_text(text, reverse_mapping):
    if not text.strip():
        return text

    doc = nlp(text)
    replacements = {}  # Collect replacements first

    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            original_value = reverse_mapping['names'].get(token.text, token.text)
            if original_value:
                replacements[token.text] = original_value

    for fake_token, real_token in replacements.items():
        text = re.sub(
            rf"\b{re.escape(fake_token)}(?=[\W_]|$)", real_token, text
        )

    return text

@time_it
def analyze_column(df):
    # Step 1: Use Presidio to analyze all columns
    comment_cols = get_comment_columns(df)

    for col in df.columns:
        if col in comment_cols: continue

        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns

def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value}{string.ascii_lowercase[counter % 26]}"
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category in {"location", "country"}: return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url":
        # Check if the URL already exists, if so, append a country extension
        fake_value = base_fake_value
        while fake_value in used_urls:
            ext = random.choice(url_extensions)
            if not fake_value.endswith(ext):
                fake_value += ext
        used_urls.add(fake_value)
        return fake_value
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 100_000_000 + 1}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value):
    global ID
    fake_value = None
    if category == "names":
        original_tokens = str(original_value).split()
        fake_tokens = []
        used_fake_names = set(reverse_mapping[category])

        for token in original_tokens:
            if token in forward_mapping[category]:
                fake_token = forward_mapping[category][token]
            else:
                fake_token = None

                if fake_data.get(category):
                    for _ in range(len(fake_data[category])):
                        candidate = fake_data[category].popleft()
                        fake_data[category].append(candidate)

                        if candidate not in used_fake_names and candidate != token:
                            fake_token = candidate
                            break
                        else:
                            # Modify it if already used
                            counter = 0
                            base = candidate
                            while True:
                                modified = modify_fake_value(category, base, counter)
                                if modified not in used_fake_names and modified != token:
                                    fake_token = modified
                                    break
                                counter += 1
                            break  # once we found a modified one, skip the rest

                # Absolute fallback: no fake_data or nothing worked
                if not fake_token:
                    counter = 0
                    base_pool = list(used_fake_names)
                    random.shuffle(base_pool)

                    for base in base_pool[:10]:  # Try 10
                        fake_token = modify_fake_value(category, base, counter)
                        if fake_token not in used_fake_names and fake_token != token:
                            break
                        counter += 1
                    else:
                        # Final fallback: just keep modifying the original token
                        while True:
                            fake_token = modify_fake_value(category, token, counter)
                            if fake_token not in used_fake_names:
                                break
                            counter += 1

                # Save mappings
                forward_mapping[category][token] = fake_token
                reverse_mapping[category][fake_token] = token
                used_fake_names.add(fake_token)

            fake_tokens.append(fake_token)

        return " ".join(fake_tokens)



    if original_value in forward_mapping[category]: return forward_mapping[category][original_value]
    # Special case for ID
    if category == 'id':
        length = 6
        while True:
            fake_value = fake.bothify(text=f'ID-{"#"*length}')
            if fake_value not in ID:
                ID[fake_value] = True
                break
            if len(ID) >= 10 ** length: length += 1
    elif category == 'url':
        domain1, domain2 = random.sample(domain_pool, 2)
        base_fake_value = f"https://{domain1.lower()}/{domain2.lower()}.co"

        if base_fake_value not in reverse_mapping["url"]:
            fake_value = base_fake_value
        else:
            counter = len(reverse_mapping["url"])
            fake_value = modify_fake_value("url", base_fake_value, counter)

    elif fake_data.get(category):
      length = len(fake_data[category])
      for _ in range(length):
            candidate = fake_data[category].popleft()
            fake_data[category].append(candidate)  # Reinsert at end (rotation)
            if candidate not in reverse_mapping[category]:
                fake_value = candidate
                break
      else:
          counter = len(reverse_mapping[category])
          base_fake_value = (
              random.choice(list(reverse_mapping[category]))
              if reverse_mapping[category]
              else f"{category}_"
          )
          fake_value = modify_fake_value(category, base_fake_value, counter)

    if not fake_value or fake_value in reverse_mapping[category]:
        counter = len(reverse_mapping[category])
        base = fake_value if fake_value else f"{category}_"
        fake_value = modify_fake_value(category, base, counter)

    # Store forward & reverse mappings
    forward_mapping[category][original_value] = fake_value
    reverse_mapping[category][fake_value] = original_value

    return fake_value


@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [
            key for key, value in entity_mapping.items() if value == entity
        ]
        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(
                lambda x: get_fake_value(category, x) if x else x
            )
    return df


def restore_original_value(category, fake_value):
    return reverse_mapping[category].get(fake_value, fake_value)

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            category = matching_keys[0]

            if category == "names":
                def reverse_name(val):
                    tokens = str(val).split()
                    original_tokens = [reverse_mapping[category].get(tok, tok) for tok in tokens]
                    return " ".join(original_tokens)
                df[col] = df[col].astype(str).apply(lambda x: reverse_name(x) if pd.notna(x) else x)

            else:
                df[col] = df[col].astype(str).apply(
                    lambda x: restore_original_value(category, str(x)) if pd.notna(str(x)) else x
                )
    comment_cols = get_comment_columns(df)
    for col in comment_cols:
        df[col] = df[col].astype(str).apply(lambda x: unmask_comment_text(x, reverse_mapping))
    return df

@time_it
def compare_files(original_csv, restored_csv):
    """Compare two CSV files and return if they are identical."""
    original_df = pd.read_csv(original_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)
    restored_df = pd.read_csv(restored_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)

    is_identical = original_df.equals(restored_df)
    print(f"\n📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")

    if not is_identical:
        print("⚠️ The restored file does not match the original. Investigate the mapping or masking logic.")

    return is_identical

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":forward_mapping,
        "reverse_mapping":reverse_mapping
    }
    with open(mapping_file, "a") as f:
        json.dump(mapping_data, f, indent=4)

def process_file(input_file, file_ext):
    input_base = os.path.splitext(os.path.basename(input_file))[0]
    output_dir = os.path.join(".", input_base)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if file_ext == ".xlsx":
        xl = pd.read_excel(input_file, sheet_name=None, dtype=str, engine='openpyxl')

        for sheet_name, df in xl.items():
            print(f"\n🔍 Processing sheet: {sheet_name}")

            original_csv = os.path.join(output_dir, f"{sheet_name}_original.csv")
            masked_csv = os.path.join(output_dir, f"{sheet_name}_masked.csv")
            restored_csv = os.path.join(output_dir, f"{sheet_name}_restored.csv")

            df.to_csv(original_csv, index=False)

            entity_columns = analyze_column(df)
            print(f"Detected entities: {entity_columns}")

            masked_df = mask_dataframe(df.copy())
            masked_df = mask_comment_columns(masked_df, forward_mapping)
            restored_df = unmask_dataframe(masked_df.copy())

            masked_df.to_csv(masked_csv, index=False)
            restored_df.to_csv(restored_csv, index=False)

            compare_files(original_csv, restored_csv)

            save_mapping(f'{input_file}-->{sheet_name}')
            entity_columns.clear()


    elif file_ext == ".csv":
        df = pd.read_csv(input_file, dtype=str, low_memory=False)
        print(f"\n🔍 Processing CSV: {input_file}")

        original_csv = os.path.join(output_dir, "original.csv")
        masked_csv = os.path.join(output_dir, "anonymized.csv")
        restored_csv = os.path.join(output_dir, "restored.csv")

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        df.to_csv(original_csv, index=False)

        entity_columns = analyze_column(df)
        print(f"Detected entities: {entity_columns}")

        masked_df = mask_dataframe(df.copy())
        masked_df = mask_comment_columns(masked_df, forward_mapping)
        restored_df = unmask_dataframe(masked_df.copy())

        masked_df.to_csv(masked_csv, index=False)
        restored_df.to_csv(restored_csv, index=False)

        compare_files(original_csv, restored_csv)

        save_mapping(input_file)
    else:
        raise ValueError("Unsupported file type. Use .csv or .xlsx only.")


if __name__ == "__main__":
    input_file = "for_colab_test.csv"
    file_ext = os.path.splitext(input_file)[-1].lower()
    process_file(input_file, file_ext)


🔍 Processing CSV: for_colab_test.csv

⏳ Execution time analyze_column: 1.643175 seconds
Detected entities: {'names': 'PERSON', 'emails': 'URL', 'phone': 'PHONE_NUMBER', 'credit': 'US_BANK_NUMBER', 'url': 'URL', 'location': 'LOCATION', 'company': 'ORG'}

⏳ Execution time mask_dataframe: 0.004188 seconds
Inside mask comment cols: ['comments']

⏳ Execution time mask_comment_columns: 0.480729 seconds

⏳ Execution time unmask_dataframe: 0.442396 seconds

📊 Are files identical? ❌ No
⚠️ The restored file does not match the original. Investigate the mapping or masking logic.

⏳ Execution time compare_files: 0.005113 seconds


In [ ]:
# Testing different mapping technique
# Testing - classify orgainization
# Problematic -> can't use to demask paragraph
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import pandas as pd
from faker import Faker
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine


ID = {}
fake_data={}
nlp = spacy.load("en_core_web_lg")
fake=Faker()
analyzer = AnalyzerEngine()

with gzip.open("faker_data.json.gz", "rt", encoding='utf-8') as f:
    fake_data_list = json.load(f)
for data in fake_data_list:
    for key,value in data.items():
        fake_data[key]=deque(value)

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

@time_it
def analyze_column(df):
    entity_columns = {}

    # Step 1: Use Presidio to analyze all columns
    for col in df.columns:
        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns


def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value} {string.ascii_uppercase[counter % 26]}."
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category == "location": return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url": return base_fake_value.replace("://", f"://sub{counter}.", 1)
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 50}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value, colname):
    global ID
    # Added
    if colname not in forward_mapping[category]: forward_mapping[category][colname] = {}
    if colname not in reverse_mapping[category]: reverse_mapping[category][colname] = {}
    if colname in forward_mapping[category] and original_value in forward_mapping[category][colname]:
      return forward_mapping[category][colname][original_value]

    if category == 'id':
      length = 6
      while True:
          fake_value = fake.bothify(text=f'ID-{"#"*length}')
          if fake_value not in ID:
              ID[fake_value] = True
              break
          # If exhausted all possibilities for the current length, increase the length
          if len(ID) >= 10 ** length:
              length += 1
    elif fake_data.get(category):
        fake_value = fake_data[category].pop() if fake_data[category] else None
    else:
        base_fake=random.choice(list(forward_mapping[category][colname].values()))
        if base_fake:
          counter=len(forward_mapping[category][colname])
          fake_value=modify_fake_value(category,base_fake,counter)

    forward_mapping[category][colname][original_value] = fake_value
    reverse_mapping[category][colname][fake_value] = original_value


    return fake_value

@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            df[col] = df[col].astype(str).apply(lambda x: get_fake_value(matching_keys[0], str(x), col) if x else str(x))
    return df

def restore_original_value(category, fake_value, colname):
    if colname in reverse_mapping[category]:
      return reverse_mapping[category][colname].get(fake_value, fake_value)
    return fake_value

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]

        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(lambda x: restore_original_value(category, str(x), col) if x else str(x))

    return df
def compare_files(original_file, restored_file):
    """Check if the original and restored files are identical."""
    file_ext = os.path.splitext(original_file)[-1].lower()
    original_df = pd.read_excel(original_file) if file_ext == ".xlsx" else pd.read_csv(original_file)
    restored_df = pd.read_excel(restored_file) if file_ext == ".xlsx" else pd.read_csv(restored_file)

    is_identical = original_df.equals(restored_df)
    print(f"📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")
    if not is_identical:
        print("⚠️ The restored file does not match the original. There may be an issue with the mapping.")

    return is_identical

def de_anonymize_paragraph(text):
  for category, col_map in reverse_mapping.items():
    for colname, mapping in col_map.items():
        for fake_value, original_value in mapping.items():
            if fake_value in text:
                text = text.replace(fake_value, original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":dict(forward_mapping),
        "reverse_mapping":dict(reverse_mapping)
    }
    with open(mapping_file, "w") as f:
        json.dump(mapping_data, f, indent=4)

if __name__=="__main__":
    input_file="smaller_100k_companies.csv"
    file_ext=os.path.splitext(input_file)[-1].lower()

    df = pd.read_excel(input_file, dtype=str) if file_ext == ".xlsx" else pd.read_csv(input_file, dtype=str, low_memory=False)
    entity_columns=analyze_column(df)
    print(entity_columns)

    anonymized_df=mask_dataframe(df)
    output_file="anonymized.xlsx" if file_ext==".xlsx" else "anonymized.csv"
    anonymized_df.to_excel(output_file,index=False) if file_ext==".xlsx" else anonymized_df.to_csv(output_file,index=False)
    print(f"✅ Anonymized data saved as {output_file}")

    save_mapping(input_file)
    restored_df=unmask_dataframe(pd.read_excel(output_file) if file_ext==".xlsx" else pd.read_csv(output_file))
    restored_file="restored.xlsx" if file_ext==".xlsx" else "restored.csv"
    restored_df.to_excel(restored_file,index=False) if file_ext==".xlsx" else restored_df.to_csv(restored_file,index=False)
    print(f"✅ Restored data saved as {restored_file}")
    compare_files(input_file, restored_file)


⏳ Execution time analyze_column: 3.097933 seconds
{'id': 'ID', 'name': 'ORG', 'domain': 'URL', 'size range': 'DATE_TIME', 'locality': 'LOCATION', 'country': 'COUNTRY', 'linkedin url': 'URL', 'current employee estimate': 'US_DRIVER_LICENSE', 'total employee estimate': 'US_DRIVER_LICENSE'}

⏳ Execution time mask_dataframe: 2.705842 seconds
✅ Anonymized data saved as anonymized.csv

⏳ Execution time unmask_dataframe: 0.398932 seconds
✅ Restored data saved as restored.csv
📊 Are files identical? ✅ Yes


In [ ]:
"""Generating fake names and emails and comparing them"""
from itertools import count
import json
import faker
import random
from collections import Counter

fake = faker.Faker()

def generate_fake_names():
    unique_names = set()

    for _ in range(500_000):
        fake_name = fake.name()
        if fake_name in unique_names: fake_name += f' {fake.name()}'
        unique_names.add(fake_name)

    with open('fake_names.json', 'w') as fp:
        json.dump(
            {
                'names': list(unique_names)
            },
            fp,
            indent=4
        )
    print('📈Done generating fake names!')

def generate_fake_email():
    unique_emails = set()
    mail_endings = [
                    'gmail.com', 'hotmail.com',
                    'yahoo.com', 'outlook.com',
                    'live.com', 'yandex.com',
                    'mail.com', 'aol.com', 'edu.com'
                ]
    for _ in range(500_000):
        fake_email = fake.email()
        while fake_email in unique_emails:
            body, domain = fake_email.split('@')
            fake_email = f'{body}.{fake.name().replace(" ", ".")}.{fake.bothify(text="#####")}@{random.choice(mail_endings)}'
        unique_emails.add(fake_email)

    with open('fake_emails.json', 'w') as fp:
        json.dump(
            {
                'emails': list(unique_emails),
            },
            fp,
            indent=4
        )

def find_repeating_values():
    file_name = 'fake_emails.json'
    with open(file_name, 'r') as fp:
        data = json.load(fp)

    values = data.get('emails', []) or data.get('names', [])

    counts = Counter(values)
    repeating_values = {item:count for item, count in counts.items() if count > 1}

    print(f'Repeating values:{len(repeating_values)}')

# find_repeating_values()
# generate_fake_email()
# generate_fake_names()

In [ ]:
# Dummy
json_data = {
    "name": "James Bat Bond",
    "email":[ "jamesbatbond@ust.com", 'kirant700@gmail.com', '12312@ust.com'],
    "backup_email": "b20cs110@mace.ac.in",
    "phone": "+973 30982167",
    "location": "Wakanda, Africa",
    "description": "Abram is a software engineer from Wakanda.",
    "metadata": {
        "emergency_contact": "+91 9678785654",
        "company": "UST Global",
        "address": {
            "city": "New York",
            "country": "USA"
        }
    }
}

In [ ]:
# Uploading json file and doing NER on attributes
import json
import time
from google.colab import files
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

def time_it(func):
  def wrapper(*args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    end = time.time()
    print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
    return result
  return wrapper

# Processing JSON Spacey
def processing_json(data):
  if isinstance(data, dict):
    return {key: processing_json(value) for key, value in data.items()}
  elif isinstance(data, list):
    return [processing_json(item) for item in data]
  elif isinstance(data, str):
    result = analyzer.analyze(
        text = data,
        entities = ['PHONE_NUMBER', 'ADDRESS', 'LOCATION', 'PERSON', 'EMAIL_ADDRESS', 'STREET_ADDRESS', 'CREDIT_CARD'],
        language = 'en'
    )
    return anonymizer.anonymize(text=data, analyzer_results=result).text if result else data
  else: return data

@time_it
def json_format_anonymized(json_data):
  processed_data = processing_json(json_data)
  print(json.dumps(processed_data, indent = 4))

def json_file_anonymized():
  file_upload = files.upload()
  for filename in file_upload.keys():
    if filename.endswith('.json'):
      with open(filename, 'r') as fp:
        json_data = json.load(fp)
        processing_json(json_data)

json_format_anonymized(json_data)

{
    "name": "<PERSON>",
    "email": [
        "<EMAIL_ADDRESS>",
        "<EMAIL_ADDRESS>",
        "<EMAIL_ADDRESS>"
    ],
    "backup_email": "<EMAIL_ADDRESS>",
    "phone": "<PHONE_NUMBER>",
    "location": "<LOCATION>, <LOCATION>",
    "description": "<PERSON> is a software engineer from <LOCATION>.",
    "metadata": {
        "emergency_contact": "<PHONE_NUMBER>",
        "company": "UST Global",
        "address": {
            "city": "<LOCATION>",
            "country": "<LOCATION>"
        }
    }
}

⏳ Execution time json_format_anonymized: 0.152774 seconds


In [ ]:
!pip install spacy
!pip install presidio_analyzer
!python -m spacy download en_core_web_lg
!pip install faker pyxlsb fastexcel faker python-calamine
!pip install --upgrade --force-reinstall "Cython" "spacy"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.0/886.0 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Main Code - Fixing demasking of comments :- Main Code
import re
import os
import gzip
import time
import json
import spacy
import string
import random
import tempfile
import pandas as pd
from faker import Faker
from pathlib import Path
from collections import defaultdict, deque
from presidio_analyzer import AnalyzerEngine

# Variable defined
ID, fake_data, used_urls, entity_columns  = {}, {}, set(), {}
COMMENT_KEYWORDS = ['comments', 'description', 'remarks', 'note', 'feedback', 'observation']

url_extensions = [
    ".com", ".net", ".org", ".edu", ".gov", ".co", ".us", ".uk", ".in", ".ru",
    ".jp", ".cn", ".de", ".fr", ".it", ".nl", ".es", ".br", ".au", ".ca",
    ".ch", ".se", ".no", ".za", ".mx", ".ar", ".be", ".kr", ".pl", ".tr",
    ".ua", ".ir", ".sa", ".ae", ".my", ".sg", ".hk", ".tw", ".nz", ".id",
    ".th", ".ph", ".vn", ".bd", ".lk", ".np", ".pk", ".cz", ".gr", ".hu",
    ".fi", ".dk", ".il", ".ie", ".pt", ".sk", ".si", ".ro", ".bg", ".rs",
    ".lt", ".lv", ".ee", ".hr", ".ba", ".md", ".ge", ".kz", ".by", ".tm",
    ".uz", ".af", ".qa", ".om", ".kw", ".bh", ".ye", ".jo", ".lb", ".sy",
    ".iq", ".ps", ".az", ".am", ".kg", ".mn", ".bt", ".mv", ".mm", ".kh",
    ".la", ".tl", ".sb", ".fj", ".pg", ".to", ".tv", ".ws", ".fm", ".ki"
]
# with gzip.open("faker_data_v3.json.gz", "rt", encoding='utf-8') as f: fake_data_list = json.load(f)
# Objects
nlp = spacy.load("en_core_web_sm")
fake=Faker()
analyzer = AnalyzerEngine()


fake_data_list = {} # Remove this
for data in fake_data_list:
    for key,value in data.items(): fake_data[key]=deque(value)
domain_pool = list(fake_data.get('url', deque()))

entity_mapping={
    'names':'PERSON',
    'emails':'EMAIL_ADDRESS',
    'phone':'PHONE_NUMBER',
    'location':'LOCATION',
    'credit':'CREDIT_CARD',
    'url':'URL',
    'country':'COUNTRY',
    'company':"ORG",
    'id':'ID',
}

mapping_file="mapping.json"
forward_mapping=defaultdict(dict)
reverse_mapping=defaultdict(dict)
comment_entity_positions = defaultdict(dict)

if os.path.exists(mapping_file):
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
        forward_mapping.update(mapping_data.get("forward_mapping", {}))
        reverse_mapping.update(mapping_data.get("reverse_mapping", {}))

# To track time
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

# For comments logic
def get_comment_columns(df):
    return [col for col in df.columns if any(kw in col.lower() for kw in COMMENT_KEYWORDS)]

def mask_comment_columns(df, mapping):
    comment_cols = get_comment_columns(df)
    print('Inside mask comment cols:', comment_cols)

    for col in comment_cols:
        for i, text in df[col].astype(str).items():
            masked_text, entities = mask_comment_text(text, mapping)
            df.at[i, col] = masked_text
            if entities: comment_entity_positions[col][i] = entities
    return df

def mask_comment_text(text, mapping):
    if not text.strip(): return text, []

    doc = nlp(text)
    masked_text = text
    entities = []
    offset = 0

    for token in doc:
        if token.pos_ == "PROPN" or token.ent_type_:
            original = token.text
            fake = mapping['names'].get(original, None)

            if fake and fake != original:
                pattern = re.compile(rf"\b{re.escape(original)}(?=[\W_]|$)")
                match = pattern.search(masked_text, token.idx + offset)
                if match:
                    start, end = match.span()
                    masked_text = masked_text[:start] + fake + masked_text[end:]
                    entities.append({
                        "start": start,
                        "end": start + len(fake),
                        "original": original,
                        "fake": fake
                    })
                    offset += len(fake) - len(original)

    return masked_text, entities

def unmask_comment_columns(df, reverse_mapping, entity_metadata):
    comment_cols = get_comment_columns(df)
    print('Inside unmask comment cols:', comment_cols)

    for col in comment_cols:
        if col not in entity_metadata:
            continue

        for i, text in df[col].astype(str).items():
            if i in entity_metadata[col]:
                df.at[i, col] = unmask_comment_text(text, entity_metadata[col][i], reverse_mapping)

    return df

def unmask_comment_text(text, entities, reverse_mapping):
    for ent in sorted(entities, key=lambda x: -x['start']):
        fake = ent['fake']
        original = reverse_mapping['names'].get(fake, ent['original'])
        text = text[:ent['start']] + original + text[ent['end']:]
    return text

@time_it
def analyze_column(df):
    # Step 1: Use Presidio to analyze all columns
    comment_cols = get_comment_columns(df)

    for col in df.columns:
        if col in comment_cols: continue

        if 'id' in col.lower():
            entity_columns[col] = 'ID'
        elif 'country' in col.lower():
            entity_columns[col] = 'COUNTRY'
        else:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            entity_counts = {}

            for value in unique_values:
                results = analyzer.analyze(text=value, language='en')
                for result in results:
                    entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

            if entity_counts:
                predominant_entity = max(entity_counts, key=entity_counts.get)
                entity_columns[col] = predominant_entity
                if predominant_entity=="LOCATION":
                  org_count=0
                  for value in unique_values:
                    doc=nlp(value)
                    for ent in doc.ents:
                      if ent.label_=="ORG":
                        org_count+=1
                  if org_count>5:
                    predominant_entity="ORG"
                entity_columns[col]=predominant_entity

    # Step 2: Use SpaCy to analyze non-numeric and unclassified columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if col not in entity_columns:
            unique_values = df[col].dropna().astype(str).unique()[:25]
            org_count = 0

            for value in unique_values:
                doc = nlp(value)
                for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        org_count += 1

            # If more than half of the sample values are ORG, classify as ORG
            if org_count > 12:
                entity_columns[col] = 'ORG'

    return entity_columns

def modify_fake_value(category, base_fake_value, counter):
    if category == "names": return f"{base_fake_value}{string.ascii_lowercase[counter % 26]}"
    elif category == "emails":
        name, domain = base_fake_value.split("@")
        return f"{name}{counter}@{domain}"
    elif category in {"location", "country"}: return f"{base_fake_value}, District {counter % 100_000_000 + 1}"
    elif category == "url":
        # Check if the URL already exists, if so, append a country extension
        fake_value = base_fake_value
        while fake_value in used_urls:
            ext = random.choice(url_extensions)
            if not fake_value.endswith(ext):
                fake_value += ext
        used_urls.add(fake_value)
        return fake_value
    elif category == "phone": return f"{base_fake_value[:-2]}{counter % 100:02d}"
    elif category == "company": return f"{base_fake_value} Group {counter % 100_000_000 + 1}"
    elif category == "credit": return f"{base_fake_value[:-4]}{counter % 10000:04d}"
    else: return f"{base_fake_value}-{counter}"


def get_fake_value(category, original_value):
    global ID
    if original_value in forward_mapping[category]: return forward_mapping[category][original_value]
    fake_value = None
    if category == "names":
        original_tokens = str(original_value).split()
        fake_tokens = []
        used_fake_names = set(reverse_mapping[category])

        for token in original_tokens:
            if token in forward_mapping[category]:
                fake_token = forward_mapping[category][token]
            else:
                fake_token = None

                if fake_data.get(category):
                    for _ in range(len(fake_data[category])):
                        candidate = fake_data[category].popleft()
                        fake_data[category].append(candidate)

                        if candidate not in used_fake_names and candidate != token:
                            fake_token = candidate
                            break
                        else:
                            # Modify it if already used
                            counter = 0
                            base = candidate
                            while True:
                                modified = modify_fake_value(category, base, counter)
                                if modified not in used_fake_names and modified != token:
                                    fake_token = modified
                                    break
                                counter += 1
                            break

                # Save mappings
                forward_mapping[category][token] = fake_token
                reverse_mapping[category][fake_token] = token
                used_fake_names.add(fake_token)

            fake_tokens.append(fake_token)

        return " ".join(fake_tokens)


    # Special case for ID
    if category == 'id':
        length = 6
        while True:
            fake_value = fake.bothify(text=f'ID-{"#"*length}')
            if fake_value not in ID:
                ID[fake_value] = True
                break
            if len(ID) >= 10 ** length: length += 1
    elif category == 'url':
        domain1, domain2 = random.sample(domain_pool, 2)
        base_fake_value = f"https://{domain1.lower()}/{domain2.lower()}.co"

        if base_fake_value not in reverse_mapping["url"]:
            fake_value = base_fake_value
        else:
            counter = len(reverse_mapping["url"])
            fake_value = modify_fake_value("url", base_fake_value, counter)

    elif fake_data.get(category):
      length = len(fake_data[category])
      for _ in range(length):
            candidate = fake_data[category].popleft()
            fake_data[category].append(candidate)  # Reinsert at end (rotation)
            if candidate not in reverse_mapping[category]:
                fake_value = candidate
                break
      else:
          counter = len(reverse_mapping[category])
          base_fake_value = (
              random.choice(list(reverse_mapping[category]))
              if reverse_mapping[category]
              else f"{category}_"
          )
          fake_value = modify_fake_value(category, base_fake_value, counter)

    if not fake_value or fake_value in reverse_mapping[category]:
        counter = len(reverse_mapping[category])
        base = fake_value if fake_value else f"{category}_"
        fake_value = modify_fake_value(category, base, counter)

    forward_mapping[category][original_value] = fake_value
    reverse_mapping[category][fake_value] = original_value

    return fake_value

@time_it
def mask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [
            key for key, value in entity_mapping.items() if value == entity
        ]
        if matching_keys:
            category = matching_keys[0]
            df[col] = df[col].astype(str).apply(
                lambda x: get_fake_value(category, x) if x else x
            )
    return df

def restore_original_value(category, fake_value):
    return reverse_mapping[category].get(fake_value, fake_value)

@time_it
def unmask_dataframe(df):
    for col, entity in entity_columns.items():
        matching_keys = [key for key, value in entity_mapping.items() if value == entity]
        if matching_keys:
            category = matching_keys[0]

            if category == "names":
                def reverse_name(val):
                    tokens = str(val).split()
                    original_tokens = [reverse_mapping[category].get(tok, tok) for tok in tokens]
                    return " ".join(original_tokens)
                df[col] = df[col].astype(str).apply(lambda x: reverse_name(x) if pd.notna(x) else x)

            else:
                df[col] = df[col].astype(str).apply(
                    lambda x: restore_original_value(category, str(x)) if pd.notna(str(x)) else x
                )
    df = unmask_comment_columns(df, reverse_mapping, comment_entity_positions)
    return df

@time_it
def compare_files(original_csv, restored_csv):
    """Compare two CSV files and return if they are identical."""
    original_df = pd.read_csv(original_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)
    restored_df = pd.read_csv(restored_csv, dtype=str).sort_index(axis=1).reset_index(drop=True)

    is_identical = original_df.equals(restored_df)
    print(f"\n📊 Are files identical? {'✅ Yes' if is_identical else '❌ No'}")

    if not is_identical:
        print("⚠️ The restored file does not match the original. Investigate the mapping or masking logic.")

    return is_identical

def de_anonymize_paragraph(text):
  for category,mapping in reverse_mapping.items():
    for fake_value,original_value in mapping.items():
      if fake_value in text:
        text=text.replace(fake_value,original_value)
  return text

def save_mapping(filename):
    mapping_data={
        "filename":filename,
        "updated_at": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "forward_mapping":forward_mapping,
        "reverse_mapping":reverse_mapping
    }
    with open(mapping_file, "a") as f:
        json.dump(mapping_data, f, indent=4)

def process_file(input_file, file_ext):
    input_base = os.path.splitext(os.path.basename(input_file))[0]
    output_dir = os.path.join(".", input_base)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if file_ext == ".xlsx":
        xl = pd.read_excel(input_file, sheet_name=None, dtype=str, engine='calamine')

        for sheet_name, df in xl.items():
            print(f"\n🔍 Processing sheet: {sheet_name}")

            # original_csv = os.path.join(output_dir, f"{sheet_name}_original.csv")
            # masked_csv = os.path.join(output_dir, f"{sheet_name}_masked.csv")
            # restored_csv = os.path.join(output_dir, f"{sheet_name}_restored.csv")

            # df.to_csv(original_csv, index=False)

            entity_columns = analyze_column(df)
            print(f"Detected entities: {entity_columns}")

            # masked_df = mask_dataframe(df.copy())
            # masked_df = mask_comment_columns(masked_df, forward_mapping)
            # restored_df = unmask_dataframe(masked_df.copy())

            # masked_df.to_csv(masked_csv, index=False)
            # restored_df.to_csv(restored_csv, index=False)

            # compare_files(original_csv, restored_csv)

            # save_mapping(f'{input_file}-->{sheet_name}')
            # entity_columns.clear()


    elif file_ext == ".csv":
        df = pd.read_csv(input_file, dtype=str, low_memory=False)
        print(f"\n🔍 Processing CSV: {input_file}")

        original_csv = os.path.join(output_dir, "original.csv")
        masked_csv = os.path.join(output_dir, "anonymized.csv")
        restored_csv = os.path.join(output_dir, "restored.csv")

        if not os.path.exists(output_dir): os.makedirs(output_dir)

        df.to_csv(original_csv, index=False)

        entity_columns = analyze_column(df)
        print(f"Detected entities: {entity_columns}")

        masked_df = mask_dataframe(df.copy())
        masked_df = mask_comment_columns(masked_df, forward_mapping)
        restored_df = unmask_dataframe(masked_df.copy())

        masked_df.to_csv(masked_csv, index=False)
        restored_df.to_csv(restored_csv, index=False)

        compare_files(original_csv, restored_csv)

        save_mapping(input_file)
    else:
        raise ValueError("Unsupported file type. Use .csv or .xlsx only.")

if __name__ == "__main__":
    input_file = "SO(real).xlsx"
    file_ext = os.path.splitext(input_file)[-1].lower()
    process_file(input_file, file_ext)


🔍 Processing sheet: Open demand

⏳ Execution time analyze_column: 24.099772 seconds
Detected entities: {'Resource Request ID': 'ID', 'Priority': 'US_DRIVER_LICENSE', 'City': 'LOCATION', 'State': 'LOCATION', 'Country': 'COUNTRY', 'Altenate Location': 'ORG', 'Campus': 'ORG', 'Job Grade': 'US_DRIVER_LICENSE', 'RR Start Date': 'DATE_TIME', 'RR End Date': 'DATE_TIME', 'Account Name': 'PERSON', 'Project ID': 'ID', 'Project Name': 'PERSON', 'WFME': 'PERSON', 'WFME ID': 'ID', 'HM': 'PERSON', 'HM ID': 'ID', 'AM': 'PERSON', 'AM ID': 'ID', 'Actual Currency': 'ORG', 'Currency': 'LOCATION', 'Replacement Type': 'PERSON', 'Client Job Title': 'US_DRIVER_LICENSE', 'OBU Name': 'PERSON', 'Project Start Date': 'DATE_TIME', 'Project End Date': 'DATE_TIME', 'Raised On': 'DATE_TIME', 'RR Finance Approved Date': 'DATE_TIME', 'Edit Requested Date': 'DATE_TIME', 'Resubmitted Date': 'DATE_TIME', 'Resubmitted Reason2': 'DATE_TIME', 'Recruiter Name': 'PERSON', 'Recruiter ID': 'ID', 'Last Updated On': 'DATE_TIME',

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp(" debugged energy grid anomalies.")

for token in doc:
  print(token.text, token.pos_)
  if token.pos_ == "PROPN" or token.ent_type_:
      print(token.text)
      # print(reverse_mapping['names'].get(token.text))


Rem PROPN
Rem
debugged VERB
energy NOUN
grid NOUN
anomalies NOUN
. PUNCT


In [ ]:
# Seeing the differences b/w codes
import pandas as pd

def difference_in_csv_sm(path):
    original_df = pd.read_csv(path)
    restored_df = pd.read_csv('./smaller_30k_companies/restored.csv')
    diff_df = original_df.compare(restored_df, keep_equal=False)

    print(diff_df.head(10))
    diff_df.to_csv('difference.csv', index=False)

def difference_in_csv(path):
    original_df = pd.read_csv(path)
    restored_df = pd.read_csv('./for_colab_test/restored.csv')
    anonymized_df = pd.read_csv('./for_colab_test/anonymized.csv')

    diff_df = original_df.compare(restored_df, keep_equal=False)

    diff_df.columns = [
        (col[0], 'original' if col[1] == 'self' else 'restored')
        for col in diff_df.columns
    ]
    diff_df.columns = pd.MultiIndex.from_tuples(diff_df.columns)

    diff_df = diff_df.reset_index()

    anonymized_rows = anonymized_df.loc[diff_df['index']]
    anonymized_rows = anonymized_rows.reset_index(drop=True)
    diff_df = diff_df.drop(columns='index')

    # Flatten multi-index columns for original/restored
    diff_flat = pd.DataFrame()
    for col in diff_df.columns.get_level_values(0).unique():
        diff_flat[f'{col} (original)'] = diff_df[col]['original']
        diff_flat[f'{col} (restored)'] = diff_df[col]['restored']

    for col in anonymized_rows.columns:
        if 'comment' in col.lower():
            diff_flat[f'{col} (anonymized)'] = anonymized_rows[col]

    diff_flat.to_csv('differences.csv', index=False)
    print(diff_flat.head(10))

def difference_in_excel(path):
  original_df = pd.read_excel(path)
  restored_df = pd.read_excel('restored.xlsx')
  diff_df = original_df.compare(restored_df, keep_equal=False)
  print(diff_df.head(10))
  diff_df.to_excel('differences.xlsx')

def difference_in_parquet(path):
  original_df = pd.read_parquet(path)
  restored_df = pd.read_parquet('Sheet1_restored.parquet')
  diff_df = original_df.compare(restored_df, keep_equal=False)
  print(diff_df.head(10))
  diff_df.to_parquet('differences.parquet')

difference_in_csv_sm('smaller_30k_companies.csv')
# difference_in_csv('for_colab_test.csv')
# difference_in_excel()
# difference_in_parquet()

Empty DataFrame
Columns: []
Index: []


In [ ]:
# Testing out fuzzy match
import nltk

nltk.download('stopwords')

text = '''Ford-Mendez Corporation Inc., founded in 1911, operates in the information
technology and services industry. Based in Port Brett, Delaware Region, Afghanistan,
the company has over 10,000 employees and a current employee estimate of 274,047.
The total employee estimate is 716,906. For more details, you can visit their LinkedIn page here.
Additionally, a key employee at the company, A Aaban Aabarana, has been working there for several years.
Kelly, Hobbs and Garcia Industries International, established in 1968,
is also a major player in the information technology and services sector. Located in Jefferyburgh,
South Dakota, Albania, this company too has over 10,000 employees, with a current employee estimate
of 190,771 and a total employee estimate of 341,369. You can learn more about the company by visiting
their LinkedIn profile here. One of their top executives, Aabarna Aabarniga, has contributed greatly to
their success.
'''

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Isolating verbs and adjectives from text
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

common_verbs = {
    'is', 'was', 'are', 'were', 'be', 'being', 'been', 'have', 'has', 'had',
    'get', 'gets', 'got', 'do', 'does', 'did', 'can', 'could', 'will', 'would',
    'shall', 'should', 'may', 'might', 'must', 'found', 'run', 'operate', 'used',
    'work', 'working', 'known', 'help', 'helped', 'helping', 'contributed', 'success', 'successful',
    'unsuccessful'
}

def get_entity_candidates(text):
    words = re.findall(r'\b\w+\b', text)
    filtered = [
        word for word in words
        if word.lower() not in stop_words
        and word.lower() not in common_verbs
        and len(word) > 2
        # and not word.isdigit()
    ]
    return filtered

corporus = "Jams Bond was found operating a project called EnrgyNet in New Yrk in 2023. It helps clients."
print(get_entity_candidates(corporus))


['Jams', 'Bond', 'operating', 'project', 'called', 'EnrgyNet', 'New', 'Yrk', '2023', 'helps', 'clients']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.3 MB/s eta 0:00:00


In [ ]:
# Testing out matching using fuzzy
import re

verb_roots = {'be', 'is', 'have', 'do', 'say', 'go', 'can', 'get', 'make', 'know', 'think', 'take', 'see', 'come', 'want', 'look', 'use', 'find', 'give', 'tell', 'work', 'call', 'try', 'ask', 'need', 'feel', 'become', 'leave', 'put', 'mean', 'keep', 'let', 'like', 'help'}
adj_roots = {'good', 'manag', 'new', 'first', 'last', 'long', 'great', 'little', 'own', 'other', 'old', 'right', 'big', 'high', 'different', 'small', 'large', 'next', 'early', 'young', 'important'}
adv_roots = {'very', 'too', 'really', 'just', 'quite', 'soon', 'again', 'still', 'always', 'often'}
prepositions = {'in', 'on', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'of', 'off', 'a', 'the', 'too', 'and', 'or', 'what', 'else', 'if'}
punctuations = {'.', ',', ';', ':', '!', '?', '(', ')', '[', ']', '{', '}', '"', "'"}
common_words = {'father', 'mother', 'daughter', 'son', 'aunty', 'aunt', 'uncle', 'sister', 'brother', }

def is_excluded(word):
    w = word.lower()
    return (
        any(re.fullmatch(rf"{root}(s|ed|ing)?", w) for root in verb_roots) or
        any(re.fullmatch(rf"{root}(er|est|ly)?", w) for root in adj_roots) or
        any(re.fullmatch(rf"{root}(ly|ing)?", w) for root in adv_roots) or
        any(re.fullmatch(rf"{root}(s|'s|ed|ly|ing)?", w) for root in common_words) or
        w in prepositions or
        w in punctuations
    )


In [ ]:
from rapidfuzz import process, fuzz

def correct_word(word, valid_names, threshold=65):
    match = process.extractOne(word, valid_names, scorer=fuzz.ratio)
    return match[0] if match and match[1] >= threshold else None

@time_it
def isolate_and_correct_entities(text, valid_names):
    tokens = text.split()
    entities = []

    for idx, word in enumerate(tokens):
        if not is_excluded(word):
            corrected = correct_word(word, valid_names)
            if corrected:
                entities.append((idx, word, corrected.capitalize()))

    return entities


In [ ]:
text = """Theressa is a manager working for Wiliams. Ruthu Jenkinz is the mother of
  Fransis Waner and Kaitlyn Meyers."""

valid_names = [name.lower() for name in forward_mapping['names']]
entities = isolate_and_correct_entities(text, valid_names)

for idx, original, corrected in entities:
    print(f"Position {idx}: '{original}' corrected to '{corrected}'")

tokens = text.split()
for idx, original, corrected in entities:
    tokens[idx] = corrected

corrected_text = ' '.join(tokens)

print("Corrected text:")
print(corrected_text)


⏳ Execution time isolate_and_correct_entities: 0.007842 seconds
Position 0: 'Theressa' corrected to 'Theresa'
Position 6: 'Wiliams.' corrected to 'Williams'
Position 7: 'Ruthu' corrected to 'Ruth'
Position 8: 'Jenkinz' corrected to 'Jenkins'
Position 13: 'Fransis' corrected to 'Francis'
Position 14: 'Waner' corrected to 'Wagner'
Position 16: 'Kaitlyn' corrected to 'Kaitlyn'
Position 17: 'Meyers.' corrected to 'Meyers'
Corrected text:
Theresa is a manager working for Williams Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers


In [ ]:
import time
from rapidfuzz import process, fuzz

def correct_word(word, valid_names, threshold=65):
    name_match = process.extractOne(word, valid_names, scorer=fuzz.ratio)
    if name_match and name_match[1] >= threshold: return name_match[0]
    return None


start_time = time.time()
f_mapping_names = [key.lower() for key in forward_mapping['names']]
end_time = time.time()

print(f'Mapping list time: {end_time-start_time}')
text = """Theressa is a manager working for Wiliams Ruthj Jenkinz Fransis Waner Kaitlyn Meyers Nicolas Stevans
Dona Berd Solushunz Stefanie Miller.""".split()

start_time = time.time()
results = [correct_word(word, f_mapping_names) for word in text]
end_time = time.time()
print(f'Result: {end_time-start_time}')

print(results)

['Alexandra', 'Alice', 'Armstrong', 'Arthur', 'Ariel', 'Alex', 'Ashley', 'Alejandro', 'Antonio', 'Andrew']
Mapping list time: 0.0009248256683349609
Result: 0.003665447235107422
['theresa', 'lisa', None, 'maynard', 'wong', 'ford', 'williams', 'ruth', 'jenkins', 'francis', 'wagner', 'kaitlyn', 'meyers', 'nicolas', 'evans', 'donna', 'beard', None, 'stefanie', 'miller']


In [ ]:
# Testing entity isolation and correction
import spacy
import re
from rapidfuzz import process, fuzz

# Load spaCy small English model (you can also try 'en_core_web_md' if you want better tagging)
nlp = spacy.load('en_core_web_sm')

verb_roots = {'be', 'is', 'have', 'do', 'say', 'go', 'can', 'get', 'make', 'know', 'think', 'take', 'see', 'come', 'want', 'look', 'use', 'find', 'give', 'tell', 'work', 'call', 'try', 'ask', 'need', 'feel', 'become', 'leave', 'put', 'mean', 'keep', 'let', 'like', 'help'}
adj_roots = {'good', 'manag', 'new', 'first', 'last', 'long', 'great', 'little', 'own', 'other', 'old', 'right', 'big', 'high', 'different', 'small', 'large', 'next', 'early', 'young', 'important'}
adv_roots = {'very', 'too', 'really', 'just', 'quite', 'soon', 'again', 'still', 'always', 'often'}
prepositions = {'in', 'on', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'of', 'off', 'a', 'the', 'too', 'and', 'or', 'what', 'else', 'if'}
punctuations = {'.', ',', ';', ':', '!', '?', '(', ')', '[', ']', '{', '}', '"', "'"}
common_words = {'father', 'mother', 'daughter', 'son', 'aunty', 'aunt', 'uncle', 'sister', 'brother'}

def is_excluded(word):
    w = word.lower()
    return (
        any(re.fullmatch(rf"{root}(s|ed|ing)?", w) for root in verb_roots) or
        any(re.fullmatch(rf"{root}(er|est|ly)?", w) for root in adj_roots) or
        any(re.fullmatch(rf"{root}(ly|ing)?", w) for root in adv_roots) or
        any(re.fullmatch(rf"{root}(s|'s|ed|ly|ing)?", w) for root in common_words) or
        w in prepositions or
        w in punctuations
    )

def correct_word(word, valid_names, threshold=65):
    match = process.extractOne(word, valid_names, scorer=fuzz.ratio)
    return match[0] if match and match[1] >= threshold else None

def isolate_and_correct_entities_spacy(text, valid_names, threshold=65):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    corrected_tokens = tokens.copy()

    for i, token in enumerate(doc):
        if token.is_punct or token.is_space:
            continue

        if token.pos_ in {"NOUN", "PROPN"}:
            word = token.text
            if not is_excluded(word):
                corrected = correct_word(word, valid_names, threshold)
                if corrected:
                    corrected_tokens[i] = corrected.capitalize()

    corrected_text = ''.join([
        token if token in punctuations or token.isspace() else ' ' + token
        for token in corrected_tokens
    ]).strip()

    return corrected_text

# --- Example usage:

text = """Theressa is a manager working for Wiliams. Ruthu Jenkinz is the mother of Fransis Waner and Kaitlyn Meyers."""

# Assume forward_mapping['names'] already loaded somewhere
valid_names = [name.lower() for name in forward_mapping['names']]

corrected_text = isolate_and_correct_entities_spacy(text, valid_names)
print(corrected_text)


Original: Nw York, Corrected: New York, Position: (15, 22), Type: GPE


In [ ]:
!pip install gliner faker rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46

In [ ]:
from gliner import GLiNER
from rapidfuzz import process, fuzz

model = GLiNER.from_pretrained("urchade/gliner_base")

def correct_word(word, valid_list, threshold=65):
    match = process.extractOne(word, valid_list, scorer=fuzz.ratio)
    return match[0] if match and match[1] >= threshold else word  # fallback if no good match

def find_and_correct_entities(text, valid_names, valid_company):
    entities = model.predict_entities(text, labels=["person", "organization"])

    corrected_entities = []

    for ent in entities:
        entity_text = ent['text']
        start = ent['start']
        end = ent['end']
        label = ent['label']

        if label == "person":
            tokens = entity_text.split()
            corrected_tokens = [
                correct_word(token.lower(), valid_names).capitalize()
                for token in tokens
            ]
            corrected = " ".join(corrected_tokens)

        elif label == "organization":
            corrected = correct_word(entity_text.lower(), valid_company).capitalize()

        else: corrected = entity_text

        corrected_entities.append({
            'original': entity_text,
            'corrected': corrected,
            'start': start,
            'end': end,
            'label': label
        })

    return corrected_entities

def replace_entities_in_text(text, entities):
    entities = sorted(entities, key=lambda x: x['start'], reverse=True)

    for ent in entities:
        text = text[:ent['start']] + ent['corrected'] + text[ent['end']:]
    return text

@time_it
def main():
  text = """Theressa is a manager working for Wiliams. Ruthu Jenkinz is the mother of Fransis Waner and Kaitlyy Meeeers.
  Donaa Gavln hasn't been seen for a while now by Nickol. THis is concerning!"""
  valid_names = [name.lower() for name in forward_mapping['names']]  # should be tokens
  valid_company = [company.lower() for company in forward_mapping['company']]

  entities = find_and_correct_entities(text, valid_names, valid_company)
  corrected_text = replace_entities_in_text(text, entities)

  print("\nCorrected Text:\n", corrected_text)

main()


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Corrected Text:
 Theresa is a manager working for Williams. Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers.
  Donna Gavin hasn't been seen for a while now by Nikon. THis is concerning!

⏳ Execution time main: 0.805174 seconds

Corrected Text:
 Theresa is a manager working for Williams. Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers.
  Donna Gavin hasn't been seen for a while now by Nikon. THis is concerning!

⏳ Execution time main: 0.640440 seconds

Corrected Text:
 Theresa is a manager working for Williams. Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers.
  Donna Gavin hasn't been seen for a while now by Nikon. THis is concerning!

⏳ Execution time main: 0.651691 seconds

Corrected Text:
 Theresa is a manager working for Williams. Ruth Jenkins is the mother of Francis Wagner and Kaitlyn Meyers.
  Donna Gavin hasn't been seen for a while now by Nikon. THis is concerning!

⏳ Execution time main: 0.645857 seconds

Corrected Text:
 Theres

In [ ]:
from gliner import GLiNER
from rapidfuzz import process, fuzz
from faker import Faker
import json
import time

model = GLiNER.from_pretrained("urchade/gliner_base")
faker = Faker()

forward_mapping = {
    "names": {
        "Theresa": "Aanik",
        "Fransis": "Ben",
        'Alvin' : 'Sam',
        'Albin' : 'Bond',
        "Kaitlyn": "Lora",
        "Nickol": "Chris",
        "Dona": "Alia",
        "Ruth": "Fay"
    },
    "company": {
        "Williams-Waller Co": "ibm",
        "Tata Consultancy Services": "tcs",
        "IBM": "oracle"
    }
}

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"\nExecution time: {end - start:.4f} seconds")
        return result
    return wrapper

def correct_with_forward_mapping(word, mapping_dict, threshold=85):
    match = process.extractOne(word, mapping_dict.keys(), scorer=fuzz.ratio)
    if match and match[1] >= threshold:
        return match[0], mapping_dict[match[0]], False
    return word, None, True  # new word

def find_and_correct_entities(text):
    entities = model.predict_entities(text, labels=["person", "organization"])
    corrected_entities = []

    new_entity_mappings = {
        "forward_mapping": {},
        "reverse_mapping": {}
    }

    for ent in entities:
        entity_text = ent['text']
        start = ent['start']
        end = ent['end']
        label = ent['label']

        if label == "person":
            tokens = entity_text.split()
            corrected_tokens = []
            for token in tokens:
                original_token = token.strip()
                mapped_key, mapped_value, is_new = correct_with_forward_mapping(original_token, forward_mapping['names'])
                if is_new:
                    fake_name = faker.first_name()
                    new_entity_mappings["forward_mapping"][original_token] = fake_name
                    new_entity_mappings["reverse_mapping"][fake_name] = original_token
                    corrected_tokens.append(mapped_key)
                else:
                    corrected_tokens.append(mapped_key)
            corrected = " ".join(corrected_tokens)

        elif label == "organization":
            mapped_key, mapped_value, is_new = correct_with_forward_mapping(entity_text, forward_mapping['company'])
            if is_new:
                fake_company = faker.company()
                new_entity_mappings["forward_mapping"][entity_text] = fake_company
                new_entity_mappings["reverse_mapping"][fake_company] = entity_text
                corrected = mapped_key
            else:
                corrected = mapped_key

        else:
            corrected = entity_text  # not person/org

        corrected_entities.append({
            'original': entity_text,
            'corrected': corrected,
            'start': start,
            'end': end,
            'label': label
        })

    return corrected_entities, new_entity_mappings

def replace_entities_in_text(text, entities):
    entities = sorted(entities, key=lambda x: x['start'], reverse=True)
    for ent in entities:
        text = text[:ent['start']] + ent['corrected'] + text[ent['end']:]
    return text

@time_it
def main():
    text = """Theressa is a manager working for Wiliams. Ruthu Jenkinz is the mother of Fransis Waner and Kaitlyy Meeeers.
    Donaa Gavln hasn't been seen for a while now by Nickol. THis is concerning! Allin is here btw"""

    entities, new_entity_mappings = find_and_correct_entities(text)
    corrected_text = replace_entities_in_text(text, entities)

    print("\nCorrected Text:\n", corrected_text)

    print("\nOld Entity Mappings:\n", json.dumps(forward_mapping, indent=2))
    print("\nNew Entity Mappings:\n", json.dumps(new_entity_mappings, indent=2))

main()


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Corrected Text:
 Theresa is a manager working for Wiliams. Ruth Jenkinz is the mother of Fransis Waner and Kaitlyn Meeeers.
    Dona Gavln hasn't been seen for a while now by Nickol. THis is concerning! Allin is here btw

Old Entity Mappings:
 {
  "names": {
    "Theresa": "Aanik",
    "Fransis": "Ben",
    "Alvin": "Sam",
    "Albin": "Bond",
    "Kaitlyn": "Lora",
    "Nickol": "Chris",
    "Dona": "Alia",
    "Ruth": "Fay"
  },
  "company": {
    "Williams-Waller Co": "ibm",
    "Tata Consultancy Services": "tcs",
    "IBM": "oracle"
  }
}

New Entity Mappings:
 {
  "forward_mapping": {
    "Wiliams": "Clark-Chung",
    "Jenkinz": "Christopher",
    "Waner": "Kayla",
    "Meeeers": "Jennifer",
    "Gavln": "Jason",
    "Nickol": "Ingram and Sons",
    "Allin": "Tina"
  },
  "reverse_mapping": {
    "Clark-Chung": "Wiliams",
    "Christopher": "Jenkinz",
    "Kayla": "Waner",
    "Jennifer": "Meeeers",
    "Jason": "Gavln",
    "Ingram and Sons": "Nickol",
    "Tina": "Allin"
  }
}


In [ ]:
from rapidfuzz import process, fuzz

lst = ['Albin', 'Alvin', 'Alwin', ]
matching_word = process.extractOne('Allin', lst, scorer=fuzz.ratio)
print(matching_word)

('Albin', 80.0, 0)


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

text = """I123-43GXP-APR-2024 I123-Kumar-Airfare-INTL-43GXP-TRV/DOH/SEA-APR-2024 I123-Kumar-Airfare-INTL-43GX-TRV/DOH/SEA-APR-2024"""

doc = nlp(text)

print("Entities:")
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")


Entities:
I123-43GXP (PRODUCT)
I123-Kumar-Airfare (PRODUCT)
I123-Kumar-Airfare-INTL-43GX-TRV/DOH/SEA (PRODUCT)


In [ ]:
import re
from gliner import GLiNER
@time_it
def identify_entities():
  model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")
  labels = ["BookingCode", "Person", "TravelRoute", "Date", "Location Code"]
  text = """I123-43GXP-APR-2024 I123-Kumar-Airfare-INTL-43GXP-TRV/DOH/SEA-APR-2024 I123-Kumar-Airfare-INTL-43GX-TRV/DOH/SEA-APR-2024"""
  text = ' '.join(re.split(r'[-\n]', text))

  print(text, end='\n\n')
  entities = model.predict_entities(text, labels=labels)
  for entity in entities:
    print(f"{entity['text']} -> {entity['label']}")

identify_entities()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


I123 43GXP APR 2024 I123 Kumar Airfare INTL 43GXP TRV/DOH/SEA APR 2024 I123 Kumar Airfare INTL 43GX TRV/DOH/SEA APR 2024

I123 43GXP -> BookingCode
APR 2024 -> Date
I123 -> Person
Kumar -> Person
INTL 43GXP -> BookingCode
TRV/DOH/SEA -> TravelRoute
APR 2024 -> Date
I123 -> Person
INTL 43GX -> BookingCode
APR 2024 -> Date


In [ ]:
!pip install gliner polars rapidfuzz pyxlsb fastexcel python-calamine gliner-spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.0/886.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [ ]:
import json
import time
import polars as pl
import pandas as pd
from tqdm import tqdm
from gliner import GLiNER

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"\nExecution time: {end - start:.4f} seconds")
        return result
    return wrapper

model = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5")

labels = [
    "Person name", "Organization name",
    "BookingCode", "Location", "Location Code",
    "Generic ID", "TravelRoute"
]

@time_it
def reading_descriptive_excel():
  df_pd = pd.read_excel("Desc-Data-Testing-1000.xlsx", engine="calamine")
  df = pl.from_pandas(df_pd)

  if "DESCRIPTION" not in df.columns:
      raise ValueError("Missing 'DESCRIPTION' column in the Excel file")

  results = []

  for desc in tqdm(df["DESCRIPTION"].to_list(), desc="Extracting Entities"):
      entities = model.predict_entities(str(desc), labels=labels)
      results.append({
          "text": desc,
          "entities": entities
      })

  with open("extracted_entities.json", "w", encoding="utf-8") as f:
      json.dump(results, f, indent=2, ensure_ascii=False)

  print("\n\n✅ JSON saved as 'extracted_entities.json'")

reading_descriptive_excel()

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting Entities: 100%|██████████| 1004/1004 [06:39<00:00,  2.51it/s]



✅ JSON saved as 'extracted_entities.json'

Execution time: 399.7302 seconds


In [ ]:
import polars as pl
from gliner import GLiNER
import json
import random
import string
import gzip
import os
from tqdm import tqdm

model = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5")

label_to_fake_category = {
    "Person": "names",
    "BookingCode": "ID-code",
    "Location": "location",
    "TravelRoute": "location",
    # "Date": None  # Skip this
}

def load_faker_data(path="faker_data_v3.json.gz"):
    with gzip.open(path, 'rt', encoding='utf-8') as f:
        return json.load(f)

def generate_booking_code():
    return "ID-" + ''.join(random.choices(string.digits, k=6))

def generate_fake_value(label, faker_data, used_fakes):
    if label == "BookingCode":
        while True:
            code = generate_booking_code()
            if code not in used_fakes:
                used_fakes.add(code)
                return code
    category = label_to_fake_category.get(label)
    if category not in faker_data:
        raise ValueError(f"No fake data available for label '{label}'")
    for val in faker_data[category]:
        if val not in used_fakes:
            used_fakes.add(val)
            return val
    raise ValueError(f"Exhausted fake values for label '{label}'")

def anonymize_excel_description(input_excel_path: str, output_json_path: str = None):
    df = pl.read_excel(input_excel_path)
    if "Description" not in df.columns: raise ValueError("Missing 'Description' column in the Excel file")

    input_filename = os.path.splitext(os.path.basename(input_excel_path))[0]
    mapping_path = f"{input_filename}.mapping.json"
    anonymized_path = f"{input_filename}.anonymized.json"
    restored_path = f"{input_filename}.restored.json"

    faker_data = load_faker_data()

    forward_mapping = {}
    reverse_mapping = {}
    used_fakes = set()

    results = []
    restored_results = []

    for desc in tqdm(df["Description"].to_list(), desc="Extracting and Anonymizing"):
        original_text = ' '.join(desc.split('-'))
        entities = model.predict_entities(str(original_text), labels=list(label_to_fake_category.keys()))

        replacements = []

        for ent in sorted(entities, key=lambda x: x['start'], reverse=True):
            label = ent['label']
            text = ent['text']

            if label == "Date": continue

            if text not in forward_mapping:
                fake_value = generate_fake_value(label, faker_data, used_fakes)
                forward_mapping[text] = fake_value
                reverse_mapping[fake_value] = text
            else:
                fake_value = forward_mapping[text]

            replacements.append((ent['start'], ent['end'], fake_value))

        # Anonymize
        anon_text = original_text
        for start, end, fake in replacements:
            anon_text = anon_text[:start] + fake + anon_text[end:]

        results.append({"original": original_text, "anonymized": anon_text})
        # Restore
        restored = anon_text
        for fake_val, original_val in reverse_mapping.items():
            restored = restored.replace(fake_val, original_val)
        restored_results.append({"anonymized": anon_text, "restored": restored})

    with open(mapping_path, "w", encoding="utf-8") as f:
        json.dump({"forward_mapping": forward_mapping, "reverse_mapping": reverse_mapping}, f, indent=2, ensure_ascii=False)

    with open(anonymized_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    with open(restored_path, "w", encoding="utf-8") as f:
        json.dump(restored_results, f, indent=2, ensure_ascii=False)

    print(f"\n✅ Anonymization complete.\n- Mapping: {mapping_path}\n- Anonymized: {anonymized_path}\n- Restored: {restored_path}")

anonymize_excel_description("Desc-Data-Testing.xlsx")


In [ ]:
import time

start = time.time()
from gliner import GLiNER
print("🔍 Loading GLiNER model once outside...")
gliner_model = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5")
end = time.time()
print(f"✅ Model loaded in {end - start:.2f} seconds")

🔍 Loading GLiNER model once outside...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

models_comparison.png:   0%|          | 0.00/156k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

✅ Model loaded in 51.29 seconds


In [ ]:
# Testing identification of entities in descriptive data
import re
import polars as pl
import pandas as pd
from functools import wraps
from collections import Counter
from typing import List, Tuple, Dict, Any

class EntityExtractor:
    def __init__(self, model):
        print("🔍 Loading GLiNER model...")
        self.model = model
        self.labels = ["Name of Person", "Organization", "Location", "BookingID", "ID",]
        self.descriptive_keywords = {
            "description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation", "skills",
            "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience", 'subskill', 'subkills',
            "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report", "skill",
            "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text", "reason"
        }

    @staticmethod
    def time_it(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(f"\n⏱️ Starting '{func.__name__}'...")
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print(f"✅ Finished '{func.__name__}' in {end - start:.2f} seconds.")
            return result
        return wrapper

    def analyze_batch(self, texts: List[str]) -> List[List[Dict[str, Any]]]:
        """Process a batch of texts and return list of entity dictionaries with positions."""
        results = []
        for text in texts:
            try:
                entities = self.model.predict_entities(text, self.labels)
                results.append([
                    {
                        "label": ent["label"],
                        "text": ent["text"],
                        "start": ent["start"],
                        "end": ent["end"]
                    }
                    for ent in entities
                ])
            except Exception: results.append([])
        return results

    @time_it
    def process_excel(self, file_path: str, batch_size: int = 100) -> dict:
        print(f"📥 Reading Excel file: {file_path}")
        df_pd = pd.read_excel(file_path, engine="calamine")

        column_entities = {}

        for col in df_pd.columns:
            if any(re.search(rf"\b{re.escape(keyword)}\b", col, re.IGNORECASE) for keyword in self.descriptive_keywords):
                column_entities[col] = "Descriptive"
                continue

            if 'id' in col.lower():
                column_entities[col] = 'ID'
                continue

            values = df_pd[col].dropna().astype(str).unique()[:5]
            values = [str(v) for v in values if str(v).strip()]
            combined_text = ". ".join(values)

            print(f"🔍 Analyzing column: {col}")
            entities = self.analyze_batch([combined_text])[0]
            label_counter = Counter(ent["label"] for ent in entities)

            if label_counter:
                most_common_label, _ = label_counter.most_common(1)[0]
                column_entities[col] = most_common_label
            else:
                column_entities[col] = None

        print(f"\n🧾 Finished analyzing {len(df_pd.columns)} columns.")
        return column_entities


    @time_it
    def save_to_parquet(self, df: pl.DataFrame, output_path: str):
        print(f"💾 Saving to {output_path}")
        df.write_parquet(output_path)

if __name__ == "__main__":
    file_path = "SO(real).xlsx"
    output_path = "entities_with_positions.parquet"
    start = time.time()
    extractor = EntityExtractor(gliner_model)
    df = extractor.process_excel(file_path)
    end = time.time()
    print(df)
    print(f"Total time: {end - start:.2f} seconds")

🔍 Loading GLiNER model...

⏱️ Starting 'process_excel'...
📥 Reading Excel file: SO(real).xlsx


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🔍 Analyzing column: RR Status
🔍 Analyzing column: RR Type
🔍 Analyzing column: Priority
🔍 Analyzing column: UST - Role
🔍 Analyzing column: City
🔍 Analyzing column: State
🔍 Analyzing column: Country
🔍 Analyzing column: Altenate Location
🔍 Analyzing column: Campus
🔍 Analyzing column: Job Grade
🔍 Analyzing column: RR Start Date
🔍 Analyzing column: RR End Date
🔍 Analyzing column: Account Name
🔍 Analyzing column: Project Name
🔍 Analyzing column: WFME
🔍 Analyzing column: HM
🔍 Analyzing column: AM
🔍 Analyzing column: Billable
🔍 Analyzing column: Actual Bill Rate
🔍 Analyzing column: Actual Currency
🔍 Analyzing column: Bill Rate
🔍 Analyzing column: Billing Frequency
🔍 Analyzing column: Currency
🔍 Analyzing column: Target ECR
🔍 Analyzing column: Accepted Resource Type
🔍 Analyzing column: Replacement Type
🔍 Analyzing column: Exclusive to UST
🔍 Analyzing column: Contract to Hire
🔍 Analyzing column: Client Job Title
🔍 Analyzing column: Client Evaluation
🔍 Analyzing column: OBU Name
🔍 Analyzing colum

In [ ]:
#

import time
from gliner import GLiNER

start = time.time()
# model = GLiNER.from_pretrained(
#     "juampahc/gliner_multi-v2.1-onnx",
#     load_onnx_model=True,
#     onnx_model_file="model.onnx"
# )
model_small = GLiNER.from_pretrained(
    "juampahc/gliner_multi-v2.1-onnx",
    load_onnx_model=True,
    onnx_model_file="model.onnx"
)
end = time.time()
print(f'📈📈Model loading time: {end - start:.2f}')

labels = ["Name of Person", "Organization", "Location", "TravelRoute", "ID"]

start_time = time.time()
entities = model.predict_entities(string, labels, flat_ner=True, threshold=0.5)
end_time = time.time()

print(f"\n⏱️ Inference Time: {end_time - start_time:.4f} seconds")
print("🔍 Detected Entities:")
for ent in entities:
    print(f"🔹 '{ent['text']}' → 🏷️ {ent['label']} 📍({ent['start']}-{ent['end']})")

In [ ]:
!pip install gliner polars rapidfuzz pyxlsb fastexcel python-calamine gliner-spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.0/886.0 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [ ]:
string = """
"Looking for a Cybersecurity Project Manager (Cybersecurity PM) who could involve in planning, executing, and overseeing cybersecurity projects to safeguard an organization's digital assets and data, ensuring compliance, and mitigating risks.

Responsibilities

•	Defining project scope, objectives, and deliverables.
•	Developing detailed project plans and timelines.
•	Tracking project progress, identifying potential issues, and implementing corrective actions.
•	Working closely with technical teams, stakeholders, and management.
•	Communicating project status, risks, and issues effectively.

Skillset

•	5+ Years of Agile PM expertise
•	Understanding of cybersecurity principles, technologies, and threats.
•	Ability to effectively communicate with technical and non-technical audiences.
•	Understanding of risk assessment and mitigation techniques.

"
Same as above
The ideal candidate will be skilled in: • Process discovery and optimization • Data analysis, visualization, and dashboarding • Identifying opportunities to improve our processes and systems, especially in finance domain. A strong business architecture mindset is a plus, with experience or familiarity in Various ERP system like Oracle and SAP. Expertise in various tools like  Process Mining tools • BizzDesign / Business Architecture tools • Power BI • Power Apps • Qualtrics Analysis • User journey mapping • Problem statement generation • Developing actionable plans
"
The Opportunity I am looking for a Technical Data Analyst who has hands on experience migrating Sales users and Sales data like Accounts, Opportunities, Contacts from Salesforce.com to Dynamics 365. He should also have strong experience in data profiling, data integration, transformations, understanding how data impacts downstream systems like Sales and Finance data warehouses/data lakes, reporting etc.
What You’ll Do
• Work with Customer internal data teams and business teams to decipher the data related requirements for the project • Need to have hands on experience in understanding the SFDC to Dynamics field mappings & data models. • Design end to end strategy to stitch objects from various systems and financial KPIs • Building integrations between SFDC, Dynamics, SAP ECC and DBX for Power BI reporting and financial metrics. • Leverage data sources across the enterprise to build sophisticated and insightful analyses and data models for Sales, Finance and Marketing • Need to have hands on experience with migrating Marketing and Sales data and users from Salesforce.com to Dynamics • Work with the Product Managers to build detailed data requirements/specifications for Engineering teams to build the solution in downstream data management and reporting systems. • Need to understand the migration challenges from similar experiences and build creative solutions to help with migrating data from SFDC to Dynamics. • Consolidate requirements and suggest building new reporting capabilities for analysis using advanced BI techniques and tools. • Proactively collaborate with various product managers to bring a perspective on all data we work on. • Conducts QA testing and validations and provides inputs to the Engineering teams – along with the PdMs • Support Release Planning, scheduling backlog items into regular releases aligned to business priority while working with the PdM’s • Supports production cutover and Production acceptance testing. • Supports post go live sessions with business, addresses and drives technical issues raised during Hyper Care. This will be done with the PdM’s and Engineering teams.

Qualifications
• Requires bachelor’s degree. Preferred candidates will have a major in computer science, MBA from reputable institution or equivalent experience. • 4+ years of data analytics, ‘data BSA’ or data product management experience with solid understanding of how to deliver data solutions in an agile environment. • Strong proficiency in SQL/SparkSQL/Python to query and manipulate large data sets. Experience with platforms like Databricks, Power BI and Tableau. • You are a self-starter, independent, hard worker, with a high degree of motivation to go above and beyond the task at hand. You anticipate and creatively implement next steps in a complex environment. • You have mastered the ability to influence outcomes, navigate, mediate to consensus with integrity. You possess great interpersonal communication, presentation skills, and social skills and a solid sense of humor. • Data requirement writing skills: collecting, prioritizing, and gathering input from multiple sources, providing accurate requirements with attention to detail. • You already know or can rapidly learn enterprise application capabilities in order to deliver transaction and event-driven data solutions (examples: SAP/HANA, MS Dynamics or Salesforce Data, ADLS/Hadoop/Databricks datalake/lakehouse solutions, and/or Kafka streams)
"
"Lead II - Data Analysis

This is an onsite role with minimum 4 days in office, and work directly under the senior leadership.
Responsibilities:
•	Data collection from various sources, clean & validate the data
•	Analyze the data for the patterns & any anomalies
•	Use predictive models & techniques for insights
•	Create dashboards/reports
Experience:
•	6 plus years of IT experience in Data Analyst/Data Scientist roles
•	Business Knowledge of Telecom eCommerce Omni channel (OSS/BSS)
•	Hands on experience in SQL, Python, Databricks
•	Hands-on experience in Data Visualization Tools – Tableau (preferred) or Power BI.
•	Hands-on experience in Data Analytics, AL/ML – LLM and do predictive analysis
•	Familiarity with ETL, Data pipelines
•	Strong analytical and problem solving skills
•	Experience with cloud platform – AWS/Azure/Google
•	Able to work with less or no guidance
"
"Responsibilities:
Develop Software: Design, develop, test, and deploy high-quality applications and microservices in both Java and Python.

Code Optimization: Ensure optimal performance and efficiency of applications by writing efficient, reusable, and maintainable code.

Collaboration: Work closely with cross-functional teams, including product managers, designers, and QA engineers to build solutions that meet business needs.

Troubleshooting and Debugging: Identify and resolve issues in existing code, ensuring minimal downtime and high availability.

Integration: Integrate third-party APIs, databases, and other systems into the applications.

Code Reviews: Participate in peer code reviews, providing constructive feedback to improve the quality of code across the team.

Documentation: Write technical documentation, including design, architecture, and user guides.

Agile Development: Work in an Agile environment, contributing to sprint planning, backlog grooming, and iterative development cycles.

Security: Adhere to best practices for application security, including secure coding techniques and data protection.

Required Skills & Qualifications:
=================================
Programming Experience: hands-on experience in developing applications using Java (Spring, Hibernate, etc.) and Python (Django, Flask, or other frameworks).

Software Development: Solid understanding of object-oriented programming (OOP) principles and design patterns.

Web Development: Experience in building RESTful APIs and web services using both Java and Python.

Databases: Proficiency with relational databases (e.g., PostgreSQL, MySQL) and NoSQL databases (e.g., MongoDB, Cassandra).

Version Control: Experience with version control systems such as Git, GitHub, or GitLab.

Testing: Knowledge of unit testing and test-driven development (TDD) using tools like JUnit, pytest, or similar frameworks.

Cloud Platforms: Experience working with cloud platforms like AWS, GCP, or Azure, including serverless architecture and containerized applications (e.g., Docker, Kubernetes).

CI/CD: Familiarity with continuous integration and continuous deployment practices, and tools like Jenkins, CircleCI, or GitLab CI.

Problem Solving: Strong problem-solving skills and the ability to think through complex technical challenges.

Collaboration & Communication: Excellent verbal and written communication skills with the ability to work effectively in a team-oriented environment.

Preferred Qualifications:
Microservices Architecture: Experience building and maintaining microservices-based applications.

Frameworks: Familiarity with Java frameworks like Spring Boot, and Python frameworks like Flask or Django.

Cloud-Native: Experience with building cloud-native applications and working with container orchestration tools like Kubernetes.

Message Queues: Experience with message brokers and queues like Kafka, RabbitMQ, or AWS SQS.

Agile Practices: Experience working in Agile software development methodologies (Scrum, Kanban).

CI/CD Pipelines: Advanced knowledge of setting up and managing CI/CD pipelines in a cloud environment.

Security Best Practices: Knowledge of secure coding practices and implementing security measures in web applications."
"Solid foundation and hands-on experience building and operating Kubernetes; Proficiency in deploying, scaling, and managing Kubernetes clusters; Experience in cloud platforms (AWS, Azure or GCP); Hands-on experience with cloud infrastructure provisioning at scale using tools like Terraform, Pulumi, Crossplane, CloudFormation, etc.; Experience developing and maintaining control planes; Proficiency in Python or Golang; Experience in devops and agile for solution delivery through CI/CD; Hands-on experience with automation and development; Ability to work independently and collaboratively

Primary Skills: 1. Understanding of Kubernetes fundamentals and experience building/operating Kubernetes.2. Golang or Python experience 3. DevOps pipeline understanding- Terraform4. Cloud: AWS or Azure or GCPExperience Range 5-8 years of experience in this tech stack. ( dont restrict the search to years of experience)
SO# created for contractor hiring"
"Sr Cloud Network Engineer
Key Responsibilities
• Oversee the network onboarding process for new users and systems into Cloud environment
• Provision and configure network resources in Cloud, ensuring compliance with security policies and government regulations.
• Implement secure network architectures, including Virtual Private Clouds (VPCs), subnets, routing tables, and network access control lists across AWS, Azure, and other cloud platforms.
• Configure and manage cloud networking services for secure connectivity between on-premises and cloud environments, such as AWS Direct Connect, Azure VPN Gateway, and transit gateways.
• Implement network security controls, such as security groups, network firewalls, and web application firewalls to protect against unauthorized access and cyber threats.
• Monitor network traffic and security logs using cloud services for flow logs, activity trails, and threat detection to identify and respond to potential security incidents.
• Collaborate with cross-functional teams to ensure secure integration of applications and services into the cloud network infrastructure.
• Conduct regular network assessments and audits to ensure compliance with internal and external requirements.
• Develop and maintain comprehensive network security policies, procedures, and documentation in compliance with security standards.
• Provide technical support and troubleshooting for Cloud network-related issues
• Stay up-to-date with the latest cloud networking services, security features, and best practices across multiple platforms

Automation Skills:
• Proficiency in Python or Golang for automation
• Strong expertise in Terraform automation with the ability to create custom templates and modules is a plus.
• Hands-on experience in CI/CD implementation and gitlab
• Experience with containerization would be a plus

Qualifications
• Assesses the candidate's experience with the network onboarding process and provisioning in cloud environments such as AWS and Azure.
• Evaluates the candidate's experience with automation tools for network configuration and management.
• Measures the candidate's hands-on experience with cloud networking services across AWS, Azure, and other major cloud providers.
• Assesses the candidate's in-depth knowledge of network security principles, protocols, and best practices for secure network design and implementation.
• Evaluates the candidate's knowledge of government network security standards and compliance requirements.
• Measures the candidate's relevant certifications which indicate their proficiency and expertise in cloud

"
"Project Manager

Responsibilities:
•	Manage multiple work streams/Scrum teams
•	Manage multiple SCRUM teams and manage all SCRUM ceremonies such as Daily Standup, Sprint Planning, Retrospective, Sprint Review meetings.
•	Working with Scrum team, as well as internal and external stakeholders, to influence and drive decision making and support organizational project or product teams.
•	Maintain project timeframe, estimates and status reports.
•	Prepare and present project and program status reports
•	Assist senior leaders in staffing, planning, report preparations
•	Develops, maintains and manages detailed project plans, action item registers and major milestone timelines for all assigned projects.
•	Conduct risk assessments for the projects.
•	Collaborate in preparing for the PI Planning
•	Assist Product Owner and team in backlog refinement.
•	Facilitate Scrum of Scrum meetings with other Scrum teams
•	Coordinate the project release and deployments.
•	Monitor project deliverables.
•	Prepare status reports, presentations and present to stakeholders.
•	Perform other duties and responsibilities as assigned
Experience:
•	10 plus years of IT experience
•	Minimum of at least 4 years of experience in a project management capacity in Project Manager & Scrum Master roles in software development projects
•	Prior experience managing multiple project teams for a common roadmap
•	Experience managing system implementation and integration projects
•	Strong presentation and communication skills
•	Strong conceptual, analytic and problem-solving skills.
•	Strong communication skills
•	Strong project and time management skills.
•	Ability to work under pressure, meet deadlines, and work on multiple projects simultaneously.
•	Must be a team player and be able to effectively interact with stakeholders at all levels
•	Strong leadership capabilities and excellent communication skills with the ability to engage with technical teams and business stakeholders alike
•	Proficiency in creating engaging PowerPoint presentations.

Preferred Experience:
•	Previous experience in the fintech or payment industry.
•	Scrum Master certification and/or PMP certification is preferred
"
"Experienced Senior Software Development Engineer in Test (SDET). Candidate to have significant experience in mobile test automation, with expertise in Appium and Java. Will be responsible for designing and executing automated test scripts, improving test frameworks, and ensuring the highest quality of our mobile applications across both iOS and Android platforms.

Key Responsibilities:
Design and Develop Test Automation Scripts:
Create, implement, and maintain automated test scripts using Appium with Java for both iOS and Android applications to ensure comprehensive test coverage.

Automation Framework Development:
Build, improve, and scale automated test frameworks in Java to support mobile testing, ensuring reusability, maintainability, and scalability.

Mobile Application Testing:
Perform end-to-end testing on mobile applications for compatibility, performance, security, and functionality across various devices, OS versions, and screen resolutions.

Cross-Platform Mobile Testing:
Develop and execute tests across multiple mobile platforms (iOS, Android) using Appium to ensure uniform test execution and results.

CI/CD Integration:
Integrate automated mobile tests into CI/CD pipelines (e.g., Jenkins, GitLab CI, etc.) to ensure continuous testing and faster feedback.

Test Execution and Monitoring:
Monitor automated test executions, debug failures, and provide actionable feedback to developers to ensure issues are quickly addressed and fixed.

Defect Identification and Root Cause Analysis:
Investigate and report defects, tracking them through to resolution while working with the development team to identify root causes and solutions.

Collaboration and Communication:
Work closely with cross-functional teams, including developers, product managers, and other SDETs, to understand application features and contribute to test strategy and planning.

Mentorship and Leadership:
Mentor junior testers and engineers, providing guidance on best practices in mobile automation using Appium and Java.

Test Reporting and Metrics:
Generate reports and analyze automated test results, providing clear and actionable insights to improve mobile application quality.

Required Skills and Qualifications:
Experience:
At least 5+ years of experience in software development or quality assurance, with 3+ years specifically focused on mobile test automation using Appium and Java.

Programming Skills:
Strong proficiency in Java for writing automated test scripts, creating frameworks, and debugging test failures.

Appium Expertise:
Deep knowledge of Appium and its integration with mobile testing on iOS and Android platforms.

Test Frameworks:
Experience in building and maintaining test automation frameworks using Java and supporting tools like TestNG, JUnit, Maven, and Gradle.

Mobile Testing:
In-depth understanding of mobile testing strategies and best practices for both Android and iOS devices.

Version Control Systems:
Experience with Git and using Git for version control and collaboration in a team environment.

CI/CD Tools:
Familiarity with CI/CD tools such as Jenkins, GitLab CI, CircleCI, or similar to integrate automated tests into the build pipeline.

Bug Tracking:
Proficiency in bug tracking tools (e.g., Jira) to log, track, and manage defects through resolution.

Analytical Skills:
Strong analytical and debugging skills, with the ability to troubleshoot and resolve complex automation issues.

Excellent Communication:
Strong written and verbal communication skills, with the ability to effectively report test results and collaborate with cross-functional teams."
"Product Manager

Develop, own, and evolve the product roadmap for cloud security solutions, ensuring alignment with organizational goals and cloud security objectives.
     Drive the execution of security product features, enhancements, and initiatives that protect our cloud environments and applications from threats.
     Collaborate closely with cloud security engineers, development teams, DevOps, and stakeholders to define product requirements and ensure seamless integration of security features.
     Engage with various teams to understand their needs and translate them into actionable security solutions, features, and backlog items.
     Prioritize the product backlog based on technical feasibility, business impact, customer value, and risk mitigation.
     Continuously refine the backlog to ensure the team is focused on the highest value work, balancing innovation with security requirements.
Cloud Security Expertise:
     Leverage deep knowledge of cloud platforms (AWS, Azure, GCP) and experience in designing, building, and operating cloud-native security tools and services.
     Ensure the cloud environments comply with internal control requirements, regulatory obligations, and external security standards (e.g., NIST, CIS, ISO), with comprehensive reporting and dashboards.
     Lead efforts to design, implement, and maintain robust security controls and solutions that proactively protect the cloud infrastructure from emerging threats.
     Act as the primary point of contact for product-related decisions and issues, ensuring swift resolution and continuous delivery of high-quality security features.
Qualifications:
     Bachelor’s degree in computer science, Information Security, or related field (or equivalent experience).
     Relevant cloud security certifications (e.g., AWS Certified Security, Microsoft Certified: Azure Security Engineer, Certified Information Systems Security Professional (CISSP)).
     5+ years of experience in product management, with a focus on cloud security or cloud-native product development.
     Proven experience in developing cloud security solutions on AWS, Azure, and GCP, including hands-on familiarity with cloud security services (e.g., AWS Shield, Azure Security Center).
     Experience working in agile development teams and environments, leveraging DevSecOps principles and integrating security into CI/CD pipelines.
     Strong understanding of security frameworks and standards (NIST, CIS, ISO) and their application in cloud environments. • Ability to manage complex cross-functional teams, projects, and priorities with excellent organizational and time-management skills.
     Proficiency in product management tools (e.g., JIRA, Confluence) and experience in backlog management, refinement, and prioritization.
     Excellent communication, stakeholder management, and leadership skills, with the ability to influence and guide technical and non-technical teams.
"
"Sr. Cloud Security Engineer

Cloud Security and tool skills:
• Experience in cloud onboarding, tool management, and issue analysis using Wiz Security platforms (or equivalent CNAPP solutions) is a must.
• Deep understanding of cloud computing, including virtualization, containerization, and microservices; Good understanding of IAM concepts.
• Knowledge of security concepts (with zero-trust design principles as a plus);
• Hands-on experience in container security.
• Hands-on experience designing and implementing for information security on public cloud;
• Experience in authoring security policies in the cloud and remediation.
• Strong knowledge of security across layers (OS, network, application, data, container, CI/CD, etc.);
• Hands-on experience with CSP security tools (like AWS Security Hub, Azure Security Center, or GCP Security Command Center) would be a plus;
• Ability to work independently and collaboratively

Automation Skills:

• Proficiency in Python for automation or in observability engineering,
• Experience in Gitops implementation and gitlab would be a plus.
• Hands-on experience in Terraform automation with ability to create custom provider is a plus.
• Experience in managing the secrets in Vault and Hashicorp Vault is a plus.
"
"Role Proficiency:

Contributes to driving the Product Vision that addresses cross-domain market needs with strong business viability; in line with the Product/Platform Portfolio with guidance from Managers. Assists Managers in delivering the desired product and platform outcomes from original inspiration through implementation and support.rnContributes to the discovery and development of the product/platform per the phases and stages of the product/platform.  as guided by the Product Managers.



Outcomes:

      A strong evangelist of Product & Platforms Division’s Vision and Philosophy.
      Performs primary and secondary research on Market Driven Products & Platforms; executes the product/platform strategy
      Ensures clear and unambiguous translation of Product Value Roadmap through deliverables to enable the product team to complete work
      Ensures adherence to the Product Value Roadmap set by Product Managers and Product Leadership
      Drafts user stories and ensures they are clearly understood by cross functional product teams
      Ensures Product requirements are gathered and prioritized
      Drives the execution of product development in collaboration with Product Team members comprised of Product Design Technology & Data Science
      Adherence to the modern product management practises and standards; providing periodic status updates
      Supports Product Managers and Senior Product Managers in product demos
  Supports in achieving the Product OKR's


Measures of Outcomes:

      Adherence to agreed product roll-out timelines
      Achievement of Product Metrics
      Achievement of Product OKR's
      Product Team Performance
     Number of domain and product certifications


Outputs Expected:

Plan to develop great products and platforms:

Strong research enabling sound Product Discovery
Co-facilitates Discovery Sprints under guidance from Product and Senior Product Managers
Write and articulate great user stories by capturing and translating solutions for identified user problems into product features that deliver value and impress users
Adherence to Product OKR's


Design to deliver vast end -user experience:

Facilitate sound user research across target market segments
Influence in implementing design user-centric experiences throughout the user's journey
Assist in defining user experience (e.g.  wire framing journey maps); partnering with product designers


Research the market to ensure an unfair advantage:

Sound understanding of market trends partner ecosystems and competitive strategies
Understanding users the marketplace the competition and future trends for the domain or type of system being developed through customer and market research competitive analysis and rapidly acquiring domain expertise
Define and articulate unique product and business differentiation


Manage business to go to and win in the markets:

Assist in implementing the GTM strategies and achieve the metrics for product success
Protect assets and mitigate risks employing IP knowledge


Manage great people:

Inspire Product teams communicate with diverse groups and influence change throughout the organization


Use great technology:

Good understanding and usage of tech stacks
Good understanding of exponential technologies including AI/ML IoT Blockchain
Agile proficiency for Rapid Product Discovery & Development
Employ eminent technology in product management by diving deep into technology trends and architectures


Employ great methods in product management :

Understand different product management and development approaches
Understand product usage models


Skill Examples:

       Skill in understanding and articulating the core positioning messaging and value propositions for the product
      Ability to support product demos to customers/end users
      Ability to define the user experience to be incorporated into the User Design
      Aptitude in creating user stories
      Ability to creates market customer and competition research based on understanding of market trends partner ecosystems and competitive strategies
       Capable of motivating the team to build and deliver impressive products


Knowledge Examples:

Knowledge Examples

      Domain / Industry Knowledge:  Working knowledge of standard business processes within the relevant industry vertical and customer business domain
      Technology Trends Knowledge: Demonstrates broad knowledge of technology trends related to multiple inter-related technologies
      Proficient in user story writing
      Knowledge of market trends partner ecosystems and competitive strategies
      Proficient in product documentation
      Expertise in people management
     Knowledge of technology trends and architectures"
"Role Proficiency:

Understands customer requirements streamline DevOps practices and translate them into reference architecture for DevOps (CI/CD) and automation components by managing multiple scrum teams.



Outcomes:

     Interprets the DevOps Tool/feature/component design to develop/support the same in accordance with specifications
     Adapts existing DevOps solutions and creates own DevOps solutions for new contexts
     Codes debugs tests documents and communicates DevOps development stages/status of DevOps support issues
     Selects appropriate technical options for development such as reusing improving or reconfiguration of existing components
     Optimises efficiency cost and quality of DevOps process tools and technology development
     Validates results with user representatives; integrates and commissions the overall solution
     Assesses current state of DevOps maturity and advises clients on improving maturity and DevOps roadmap
     Assess readiness of clients to adopt DevOps practices advising clients on appropriate change approaches and technology choices
     Looks for patterns of struggle and experiments with new designs and new ways of applying existing designs to solve problems
  Guides DevOps leads and engineers and observe how the processes are working in practice
  Works towards reducing variations in practice by challenging status quo and tweaking processes and tools
  Resolve architecture issues and deliver / own architecture of application solutions spanning across multiple technologies for projects of the following nature - high revenue projects / complex projects / large strategic maintenance projects
  Manage multiple stakeholders and handle their expectations
  Contribute to business development and organizational activities
  Support Architect/Sr. Architect in drafting recommendations based on findings of Proof of Concept


Measures of Outcomes:

     Quality of Deliverables
     Defect injection at various stages of lifecycle
     SLA/KPI for onboarding projects or applications
     Percentage achievement of specification/completeness/on-time delivery
     # of reusable components / processes developed
     Delivery efficiency (Adherence to UST defined principles)
     Contribution to technology capability development (e.g. Training Webinars Blogs)
     Customer feedback on overall technical quality (zero technology related escalations)
     # of white papers/document assets contributed to
  Feedback from Project Team/Program Management on project support
  Error rate/completion rate at various stages of SDLC/PDLC
  Uptime/System reliability metrics (if the design is not meeting with agreed SLA’s)
  # of domain certification/ product certification obtained
 Adoption rates of CI/CD tools DevOps practices (efficiency improvement rate)


Outputs Expected:

Automated components :

Deliver components that automat parts to install components/configure of software/tools in on  premises and on cloud
Deliver components that automate parts of the build/deploy for applications


Configured components:

Configure tools and automation framework into the overall DevOps design


Scripts:

Supervise/Develop automation scripts (like Powershell/Shell/Python scripts) that automate installation/configuration/build/deployment tasks


Training/SOPs :

Create Training plans/SOPs to help DevOps Engineers with DevOps activities and to onboard users
Create Training plans/SOPs to help technical deal with already existing CI/CD Pipelines and components; onboarding users


Measure Process Efficiency/Effectiveness:

Deployment frequency innovation and technology changes


Operations:

Change lead time/volume
Failed deployments
Defect volume and escape rate
Meantime to detection and recovery


Solution Definition & Design:

Define Architecture for the small/mid-sized type of project
Design the technical framework and implement the same
Identify and conduct design of complex sub-components /module in collaboration with project team architects and client SME
Present the detailed design documents to relevant stakeholders and seek feedback
Undertake project specific Proof of Concepts activities to validate technical feasibility with guidance from the Application Architect
Implement best optimized solution and resolve performance issues
Support the team in the design of functional modules and review the deliverables
Conduct code reviews


Project Estimation:

Provide support for project estimations on business proposals and support sprint level / component level estimates
Articulate estimation methodology module level estimations for more standard projects with focus on estimation effort alone


Measure Process Efficiently/Effectiveness:

Measure and pay attention to efficiency/effectiveness of current process and make changes to make them more efficient/effective
Facilitate development processes and operations
Architecting overall comprehensive DevOps practice effectiveness


Skill Exampl"
"ures of Outcomes:

NA



Outputs Expected:

Contractor:

NA


Skill Examples:

NA



Knowledge Examples:

Knowledge ExamplesNA"
"Role Proficiency:

Act creatively to develop applications and select appropriate technical options optimizing application development maintenance and performance by employing design patterns and reusing proven solutions account for others' developmental activities



Outcomes:

     Interpret the application/feature/component design to develop the same in accordance with specifications.
     Code debug test document and communicate product/component/feature development stages.
     Validate results with user representatives; integrates and commissions the overall solution
     Select appropriate technical options for development such as reusing improving or reconfiguration of existing components or creating own solutions
     Optimises efficiency cost and quality.
     Influence and improve customer satisfaction
 Set FAST goals for self/team; provide feedback to FAST goals of team members


Measures of Outcomes:

     Adherence to engineering process and standards (coding standards)
     Adherence to project schedule / timelines
     Number of technical issues uncovered during the execution of the project
     Number of defects in the code
     Number of defects post delivery
     Number of non compliance issues
  On time completion of mandatory compliance trainings


Outputs Expected:

Code:

Code as per design
Follow coding standards templates and checklists
Review code – for team and peers


Documentation:

Create/review templates checklists guidelines standards for design/process/development
Create/review deliverable documents. Design documentation r and requirements test cases/results


Configure:

Define and govern configuration management plan
Ensure compliance from the team


Test:

Review and create unit test cases scenarios and execution
Review test plan created by testing team
Provide clarifications to the testing team


Domain relevance:

Advise Software Developers on design and development of features and components with a deep understanding of the business problem being addressed for the client.
Learn more about the customer domain identifying opportunities to provide valuable addition to customers
Complete relevant domain certifications


Manage Project:

Manage delivery of modules and/or manage user stories


Manage Defects:

Perform defect RCA and mitigation
Identify defect trends and take proactive measures to improve quality


Estimate:

Create and provide input for effort estimation for projects


Manage knowledge:

Consume and contribute to project related documents share point libraries and client universities
Review the reusable documents created by the team


Release:

Execute and monitor release process


Design:

Contribute to creation of design (HLD LLD SAD)/architecture for Applications/Features/Business Components/Data Models


Interface with Customer:

Clarify requirements and provide guidance to development team
Present design options to customers
Conduct product demos


Manage Team:

Set FAST goals and provide feedback
Understand aspirations of team members and provide guidance opportunities etc
Ensure team is engaged in project


Certifications:

Take relevant domain/technology certification


Skill Examples:

     Explain and communicate the design / development to the customer
     Perform and evaluate test results against product specifications
     Break down complex problems into logical components
     Develop user interfaces business software components
     Use data models
     Estimate time and effort required for developing / debugging features / components
     Perform and evaluate test in the customer or target environment
     Make quick decisions on technical/project related challenges
     Manage a Team mentor and handle people related issues in team
  Maintain high motivation levels and positive dynamics in the team.
  Interface with other teams designers and other parallel practices
  Set goals for self and team. Provide feedback to team members
  Create and articulate impactful technical presentations
  Follow high level of business etiquette in emails and other business communication
  Drive conference calls with customers addressing customer questions
   Proactively ask for and offer help
  Ability to work under pressure determine dependencies risks facilitate planning; handling multiple tasks.
  Build confidence with customers by meeting the deliverables on time with quality.
  Estimate time and effort resources required for developing / debugging features / components
  Make on appropriate utilization of Software / Hardware’s.
  Strong analytical and problem-solving abilities


Knowledge Examples:"
"Sr Cloud Network Engineer
Key Responsibilities
• Oversee the network onboarding process for new users and systems into Cloud environment
• Provision and configure network resources in Cloud, ensuring compliance with security policies and government regulations.
• Implement secure network architectures, including Virtual Private Clouds (VPCs), subnets, routing tables, and network access control lists across AWS, Azure, and other cloud platforms.
• Configure and manage cloud networking services for secure connectivity between on-premises and cloud environments, such as AWS Direct Connect, Azure VPN Gateway, and transit gateways.
• Implement network security controls, such as security groups, network firewalls, and web application firewalls to protect against unauthorized access and cyber threats.
• Monitor network traffic and security logs using cloud services for flow logs, activity trails, and threat detection to identify and respond to potential security incidents.
• Collaborate with cross-functional teams to ensure secure integration of applications and services into the cloud network infrastructure.
• Conduct regular network assessments and audits to ensure compliance with internal and external requirements.
• Develop and maintain comprehensive network security policies, procedures, and documentation in compliance with security standards.
• Provide technical support and troubleshooting for Cloud network-related issues
• Stay up-to-date with the latest cloud networking services, security features, and best practices across multiple platforms

Automation Skills:
• Proficiency in Python or Golang for automation
• Strong expertise in Terraform automation with the ability to create custom templates and modules is a plus.
• Hands-on experience in CI/CD implementation and gitlab
• Experience with containerization would be a plus

Qualifications
• Assesses the candidate's experience with the network onboarding process and provisioning in cloud environments such as AWS and Azure.
• Evaluates the candidate's experience with automation tools for network configuration and management.
• Measures the candidate's hands-on experience with cloud networking services across AWS, Azure, and other major cloud providers.
• Assesses the candidate's in-depth knowledge of network security principles, protocols, and best practices for secure network design and implementation.
• Evaluates the candidate's knowledge of government network security standards and compliance requirements.
• Measures the candidate's relevant certifications which indicate their proficiency and expertise in cloud

"
"Stakeholder Management and Overall Project Management.

Job Description - 7-9 years of experience as a Technical Product Owner in a fast-paced software development environment.
Experience in managing the development of complex software products, ideally in [, eCommerce, Lifesciences domain].
Proven experience in working with engineering teams and a deep understanding of the software development lifecycle.
Strong technical background, with a good understanding of software architecture, databases, APIs, and cloud technologies.
Familiarity with Agile methodologies (Scrum, Kanban) and experience working in Agile product development teams.
Proficiency in writing user stories, defining acceptance criteria, and managing product backlogs.
Hands-on experience with product management tools (e.g., Jira, Confluence).
Exceptional communication skills, both written and verbal, with the ability to explain complex technical concepts in a simple and understandable way.
Proven ability to collaborate effectively with cross-functional teams.
Strong negotiation and stakeholder management skills, able to balance competing priorities and drive consensus.
Strong analytical and problem-solving skills with the ability to make data-driven decisions. Ability to balance short-term deliverables with long-term strategic goals.
Understanding of product metrics and how to use them to optimize product performance and customer satisfaction.
A background in development (e.g., experience as a software developer, systems engineer, etc.)

"
"Provide technical and business project leadership across functional boundaries for different phases of product development and support programs. This leadership will include projects pertaining to New Product Introduction, Line Qualification for new product launch and batch qualification for new product launch.

Manage the production scale up of new Skin Care and other Personal Care consumer products

Lead the Tech Transfer process of Skin Care and other Personal Care consumer products

Guide cross functional teams; internal/external engineering and resolve inter-functional issues

Provide documentation of the project and program activities and deliverables

Ensure attainment of project and program schedules, budgets, procedures, and revenues by developing and auditing the budget and approval process

Direct processes to promote and implement new product capability:
•	Bid/quote process
•	Equipment purchase
•	Project planning
•	Pilot batches
•	Trial batches
•	Line trials
•	Specification and artwork development
•	Stability validation protocols
•	Production start-up

Carry-out communication with Senior management on project and program status
•	Create, maintain, and present written reporting on status, tests, costs, issues, and performance against established targets on a continuing basis

Research business and technical issues to establish what is being done and where improvement is feasible

Arrange proper system integration by overseeing the development of the systems integration and systems assurance test plans and procedures
•	analyze the results

Establish and communicate project and program schedules, objectives, priorities, and targets

Oversee status of programs and projects
•	provide technical interpretation and knowledge
•	ensure adherence to policies, procedures, government regulations, and customer specifications

This position is based in the Lancaster, PA area
"
"In this role, you will have the opportunity to support the Corrective and Preventive
Action (CAPA) process from issue identification to closure and ensure adequate
execution of the process and the quality of CAPA record content. You will be part of our
Quality & Regulatory organization.

Key Responsibilities:
•	Project Management: Project manage all CAPA activities from initiation to closure, ensuring timely and effective execution.
•	Issue Identification and Assessment: Identify and assess issues, ensuring accurate and thorough documentation.
•	CAPA Data Review: Review CAPA data sources and assist with root cause analysis and quality problem-solving.
•	Root Cause Analysis: Participate in and/or lead root cause analysis activities to identify the underlying causes of issues.
•	CAPA Record Quality: Ensure the quality and completeness of CAPA records.
•	Audit and Review Board Representation: Support and prepare spokespersons in representing CAPAs during audits and CAPA Review Board meetings.
•	Timely CAPA Engineering Tasks: Perform timely, detailed CAPA engineering tasks like assessing issue descriptions and reviewing CAPA data sources.
•	Process Improvement: Contribute to the continuous improvement of CAPA processes and procedures.
•	Ensuring Products Meet Standards: Ensuring that our products meet the highest standards.
"

SO# is created for internal purpose
Allocation for David Fraley.
The ideal candidate will be skilled in:  Process discovery and optimization  Data analysis, visualization, and dashboarding  Identifying opportunities to improve our processes and systems, especially in finance domain. A strong business architecture mindset is a plus, with experience or familiarity in Various ERP system like Oracle and SAP. Expertise in various tools like  Process Mining tools  BizzDesign / Business Architecture tools  Power BI  Power Apps  Qualtrics Analysis  User journey mapping  Problem statement generation  Developing actionable plans
 New requirement
NA
Please look for lead developers with experience in Springboot, Java microservices, GraphQL with hands on experience on Python as well. Resource has to be in Atlanta.
NA
Contractor SO#
NA
Please provide Senior profiles with IOS/Android Mobile Automation Testing with Appium and Java expertise.
SO# created to onboard Mosfaqus Salehin
Mandatory Skills(Only 3-4)Wiz Security Platform or equivalent CNAPP solutionCross-platform securityContainer SecurityTerraformWork independentlyOptional SkillsKnowledge of Zero-Trust Design principlesManaging secrets (Vault or Hashicorp)
NA
SO Created to allocate Mona Sitlani UID 150942
SO# is created to convert Sundar Munusuri- 15128
SO# is created to convert 270106Raguram Veeraparakaramapandian
NA
 creating new SO for J&amp;J with product owner JD.
Based in Lancaster PA
Preference is for resource to be near Pittsburgh, PA but remote is also an option



Software Engineering,Software Developer,Full Stack
Oracle,Sap,Process Mining,Power Bi
Databricks,Spark Sql,Tableau
SQL,PYTHON,DATA BRICKS,TELECOM EXPERIENCE
Aws Services,Java Microservices,Graphql
kubernetes,kubernetes fundamental,golang,Terraform
Kubernetes,Aws Cloud,Golang
Project Management,agile,jira
Android,Ios,Appium Testing
cloud security,cloud platform,AWS CLOUD
Security,cross platform security,container security
proudct,product manager,agile
java,spring,microservices
gitlab,devops,cloud security
data,analyst,Data Analysis,AWS
Kubernetes,Aws Cloud,Golang
Project Management,Stakeholder Management,Scrum Master
Timeline,Project Management,Excellent Communication
Capa,Quality Assurance,Quality Management


David Fraley 284050
Naveen shortlisted, waiting for final confirmation from client.
Technical Data Analyst  Vinesh/ RH ADOBE 3/20
Sasmitha Das CI pending. Amarnath and Anish L2 to be scheduled.  (Nishant Panel) Vijay/ Beeline COMCAST Data Analyst
Client Ref. Poorna Chandra L1  scheduled with Jyothsih Glider 4/8. Java/Python Developer Jyothish/ TMO - Deb/ Beeline
Keerthi  submitted to Client 4/7.  2 L2 Interview to be set up Abin.24/3 : wating 4 L2 (sukesh to help on panel ) 21/3 : Durga Maruthi L2 to be scheduled,.Srujan Sama,Sravan Kumar Ayyagari ,Yedunandan Vitakula (L2 to be scheduled)  Replacement of Teja Sai Ganta (288113) TMO Deb/ Beeline 3/14 - Kiran Krishnan Coordinate for Panel. Frisco/ Remote - Punna Rao / Donald/ Bujail/ Sujith - Panel
Beeline/ Vijay. 2/4 : Sathya Nagaraj submitted to Client 4/7.  Roman L3 to be scheduled with Sujith (Panel)   Vignana (L2 with Raja) - Ajay Pasunoor L1 Completed (need to set up with Panel) 25/3 : Sathya Nagaraj (L3 to be set up with sujith) , Pradeep Dasari L2 with Raja as on 25th  Glider L1 in progress 3/19. TMO Sr. Cloud Network Engineer Vijay/ Beeline 3/18. CWR - Raja and Sagar - Panel
Anthony, Kiran and Dennis CI FB awaited.   Russ L2 with Sujith 4/7 .  Pradeep Murthy Declined.   CI on 3/14 awaiting Client confirmation 3/18.  Anthony Interview with Sujith 3/18.  Sheetal - backup.  Pradeep L1 with Sujith 3/6.   Jessica and Zaibi CI Reject on 3/5. Jessica CI 3/3 Awaiting CI FB and Zaibi  Khan CI 3/4.  - to be submitted to Client 2/28.    Shibu H1B Offshore, HM review. 2/26.    Arthur L1 Reject.  (TP) L1 on 2/24.  Sr Project Manager CMCMST Vijay/ Beeline 2/20, Shared TP list PMs , awaiting HM response 2/20
Vijay/ Beeline 4/2 . Shanmugapriya Kaveripakkam Arunagiri OIP Tentative DOJ 4/8
Vijay/ Beeline 3/14. BGV in progress. Updated SO with change in Bill Rate  $135.08 Mosfaqus Salehin Offer in progress 3/13 Tentative DOJ 3/24. (135.08)   Saju will revert on Actual bill rate to proceed as per Rate List.   New SO received 3/5. Vijay/ Beeline. Mosfaqus, Susai Chandran and Ashish CI scheduled 3/6.     Only 1 role as per HM. Client Cancelled other 2 roles.  Mosfaqus CI 1st pref. for CI .  pending. Sriram Ac Ref. L2 2/27.  (Vijay to schedule L1) Product Manager Sukesh/ Saju Vijay/ Beeline -  2/19
Ramchand Tripuraneni  Joined 4/8. Awating Revised bill rate 21/3  : Ramkrishna  selected  Ramchand  CI on 3/20.     3 L2 in progress, Saju to check with Sujith (Panel) if Interview 2/22.  Awaiting Panel Slots 2/21.  Glider L1 in progress Sr. Cloud Security Engineer / Sukesh/ Saju 2/18 (Vijay/ Beeline 2/18) Frisco (only Harivijayan Panel, Panel slot challenge)  Sujith Surendran, Abin Mathew L2 Panel.
Checking with Fixed Term Offer with Candidate. Conversion Dennis Kim[285154]  CWR to FTE RH/ Arun
31/3  : Still awating doc from ac.   Mona Sitlani  .25/3 : Documents pending with Account (Saju)24/3 : Awating account documents , Updated SO Arun/ RH 3/17. CWR to FTE SO/RR type Replacement 3/17. Mona Sitlani 150942 - Candidate docs received 3/10. Awaiting A/c docs. Tentative DOJ 4th Apr'25.LCA Poting completed 3/13.  Candidate docs (awaiting New Passport ) . Conversion   LCA initiated. Arun/ RH 2/11, Offer accepted BGV inititated, Pending H1B docs from Candidate 2/18
Offer Accepted Awaiting Candidate/ Account Specific Docs. Sundar Munusuri- 151228 Conversion Arun/ RH 4/2
Raguram Veeraparakaramapandian’s. Case filed, Tentative DOJ 4/28.Received Receipt # 4/7.  LCA initiated 2/27. Awaiting Filing Status 3/19.  LCA posting end date 3/19. Accepted New Offer 2/24. LCA initiated 3/4. 10 days posting.  as on 3/4.  Awaiting Comp Aproval 2/10.      Awaiting Account Specific Docs. from Neha/ Venkat Account.     Offer Accpeted, Awaiting H1B Docs . Conversion on Hold. Career Archt. Ace title not available. Pending with Comp. Tentative DOJ 1/20/25.  Team.  Offer in Progress. Data Engineer.   Raguram Veeraparakaramapandian’s  RTMO Conversion. Tentative DOC Jan'25 Arun/ RH
Duplicate SO FTE. Glider L1 in progress 3/19. TMO Sr. Cloud Network Engineer Arun/ RH 3/18. FTE
Sowmyasree
1 L1 scheduled
AISHA QADEER SIDDIQU


Allocation WIP/Confirmed
Client feedback pending
Client interview Scheduled
Client interview Scheduled
Client interview Scheduled
Client interview Scheduled
Client interview Scheduled
Client interview Scheduled
Fulfilled and Closed
Fulfilled and Closed
Fulfilled and Closed
Conversion
Conversion
Conversion
Conversion
Duplicate SO
Allocation WIP/Confirmed
L1 Interview - Scheduled
L1 Interview - Scheduled


Allocation WIP
Less Profile flow, Niche Skill with location and Budget constraints (MEDIUM)
Less Profile flow
SO is opened between 16-30 days, Additional Profiles needed, L1 rejection rates remain at 30–50%
Less Profile flow, SO is opened between 16-30 days
SO is opened between 16-30 days, L1 rejection rates remain at 30–50%
SO is opened between 16-30 days, L1 rejection rates remain at 30–50%
SO Opened for more than 30 days, Less Profile flow
Fulfilled
Fulfilled
Fulfilled
Conversion
Conversion
Conversion
Conversion
Less Profile flow, SO is opened between 16-30 days
Allocation WIP
SO Opened for more than 30 days, Niche Skill with location and Budget constraints (MEDIUM), Additional Profiles needed, L1 rejection rates remain at 30–50%
Rejection rates remain between 30–50% due to niche skills, location, and budget constraints


"""

In [ ]:
# Loading GLiNER for testing descriptive data
import time

start = time.time()
from gliner import GLiNER
# model = GLiNER.from_pretrained(
#     "juampahc/gliner_multi-v2.1-onnx",
#     load_onnx_model=True,
#     onnx_model_file="model.onnx"
# )
model = GLiNER.from_pretrained("urchade/gliner_smallv2.1")
end = time.time()
print(f'📈📈 Model loading time: {end - start:.2f} seconds')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


📈📈 Model loading time: 34.61 seconds


In [ ]:
# Chunking for descriptive data using GLiNER
def chunk_text(text, max_len=2048, overlap=48):
    chunks = []
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + max_len, text_len)
        chunks.append((text[start:end], start))
        if end == text_len:
            break
        start = end - overlap
    return chunks

def merge_entities(chunked_entities):
    merged = []
    seen = set()
    for ents, offset in chunked_entities:
        for e in ents:
            start = e['start'] + offset
            end = e['end'] + offset
            key = (e['text'], start, end, e['label'])
            if key not in seen:
                seen.add(key)
                merged.append({
                    'text': e['text'],
                    'start': start,
                    'end': end,
                    'label': e['label']
                })
    return merged

labels = ["Name of Person", "Organization", "Location", "TravelRoute", "ID"]

chunks = chunk_text(string, max_len=1024, overlap=50)
print(f"✂️ Total chunks created: {len(chunks)}")

chunked_entities = []
start_time = time.time()

for i, (chunk, offset) in enumerate(chunks):
    ents = model.predict_entities(chunk, labels, flat_ner=True, threshold=0.5)
    chunked_entities.append((ents, offset))
    print(f"✅ Processed chunk {i+1}/{len(chunks)} with {len(ents)} entities")

merged_entities = merge_entities(chunked_entities)

end_time = time.time()
print("🔍 Merged Detected Entities:")
for ent in merged_entities:
    print(f"🔹 '{ent['text']}' → 🏷️ {ent['label']} 📍({ent['start']}-{ent['end']})")

print(f"\n⏱️ Total Inference Time: {end_time - start_time:.4f} seconds")

✂️ Total chunks created: 54
✅ Processed chunk 1/54 with 0 entities
✅ Processed chunk 2/54 with 4 entities
✅ Processed chunk 3/54 with 9 entities
✅ Processed chunk 4/54 with 5 entities
✅ Processed chunk 5/54 with 6 entities
✅ Processed chunk 6/54 with 1 entities
✅ Processed chunk 7/54 with 1 entities
✅ Processed chunk 8/54 with 2 entities
✅ Processed chunk 9/54 with 2 entities
✅ Processed chunk 10/54 with 1 entities
✅ Processed chunk 11/54 with 9 entities
✅ Processed chunk 12/54 with 0 entities
✅ Processed chunk 13/54 with 4 entities
✅ Processed chunk 14/54 with 1 entities
✅ Processed chunk 15/54 with 0 entities
✅ Processed chunk 16/54 with 2 entities
✅ Processed chunk 17/54 with 3 entities
✅ Processed chunk 18/54 with 4 entities
✅ Processed chunk 19/54 with 0 entities
✅ Processed chunk 20/54 with 2 entities
✅ Processed chunk 21/54 with 5 entities
✅ Processed chunk 22/54 with 7 entities
✅ Processed chunk 23/54 with 4 entities
✅ Processed chunk 24/54 with 2 entities
✅ Processed chunk 25/

In [ ]:
# Testing the pipeling of GLiNER with Spacy

import spacy

start = time.time()

custom_config = {
    "gliner_model": "urchade/gliner_small-v2.1",
    "chunk_size": 2048,
    "labels": ["Name of Person", "Organization", "Location", "TravelRoute", "ID"],
    "style": "ent"
}

nlp = spacy.blank("en")
nlp.add_pipe("gliner_spacy")
end = time.time()

print(f'Time taken to load and set pipeline: {end - start:.2f}')

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Time taken to load and set pipeline: 9.29


In [ ]:
statement = """David Fraley 284050
Naveen shortlisted, waiting for final confirmation from client.
Technical Data Analyst  Vinesh/ RH ADOBE 3/20
Sasmitha Das CI pending. Amarnath and Anish L2 to be scheduled.  (Nishant Panel) Vijay/ Beeline COMCAST Data Analyst
Client Ref. Poorna Chandra L1  scheduled with Jyothsih Glider 4/8. Java/Python Developer Jyothish/ TMO - Deb/ Beeline
Keerthi  submitted to Client 4/7.  2 L2 Interview to be set up Abin.24/3 : wating 4 L2 (sukesh to help on panel ) 21/3 : Durga Maruthi L2 to be scheduled,.Srujan Sama,Sravan Kumar Ayyagari ,Yedunandan Vitakula (L2 to be scheduled)  Replacement of Teja Sai Ganta (288113) TMO Deb/ Beeline 3/14 - Kiran Krishnan Coordinate for Panel. Frisco/ Remote - Punna Rao / Donald/ Bujail/ Sujith - Panel
Beeline/ Vijay. 2/4 : Sathya Nagaraj submitted to Client 4/7.  Roman L3 to be scheduled with Sujith (Panel)   Vignana (L2 with Raja) - Ajay Pasunoor L1 Completed (need to set up with Panel) 25/3 : Sathya Nagaraj (L3 to be set up with sujith) , Pradeep Dasari L2 with Raja as on 25th  Glider L1 in progress 3/19. TMO Sr. Cloud Network Engineer Vijay/ Beeline 3/18. CWR - Raja and Sagar - Panel
Anthony, Kiran and Dennis CI FB awaited.   Russ L2 with Sujith 4/7 .  Pradeep Murthy Declined.   CI on 3/14 awaiting Client confirmation 3/18.  Anthony Interview with Sujith 3/18.  Sheetal - backup.  Pradeep L1 with Sujith 3/6.   Jessica and Zaibi CI Reject on 3/5. Jessica CI 3/3 Awaiting CI FB and Zaibi  Khan CI 3/4.  - to be submitted to Client 2/28.    Shibu H1B Offshore, HM review. 2/26.    Arthur L1 Reject.  (TP) L1 on 2/24.  Sr Project Manager CMCMST Vijay/ Beeline 2/20, Shared TP list PMs , awaiting HM response 2/20
Vijay/ Beeline 4/2 . Shanmugapriya Kaveripakkam Arunagiri OIP Tentative DOJ 4/8
Vijay/ Beeline 3/14. BGV in progress. Updated SO with change in Bill Rate  $135.08 Mosfaqus Salehin Offer in progress 3/13 Tentative DOJ 3/24. (135.08)   Saju will revert on Actual bill rate to proceed as per Rate List.   New SO received 3/5. Vijay/ Beeline. Mosfaqus, Susai Chandran and Ashish CI scheduled 3/6.     Only 1 role as per HM. Client Cancelled other 2 roles.  Mosfaqus CI 1st pref. for CI .  pending. Sriram Ac Ref. L2 2/27.  (Vijay to schedule L1) Product Manager Sukesh/ Saju Vijay/ Beeline -  2/19
Ramchand Tripuraneni  Joined 4/8. Awating Revised bill rate 21/3  : Ramkrishna  selected  Ramchand  CI on 3/20.     3 L2 in progress, Saju to check with Sujith (Panel) if Interview 2/22.  Awaiting Panel Slots 2/21.  Glider L1 in progress Sr. Cloud Security Engineer / Sukesh/ Saju 2/18 (Vijay/ Beeline 2/18) Frisco (only Harivijayan Panel, Panel slot challenge)  Sujith Surendran, Abin Mathew L2 Panel.
Checking with Fixed Term Offer with Candidate. Conversion Dennis Kim[285154]  CWR to FTE RH/ Arun
31/3  : Still awating doc from ac.   Mona Sitlani  .25/3 : Documents pending with Account (Saju)24/3 : Awating account documents , Updated SO Arun/ RH 3/17. CWR to FTE SO/RR type Replacement 3/17. Mona Sitlani 150942 - Candidate docs received 3/10. Awaiting A/c docs. Tentative DOJ 4th Apr'25.LCA Poting completed 3/13.  Candidate docs (awaiting New Passport ) . Conversion   LCA initiated. Arun/ RH 2/11, Offer accepted BGV inititated, Pending H1B docs from Candidate 2/18
Offer Accepted Awaiting Candidate/ Account Specific Docs. Sundar Munusuri- 151228 Conversion Arun/ RH 4/2
Raguram Veeraparakaramapandian’s. Case filed, Tentative DOJ 4/28.Received Receipt # 4/7.  LCA initiated 2/27. Awaiting Filing Status 3/19.  LCA posting end date 3/19. Accepted New Offer 2/24. LCA initiated 3/4. 10 days posting.  as on 3/4.  Awaiting Comp Aproval 2/10.      Awaiting Account Specific Docs. from Neha/ Venkat Account.     Offer Accpeted, Awaiting H1B Docs . Conversion on Hold. Career Archt. Ace title not available. Pending with Comp. Tentative DOJ 1/20/25.  Team.  Offer in Progress. Data Engineer.   Raguram Veeraparakaramapandian’s  RTMO Conversion. Tentative DOC Jan'25 Arun/ RH
Duplicate SO FTE. Glider L1 in progress 3/19. TMO Sr. Cloud Network Engineer Arun/ RH 3/18. FTE
Sowmyasree
1 L1 scheduled
AISHA QADEER SIDDIQU"""
sent = model.predict_entities(statement, ['Person', 'Organization'])
for entity in sent:
  print(f'{entity["text"]} - {entity["label"]}')

/usr/local/lib/python3.11/dist-packages/gliner/data_processing/processor.py:296: UserWarning: Sentence of length 961 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


David Fraley - Person
Naveen - Person
Vinesh - Person
RH ADOBE - Organization
Sasmitha Das - Person
Amarnath - Person
Beeline - Organization
Poorna Chandra - Person
Jyothish - Person
TMO - Organization
Beeline - Organization
Keerthi - Person
sukesh - Person
Durga Maruthi - Person
Srujan Sama - Person
Sravan Kumar Ayyagari - Person
Yedunandan Vitakula - Person
Teja Sai Ganta - Person
TMO - Organization
Beeline - Organization
Kiran Krishnan - Person
Punna Rao - Person
Sujith - Person
Beeline - Organization
Sathya Nagaraj - Person
Sujith - Person
Ajay Pasunoor - Person
Sathya Nagaraj - Person
Pradeep Dasari - Person
TMO - Organization
Beeline - Organization
Raja - Person
Sagar - Person
Anthony - Person
Kiran - Person
CI FB - Organization
Sujith - Person
Pradeep Murthy - Person
Anthony - Person
Sujith - Person
Sheetal - Person
Sujith - Person
Jessica - Person
Jessica - Person
CI FB - Organization
Zaibi  Khan - Person
CMCMST - Organization
Beeline - Organization


In [ ]:
start = time.time()
doc = nlp(string)
end = time.time()

for ent in doc.ents:
    print(ent.text, '--->', ent.label_)

print(f'🥇🥇Total process timing: {end - start:.2f}')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🥇🥇Total process timing: 132.18
Cybersecurity Project Manager ---> person
organization ---> organization
technical teams ---> person
stakeholders ---> person
Agile PM ---> person
Technical Data Analyst ---> person
Sales users ---> person
Salesforce.com ---> organization
Dynamics 365 ---> organization
Sales and Finance ---> organization
Customer ---> person
business teams ---> organization
SFDC ---> organization
SFDC ---> organization
enterprise ---> organization
users ---> person
Salesforce.com ---> organization
Dynamics ---> organization
Product Managers ---> person
Engineering teams ---> organization
SFDC ---> organization
Dynamics ---> organization
managers ---> person
Engineering teams ---> organization
PdMs ---> organization
PdM ---> organization
PdM ---> organization
Engineering teams ---> organization
Preferred candidates ---> person
institution ---> school
BSA ---> organization
Databricks ---> organization
Power BI ---> organization
Tableau ---> organization
self-starter ---> pe

In [ ]:
import time
from multiprocessing import Pool, cpu_count

# model = GLiNER.from_pretrained("urchade/gliner_small-v2.1")

labels = ["Person", "Organization", "Location", "TravelRoute", "ID"]

def chunk_text(text, max_len=2048, overlap=48):
    chunks = []
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + max_len, text_len)
        chunks.append((text[start:end], start))
        if end == text_len:
            break
        start = end - overlap
    return chunks

def merge_entities(chunked_entities):
    merged = []
    seen = set()
    for ents, offset in chunked_entities:
        for e in ents:
            start = e["start"] + offset
            end = e["end"] + offset
            key = (e["text"], start, end, e["label"])
            if key not in seen:
                seen.add(key)
                merged.append({
                    "text": e["text"],
                    "start": start,
                    "end": end,
                    "label": e["label"]
                })
    return merged

def process_chunk(args):
    chunk, offset = args
    ents = model.predict_entities(chunk, labels, flat_ner=True, threshold=0.5)
    return (ents, offset)


chunks = chunk_text(string, max_len=512, overlap=12)
print(f"✂️ Total chunks: {len(chunks)}")

start_time = time.time()

with Pool(processes=min(cpu_count(), 4)) as pool:
    chunked_entities = pool.map(process_chunk, chunks)

merged_entities = merge_entities(chunked_entities)

end_time = time.time()

print("🔍 Merged Detected Entities:")
for ent in merged_entities:
    print(f"🔹 '{ent['text']}' → 🏷️ {ent['label']} 📍({ent['start']}-{ent['end']})")

print(f"\n⏱️ Total Inference Time: {end_time - start_time:.2f} seconds")

In [ ]:
!pip install -q spacy gliner
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Removing non-nouuns and sending them to GLiNER
# GLiNER Model:- urchade/gliner_small-v2.1
import re
import json
import time
import spacy
from gliner import GLiNER

start_spacy = time.time()
nlp = spacy.load('en_core_web_sm')
end_spacy = time.time()

start_gliner = time.time()
model = GLiNER.from_pretrained("urchade/gliner_small-v2.1")
end_gliner = time.time()

patterns = {
    'email': r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
    'phone': r'\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{4}\b',
    'id': r'\b[A-Z]{5}[0-9]{4}[A-Z]\b'
}

gliner_labels = ["Person", "Organization", "Location", "ID"]

def time_it(func):
  def wrapper(*args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    end = time.time()
    print(f'⏱️ {func.__name__} took {end - start:.2f} seconds')
    return result
  return wrapper

@time_it
def keep_nouns_with_positions(text):
    doc = nlp(text)
    noun_tokens = []
    original_positions = []
    for token in doc:
        if token.pos_ in ['PROPN']:
            noun_tokens.append(token.text)
            original_positions.append((token.idx, token.idx + len(token.text)))
    noun_text = ' '.join(noun_tokens)
    return noun_text, original_positions

@time_it
def chunk_text(text, max_len=2048, overlap=48):
    chunks = []
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + max_len, text_len)
        chunks.append((text[start:end], start))
        if end == text_len:
            break
        start = end - overlap
    return chunks

def map_positions(ents, noun_positions, chunk_offset):
    mapped_ents = []
    for ent in ents:
        ent_start = ent['start']
        ent_end = ent['end']
        start_char = None
        for orig_start, orig_end in noun_positions:
            if orig_start <= ent_start < orig_end:
                start_char = orig_start + (ent_start - orig_start)
                break
        if start_char is None:
            start_char = ent_start
        end_char = None
        for orig_start, orig_end in noun_positions:
            if orig_start < ent_end <= orig_end:
                end_char = orig_start + (ent_end - orig_start)
                break
        if end_char is None:
            end_char = ent_end

        mapped_ents.append({
            'text': ent['text'],
            'start': start_char + chunk_offset,
            'end': end_char + chunk_offset,
            'label': ent['label']
        })
    return mapped_ents

@time_it
def extract_entities(text):
    regex_entities = []
    for label, pattern in patterns.items():
        for match in re.finditer(pattern, text):
            regex_entities.append({
                'text': match.group(),
                'start': match.start(),
                'end': match.end(),
                'label': label
            })

    noun_text, noun_positions = keep_nouns_with_positions(text)

    with open('filtered_nouns_text.txt', 'w', encoding='utf-8') as f:
        for token in noun_text.split():
            f.write(token + '\n')


    chunks = chunk_text(noun_text, max_len=512, overlap=48)

    gliner_entities = []
    for chunk_text_part, offset in chunks:
        ents = model.predict_entities(chunk_text_part, gliner_labels, flat_ner=True, threshold=0.5)
        mapped = map_positions(ents, noun_positions, offset)
        gliner_entities.extend(mapped)

    all_entities = regex_entities + gliner_entities
    all_entities.sort(key=lambda x: x['start'])

    return all_entities

if __name__ == "__main__":

    start_total = time.time()
    entities = extract_entities(string)
    end_total = time.time()

    print(f"🕒 SpaCy loading time: {end_spacy - start_spacy:.2f} seconds")
    print(f"🕒 GLiNER loading time: {end_gliner - start_gliner:.2f} seconds")
    print(f"🕒 Total processing time: {end_total - start_total:.2f} seconds")

    with open('extracted_entities.json', 'w', encoding='utf-8') as f:
        json.dump({'entities': entities}, f, indent=4, ensure_ascii=False)

    for ent in entities:
        print(f"🔹 {ent['label'].upper()}: '{ent['text']}' [{ent['start']}:{ent['end']}]")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


⏱️ keep_nouns_with_positions took 1.86 seconds
⏱️ chunk_text took 0.00 seconds
⏱️ extract_entities took 13.20 seconds
🕒 SpaCy loading time: 1.00 seconds
🕒 GLiNER loading time: 14.07 seconds
🕒 Total processing time: 13.20 seconds
🔹 ORGANIZATION: 'Oracle' [105:111]
🔹 ORGANIZATION: 'SAP' [112:115]
🔹 PERSON: 'Technical Data Analyst' [220:242]
🔹 ORGANIZATION: 'Salesforce.com' [252:266]
🔹 ORGANIZATION: 'Dynamics' [267:275]
🔹 ORGANIZATION: 'Dynamics' [299:307]
🔹 ORGANIZATION: 'SFDC' [308:312]
🔹 ORGANIZATION: 'Dynamics' [313:321]
🔹 ORGANIZATION: 'SAP' [322:325]
🔹 ORGANIZATION: 'Salesforce.com' [371:385]
🔹 ORGANIZATION: 'Dynamics' [386:394]
🔹 ORGANIZATION: 'SFDC' [412:416]
🔹 ORGANIZATION: 'Dynamics' [417:425]
🔹 ORGANIZATION: 'Tableau' [570:577]
🔹 ORGANIZATION: 'Salesforce' [599:609]
🔹 ID: 'Lead II' [644:651]
🔹 PERSON: 'Data Analyst' [686:698]
🔹 PERSON: 'Data Scientist' [699:713]
🔹 LOCATION: 'Telecom' [735:742]
🔹 ORGANIZATION: 'Databricks' [783:793]
🔹 ORGANIZATION: 'Tableau' [827:834]
🔹 LOCATION

In [ ]:
# Removing non-nouuns and sending them to GLiNER
# GLiNER Model:- gliner-community/gliner_small-v2.5
import re
import json
import time
import spacy
from gliner import GLiNER

start_spacy = time.time()
nlp = spacy.load('en_core_web_sm')
end_spacy = time.time()

start_gliner = time.time()
model = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5")
end_gliner = time.time()

patterns = {
    'email': r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
    'phone': r'\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{4}\b',
    'id': r'\b[A-Z]{5}[0-9]{4}[A-Z]\b'
}

gliner_labels = ["Person", "Organization", "Location", "ID"]

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'⏱️ {func.__name__} took {end - start:.2f} seconds')
        return result
    return wrapper

@time_it
def keep_nouns_with_positions(text):
    doc = nlp(text)
    noun_tokens = []
    original_positions = []
    for token in doc:
        if token.pos_ == 'PROPN':
            noun_tokens.append(token.text)
            original_positions.append((token.idx, token.idx + len(token.text)))
    noun_text = ' '.join(noun_tokens)
    return noun_text, original_positions

@time_it
def chunk_text(text, max_len=512, overlap=48):
    chunks = []
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + max_len, text_len)
        chunks.append((text[start:end], start))
        if end == text_len:
            break
        start = end - overlap
    return chunks

def map_positions(ents, noun_positions, chunk_offset):
    mapped_ents = []
    for ent in ents:
        ent_start = ent['start']
        ent_end = ent['end']
        start_char = None
        for orig_start, orig_end in noun_positions:
            if orig_start <= ent_start < orig_end:
                start_char = orig_start + (ent_start - orig_start)
                break
        if start_char is None:
            start_char = ent_start
        end_char = None
        for orig_start, orig_end in noun_positions:
            if orig_start < ent_end <= orig_end:
                end_char = orig_start + (ent_end - orig_start)
                break
        if end_char is None:
            end_char = ent_end

        mapped_ents.append({
            'text': ent['text'],
            'start': start_char + chunk_offset,
            'end': end_char + chunk_offset,
            'label': ent['label']
        })
    return mapped_ents

@time_it
def extract_entities(text):
    regex_entities = []
    for label, pattern in patterns.items():
        for match in re.finditer(pattern, text):
            regex_entities.append({
                'text': match.group(),
                'start': match.start(),
                'end': match.end(),
                'label': label
            })

    noun_text, noun_positions = keep_nouns_with_positions(text)

    with open('filtered_nouns_text.txt', 'w', encoding='utf-8') as f:
        for token in noun_text.split():
            f.write(token + '\n')

    chunks = chunk_text(noun_text, max_len=512, overlap=24)

    gliner_entities = []
    for i, (chunk, offset) in enumerate(chunks):
        ents = model.predict_entities(chunk, gliner_labels, flat_ner=True, threshold=0.20)
        print(f"✅ Processed chunk {i+1}/{len(chunks)} with {len(ents)} entities")
        mapped = map_positions(ents, noun_positions, offset)
        gliner_entities.extend(mapped)

    all_entities = regex_entities + gliner_entities
    all_entities.sort(key=lambda x: x['start'])

    return all_entities

if __name__ == "__main__":
    start_total = time.time()
    entities = extract_entities(string)
    end_total = time.time()

    print(f"\n🕒 SpaCy loading time: {end_spacy - start_spacy:.2f} seconds")
    print(f"🕒 GLiNER loading time: {end_gliner - start_gliner:.2f} seconds")

    with open('extracted_entities.json', 'w', encoding='utf-8') as f:
        json.dump({'entities': entities}, f, indent=4, ensure_ascii=False)

    for ent in entities:
        print(f"🔹 {ent['label'].upper()}: '{ent['text']}' [{ent['start']}:{ent['end']}]")

    print(f"🕒 Total processing time: {end_total - start_total:.2f} seconds")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


⏱️ keep_nouns_with_positions took 1.56 seconds
⏱️ chunk_text took 0.00 seconds
✅ Processed chunk 1/22 with 11 entities
✅ Processed chunk 2/22 with 9 entities
✅ Processed chunk 3/22 with 3 entities
✅ Processed chunk 4/22 with 14 entities
✅ Processed chunk 5/22 with 12 entities
✅ Processed chunk 6/22 with 12 entities
✅ Processed chunk 7/22 with 14 entities
✅ Processed chunk 8/22 with 15 entities
✅ Processed chunk 9/22 with 9 entities
✅ Processed chunk 10/22 with 3 entities
✅ Processed chunk 11/22 with 8 entities
✅ Processed chunk 12/22 with 7 entities
✅ Processed chunk 13/22 with 12 entities
✅ Processed chunk 14/22 with 2 entities
✅ Processed chunk 15/22 with 19 entities
✅ Processed chunk 16/22 with 18 entities
✅ Processed chunk 17/22 with 22 entities
✅ Processed chunk 18/22 with 39 entities
✅ Processed chunk 19/22 with 40 entities
✅ Processed chunk 20/22 with 25 entities
✅ Processed chunk 21/22 with 17 entities
✅ Processed chunk 22/22 with 4 entities
⏱️ extract_entities took 10.32 secon

In [ ]:
!pip install -q optimum[onnxruntime] onnx

!pip install optimum[onnxruntime]
!pip uninstall -y onnxruntime
!pip install onnxruntime-gpu

  Using cached onnxruntime-1.22.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
Using cached onnxruntime-1.22.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2


In [ ]:
# Trying out ONNX optimiziation for classifier model
import time
from transformers import pipeline, AutoTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

classifier_model= "valhalla/distilbart-mnli-12-3"

start_onnx_conv = time.time()
ort_model = ORTModelForSequenceClassification.from_pretrained(classifier_model, export=True)
tokenizer = AutoTokenizer.from_pretrained(classifier_model)
end_onnx_conv = time.time()

print(f'ONNX Conversion time: {end_onnx_conv - start_onnx_conv:.2f}\n\n')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/modeling_bart.py:232: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.11/dist-packages/transformers/models/bart/modeling_bart.py:239: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.size() != (bsz, 1, tgt_len, src_len):
/usr/local/lib/python3.11/dist-packages/transformers/models/bart/modeling_bart.py:271: TracerWarning: Converting a tensor to a Python boolean might cause the trace to b

ONNX Conversion time: 53.78




In [ ]:
string = """
During the annual leadership summit hosted by Zenith Technologies, Dr. Amelia Carter presented her groundbreaking research on sustainable AI systems. As the Chief Innovation Officer at NeuroGrid Solutions, she’s often consulted by major tech giants like Infinisoft Corp and Helix Data Systems. Alongside her was Jonathan Reyes, the Head of Global Strategy at Orion Analytics, whose presentation on "AI in Defense Systems" drew applause from both military and civilian representatives. Their keynote was followed by a Q&A session moderated by Lena Moritz, a journalist from The Future Daily, who has covered tech industry trends for over a decade.

In attendance was also Rajesh Kumar, CTO of CyberPulse Labs, known for his team's breakthrough work in neural network optimization. Rajesh, who holds security clearance Level 5 under ID CP-9938472A, emphasized the need for ethical frameworks and cross-border regulations. Meanwhile, Emily Sanderson, Project Director at EnviroMind AI, discussed how their systems are now deployed in over 35 countries, aiding in climate change modeling. She provided her contact for follow-ups: emily.s@enviromind.org, and a direct line: +1-415-983-2094.

Behind the scenes, Carlos D'Souza, an independent security consultant from NetSentinel Group, was seen coordinating with Dr. Fiona MacGregor from BioSys Intelligence to ensure secure real-time data exchange during the live demos. Fiona's employee ID badge, tagged BIS-4428391Z, granted her full access to restricted zones during the showcase. Notably, a brief appearance was made by Sarah-Lee Thomson, the former Head of R&D at NeuraLink Global, whose early research in predictive models laid the foundation for many AI startups today.

Lunch was sponsored by QuantumEdge Ventures, a VC firm backing companies like NovaSynth AI and PixelForge Interactive. The event was catered by CloudKitchen Co., and the team lead, Michel Tanaka, made sure every guest had a customized meal option. Later in the day, a panel on AI and mental health featured Dr. Benjamin Osei, a psychologist from MindMap Collective, who shared anonymized case studies supported by data IDs like MMH-20230521-1123 for reproducibility in research.

At the networking gala, Chloe Martinez, known for her work with CivicNet Systems, introduced her recent paper titled “Decentralized Governance and Ethical AI” and exchanged business cards that bore her email: cmartinez@civicnet.com. The evening ended with a fireside chat between Elijah Banks, founder of CodeSight Innovations, and Anita Kapoor, CEO of EcoAI Robotics, discussing the future of AI legislation. Attendees were sent digital summaries via encrypted emails and each delegate’s badge ID, such as DEL-7789210, was logged for security auditing.


"""

print(len(string.split()))

406


In [ ]:
# Filtering out person name (Removing the false +ve from GLiNER O/P)
import re
import json
import time
import spacy
from gliner import GLiNER
from transformers import pipeline, AutoTokenizer
# from optimum.onnxruntime import ORTModelForSequenceClassification

classifier_model= "valhalla/distilbart-mnli-12-3"

# start_onnx_conv = time.time()
# ort_model = ORTModelForSequenceClassification.from_pretrained(classifier_model, export=True)
# tokenizer = AutoTokenizer.from_pretrained(classifier_model)
# end_onnx_conv = time.time()

# print(f'ONNX Conversion time: {end_onnx_conv - start_onnx_conv:.2f}\n\n')

start_spacy = time.time()
nlp = spacy.load('en_core_web_sm')
end_spacy = time.time()

start_gliner = time.time()
model = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5")
end_gliner = time.time()

start_zero_shot = time.time()
# # zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
# zero_shot = pipeline("zero-shot-classification", model = ort_model, tokenizer= tokenizer)
zero_shot_labels = ["person name", "job title"]
end_zero_shot = time.time()

classifier_cache = {}

patterns = {
    'email': r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
    'phone': r'\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{4}\b',
    'id': r'\b[A-Z]{5}[0-9]{4}[A-Z]\b'
}

gliner_labels = ["Person", "Organization", "Location", "ID"]

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'⏱️ {func.__name__} took {time.time() - start:.2f} seconds')
        return result
    return wrapper

@time_it
def keep_nouns_with_positions(text):
    doc = nlp(text)
    noun_tokens = []
    original_positions = []
    for token in doc:
        if token.pos_ == 'PROPN':
            noun_tokens.append(token.text)
            original_positions.append((token.idx, token.idx + len(token.text)))
    noun_text = ' '.join(noun_tokens)
    return noun_text, original_positions

@time_it
def chunk_text(text, max_len=512, overlap=24):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + max_len, len(text))
        chunks.append((text[start:end], start))
        if end == len(text):
            break
        start = end - overlap
    return chunks

def map_positions(ents, noun_positions, chunk_offset):
    mapped_ents = []
    for ent in ents:
        ent_start = ent['start']
        ent_end = ent['end']
        start_char = None
        for orig_start, orig_end in noun_positions:
            if orig_start <= ent_start < orig_end:
                start_char = orig_start + (ent_start - orig_start)
                break
        if start_char is None:
            start_char = ent_start

        end_char = None
        for orig_start, orig_end in noun_positions:
            if orig_start < ent_end <= orig_end:
                end_char = orig_start + (ent_end - orig_start)
                break
        if end_char is None:
            end_char = ent_end

        mapped_ents.append({
            'text': ent['text'],
            'start': start_char + chunk_offset,
            'end': end_char + chunk_offset,
            'label': ent['label']
        })
    return mapped_ents

def batch_classify_with_cache(entities, batch_size = 8):
    to_classify = list({ent["text"] for ent in entities if ent["label"] == "Person" and ent["text"] not in classifier_cache})

    for i in range(0, len(to_classify), batch_size):
        batch = to_classify[i:i + batch_size]
        results = zero_shot(batch, candidate_labels=zero_shot_labels)
        for text, result in zip(batch, results):
            classifier_cache[text] = result["labels"][0]

def filter_person_entities_with_classifier(entities):

    batch_classify_with_cache(entities = entities, batch_size= 8)

    filtered = []
    for ent in entities:
        if ent["label"] != "Person":
            filtered.append(ent)
        else:
            top_label = classifier_cache.get(ent["text"])
            if top_label == "person name":
                filtered.append(ent)
            else:
                print(f"❌ Rejected as job title: {ent['text']}")
    return filtered

@time_it
def extract_entities(text):
    regex_entities = []
    for label, pattern in patterns.items():
        for match in re.finditer(pattern, text):
            regex_entities.append({
                'text': match.group(),
                'start': match.start(),
                'end': match.end(),
                'label': label
            })

    noun_text, noun_positions = keep_nouns_with_positions(text)
    chunks = chunk_text(noun_text, max_len=512, overlap=48)

    start_gliner_extraction = time.time()

    gliner_entities = []
    for i, (chunk, offset) in enumerate(chunks):
        ents = model.predict_entities(chunk, gliner_labels, flat_ner=True, threshold=0.20)
        print(f"✅ Processed chunk {i+1}/{len(chunks)} with {len(ents)} entities")
        mapped = map_positions(ents, noun_positions, offset)
        gliner_entities.extend(mapped)

    end_gliner_extraction = time.time()
    print(f'🥇 Time through GLiNER: {end_gliner_extraction - start_gliner_extraction:.2f}\n\n')

    start_classifier_time = time.time()
    # gliner_entities = filter_person_entities_with_classifier(gliner_entities)
    end_classifier_time = time.time()

    print(f'🥇 Time through Classifier: {end_classifier_time - start_classifier_time:.2f}')

    all_entities = regex_entities + gliner_entities
    all_entities.sort(key=lambda x: x['start'])

    return all_entities

if __name__ == "__main__":
    start_total = time.time()
    entities = extract_entities(string)
    end_total = time.time()

    print(f"\n🕒 SpaCy loading time: {end_spacy - start_spacy:.2f} seconds")
    print(f"🕒 GLiNER loading time: {end_gliner - start_gliner:.2f} seconds")
    print(f"🕒 Zero Shot Classifier loading time: {end_zero_shot - start_zero_shot:.2f} seconds")

    with open('filtered_entity_mapping.json', 'w', encoding='utf-8') as f:
        json.dump({'entities': entities}, f, indent=4, ensure_ascii=False)

    for ent in entities:
        print(f"🔹 {ent['label'].upper()}: '{ent['text']}' [{ent['start']}:{ent['end']}]")

    print(f"🕒 Total processing time: {end_total - start_total:.2f} seconds")


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/23.3k [00:00<?, ?B/s]

models_comparison.png:   0%|          | 0.00/156k [00:00<?, ?B/s]

...

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


⏱️ keep_nouns_with_positions took 4.30 seconds
⏱️ chunk_text took 0.00 seconds
✅ Processed chunk 1/23 with 11 entities
✅ Processed chunk 2/23 with 10 entities
✅ Processed chunk 3/23 with 4 entities
✅ Processed chunk 4/23 with 13 entities
✅ Processed chunk 5/23 with 14 entities
✅ Processed chunk 6/23 with 13 entities
✅ Processed chunk 7/23 with 11 entities
✅ Processed chunk 8/23 with 16 entities
✅ Processed chunk 9/23 with 12 entities
✅ Processed chunk 10/23 with 5 entities
✅ Processed chunk 11/23 with 3 entities
✅ Processed chunk 12/23 with 11 entities
✅ Processed chunk 13/23 with 8 entities
✅ Processed chunk 14/23 with 7 entities
✅ Processed chunk 15/23 with 3 entities
✅ Processed chunk 16/23 with 27 entities
✅ Processed chunk 17/23 with 11 entities
✅ Processed chunk 18/23 with 31 entities
✅ Processed chunk 19/23 with 38 entities
✅ Processed chunk 20/23 with 34 entities
✅ Processed chunk 21/23 with 23 entities
✅ Processed chunk 22/23 with 18 entities
✅ Processed chunk 23/23 with 5 ent

In [ ]:
# Filtering out person name (Removing the false +ve from GLiNER O/P)
# Entire descriptive column is being tested
import re
import json
import time
import spacy
import pandas as pd
from gliner import GLiNER
from transformers import pipeline, AutoTokenizer
# from optimum.onnxruntime import ORTModelForSequenceClassification

classifier_model= "valhalla/distilbart-mnli-12-3"

# start_onnx_conv = time.time()
# ort_model = ORTModelForSequenceClassification.from_pretrained(classifier_model, export=True)
# tokenizer = AutoTokenizer.from_pretrained(classifier_model)
# end_onnx_conv = time.time()

# print(f'ONNX Conversion time: {end_onnx_conv - start_onnx_conv:.2f}\n\n')

start_spacy = time.time()
nlp = spacy.load('en_core_web_sm')
end_spacy = time.time()

start_gliner = time.time()
model = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5")
end_gliner = time.time()

start_zero_shot = time.time()
# zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
zero_shot = pipeline("zero-shot-classification", model = ort_model, tokenizer= tokenizer)
zero_shot_labels = ["person name", "job title"]
end_zero_shot = time.time()

classifier_cache = {}

patterns = {
    'email': r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
    'phone': r'\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{2,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{4}\b',
    'id': r'\b[A-Z]{5}[0-9]{4}[A-Z]\b'
}

gliner_labels = ["Person", "Organization", "Location", "ID"]

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'⏱️ {func.__name__} took {time.time() - start:.2f} seconds')
        return result
    return wrapper

@time_it
def keep_nouns_with_positions(text):
    doc = nlp(text)
    noun_tokens = []
    original_positions = []
    for token in doc:
        if token.pos_ == 'PROPN':
            noun_tokens.append(token.text)
            original_positions.append((token.idx, token.idx + len(token.text)))
    noun_text = ' '.join(noun_tokens)
    return noun_text, original_positions

@time_it
def chunk_text(text, max_len=512, overlap=24):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + max_len, len(text))
        chunks.append((text[start:end], start))
        if end == len(text):
            break
        start = end - overlap
    return chunks

def map_positions(ents, noun_positions, chunk_offset):
    mapped_ents = []
    for ent in ents:
        ent_start = ent['start']
        ent_end = ent['end']
        start_char = None
        for orig_start, orig_end in noun_positions:
            if orig_start <= ent_start < orig_end:
                start_char = orig_start + (ent_start - orig_start)
                break
        if start_char is None:
            start_char = ent_start

        end_char = None
        for orig_start, orig_end in noun_positions:
            if orig_start < ent_end <= orig_end:
                end_char = orig_start + (ent_end - orig_start)
                break
        if end_char is None:
            end_char = ent_end

        mapped_ents.append({
            'text': ent['text'],
            'start': start_char + chunk_offset,
            'end': end_char + chunk_offset,
            'label': ent['label']
        })
    return mapped_ents

def batch_classify_with_cache(entities, batch_size = 8):
    to_classify = list({ent["text"] for ent in entities if ent["label"] == "Person" and ent["text"] not in classifier_cache})

    for i in range(0, len(to_classify), batch_size):
        batch = to_classify[i:i + batch_size]
        results = zero_shot(batch, candidate_labels=zero_shot_labels)
        for text, result in zip(batch, results):
            classifier_cache[text] = result["labels"][0]

def filter_person_entities_with_classifier(entities):
    batch_classify_with_cache(entities = entities, batch_size= 8)

    filtered = []
    for ent in entities:
        if ent["label"] != "Person":
            filtered.append(ent)
        else:
            top_label = classifier_cache.get(ent["text"])
            if top_label == "person name":
                filtered.append(ent)
            else:
                print(f"❌ Rejected as job title: {ent['text']}")
    return filtered

@time_it
def extract_entities(text):
    regex_entities = []
    for label, pattern in patterns.items():
        for match in re.finditer(pattern, text):
            regex_entities.append({
                'text': match.group(),
                'start': match.start(),
                'end': match.end(),
                'label': label
            })

    noun_text, noun_positions = keep_nouns_with_positions(text)
    chunks = chunk_text(noun_text, max_len=512, overlap=48)

    start_gliner_extraction = time.time()

    gliner_entities = []
    for i, (chunk, offset) in enumerate(chunks):
        ents = model.predict_entities(chunk, gliner_labels, flat_ner=True, threshold=0.20)
        print(f"✅ Processed chunk {i+1}/{len(chunks)} with {len(ents)} entities")
        mapped = map_positions(ents, noun_positions, offset)
        gliner_entities.extend(mapped)

    end_gliner_extraction = time.time()
    print(f'🥇 Time through GLiNER: {end_gliner_extraction - start_gliner_extraction:.2f}\n\n')

    start_classifier_time = time.time()
    gliner_entities = filter_person_entities_with_classifier(gliner_entities)
    end_classifier_time = time.time()

    print(f'🥇 Time through Classifier: {end_classifier_time - start_classifier_time:.2f}')

    all_entities = regex_entities + gliner_entities
    all_entities.sort(key=lambda x: x['start'])

    return all_entities

@time_it
def process_column_text(col_name, text):
    print(f"\n📂 Processing column: {col_name}")
    start_time = time.time()
    entities = extract_entities(text)
    end_time = time.time()
    print(f"🕓 Time taken for column '{col_name}': {end_time - start_time:.2f} seconds")
    return entities

if __name__ == "__main__":
    start_total = time.time()

    excel_file = "SO(real).xlsx"
    descriptive_columns = [
        "Comments", "Reasons", "Job Description", "Notes for WFM or TA", "UST Role Description"
    ]

    df = pd.read_excel(excel_file, dtype=str, engine='calamine')
    all_column_entities = {}

    for col in descriptive_columns:
        if col not in df.columns:
            print(f"⚠️ Column '{col}' not found in Excel file.\n")
            continue

        full_text = ' '.join(df[col].dropna().astype(str).tolist())
        column_entities = process_column_text(col, full_text)
        all_column_entities[col] = column_entities

        with open(f'filtered_entity_mapping_{col}.json', 'w', encoding='utf-8') as f:
            json.dump({'entities': column_entities}, f, indent=4, ensure_ascii=False)

        for ent in column_entities:
            print(f"🔹 {ent['label'].upper()}: '{ent['text']}' [{ent['start']}:{ent['end']}]")

    end_total = time.time()

    with open("desc_entities.json", "w", encoding="utf-8") as f:
        json.dump(all_column_entities, f, indent=2, ensure_ascii=False)

    print(f"\n🕒 SpaCy loading time: {end_spacy - start_spacy:.2f} seconds")
    print(f"🕒 GLiNER loading time: {end_gliner - start_gliner:.2f} seconds")
    print(f"🕒 Zero Shot Classifier loading time: {end_zero_shot - start_zero_shot:.2f} seconds")
    print(f"🕒 Total processing time: {end_total - start_total:.2f} seconds")


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Device set to use cuda:0



📂 Processing column: Comments


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


⏱️ keep_nouns_with_positions took 5.30 seconds
⏱️ chunk_text took 0.00 seconds
✅ Processed chunk 1/139 with 34 entities
✅ Processed chunk 2/139 with 40 entities
✅ Processed chunk 3/139 with 39 entities
✅ Processed chunk 4/139 with 21 entities
✅ Processed chunk 5/139 with 18 entities
✅ Processed chunk 6/139 with 29 entities
✅ Processed chunk 7/139 with 32 entities
✅ Processed chunk 8/139 with 40 entities
✅ Processed chunk 9/139 with 34 entities
✅ Processed chunk 10/139 with 29 entities
✅ Processed chunk 11/139 with 30 entities
✅ Processed chunk 12/139 with 25 entities
✅ Processed chunk 13/139 with 28 entities
✅ Processed chunk 14/139 with 35 entities
✅ Processed chunk 15/139 with 14 entities
✅ Processed chunk 16/139 with 21 entities
✅ Processed chunk 17/139 with 32 entities
✅ Processed chunk 18/139 with 34 entities
✅ Processed chunk 19/139 with 23 entities
✅ Processed chunk 20/139 with 20 entities
✅ Processed chunk 21/139 with 15 entities
✅ Processed chunk 22/139 with 23 entities
✅ Proc

ValueError: [E088] Text of length 3289297 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [ ]:
# Testing out the classifier

test_string = '''
David Fraley | Naveen | Vinesh | RH ADOBE | Sasmitha Das | Amarnath Nishant | Vijay' | Beeline | COMCAST | Poorna Chandra | Jyothsih | Glider | Jyothish | TMO Deb/ Beeline | Keerthi | Durga Maruthi | Srujan | Sravan Kumar | Ayyagari | Vitakula | Teja Sai Ganta | TMO Deb/ Beeline | Kiran Krishnan
'''

start = time.time()
for word in test_string.split(' | '):
  result = classifier(word, candidate_labels=["person name", "job title"])
  print(f"{result['sequence']} -> {result['labels'][0]}")
end = time.time()

print(f'Total time: {end - start:.2f}')


David Fraley -> person name
Naveen -> person name
Vinesh -> person name
RH ADOBE -> job title
Sasmitha Das -> person name
Amarnath Nishant -> person name
Vijay' -> person name
Beeline -> person name
COMCAST -> job title
Poorna Chandra -> person name
Jyothsih -> person name
Glider -> job title
Jyothish -> person name
TMO Deb/ Beeline -> job title
Keerthi -> person name
Durga Maruthi -> person name
Srujan -> person name
Sravan Kumar -> person name
Ayyagari -> person name
Vitakula -> person name
Teja Sai Ganta -> person name
TMO Deb/ Beeline -> job title
Kiran Krishnan
 -> person name
Total time: 19.33


In [ ]:
# NLTK corpus testing
import nltk
import time
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from functools import lru_cache

start_download = time.time()
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
end_download = time.time()

print(f'Total time for downloading: {end_download - start_download:.2f}')

def time_it(func):
  def wrapper(*args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    print(f'⏱️ {func.__name__} took {time.time() - start:.2f} seconds')
    return result
  return wrapper

@time_it
def get_proper_nouns(text):
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    proper_nouns = [word for word, tag in tagged if tag in ('NNP', 'NNPS')]
    return proper_nouns

@lru_cache(maxsize=100000)
def is_in_dictionary(word):
    return bool(wn.synsets(word))

@time_it
def filter_non_dict_words(words):
    filtered = [word for word in words if not is_in_dictionary(word)]
    return filtered

@time_it
def extract_final_words(text):
    proper_nouns = get_proper_nouns(text)
    final_words = filter_non_dict_words(proper_nouns)

    return final_words
@time_it
def save_words_to_file(words, filename="filtered_output.txt"):
    with open(filename, "w", encoding="utf-8") as f:
        for word in words:
            f.write(word + "\n")
    print(f"{len(words)} words saved to '{filename}'")

if __name__ == "__main__":
    input_text = """
    James Bond works at Microsoft. He met Angela in Paris on Monday.
    They discussed Quantum mechanics and Artificial Intelligence.
    """
    results = extract_final_words(input_text)
    save_words_to_file(results)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Total time for downloading: 0.09
⏱️ get_proper_nouns took 0.11 seconds
⏱️ filter_non_dict_words took 2.62 seconds
⏱️ extract_final_words took 2.73 seconds
2 words saved to 'filtered_output.txt'
⏱️ save_words_to_file took 0.00 seconds


In [ ]:
# Installing dictionary
!pip install pyenchant

  Using cached pyenchant-3.2.2-py3-none-any.whl.metadata (3.8 kB)
Using cached pyenchant-3.2.2-py3-none-any.whl (55 kB)


In [ ]:
# Testing out python Enchant Dictionary

import re
import json
from gliner import GLiNER

model = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5")

# Define labels you want GLiNER to detect
gliner_labels = ["Person", "Organization", "Location", "ID"]

# Read the full string from the text file
with open("filtered_desc_strings.txt", "r", encoding="utf-8") as f:
    full_text = f.read().strip()

# Calculate word positions in the full string
def get_word_positions(text):
    positions = []
    for match in re.finditer(r'\S+', text):
        word = match.group()
        start = match.start()
        end = match.end()
        positions.append((word, start, end))
    return positions

word_positions = get_word_positions(full_text)

# Predict entities using GLiNER
entities = model.predict_entities(full_text, gliner_labels, flat_ner=True, threshold=0.2)

# Filter entities to ensure correct positions
final_entities = []
for ent in entities:
    final_entities.append({
        "text": ent["text"],
        "start": ent["start"],
        "end": ent["end"],
        "label": ent["label"]
    })

# Save result to JSON
with open("desc_entities_from_textfile.json", "w", encoding="utf-8") as f:
    json.dump(final_entities, f, indent=2, ensure_ascii=False)

# Optional: Print summary
for ent in final_entities:
    print(f"{ent['label'].upper()} - '{ent['text']}' at ({ent['start']}, {ent['end']})")


In [ ]:
!pip install requests

In [ ]:
# Downloading English dictionary sources
import requests

def downloading_word_list():
    url = 'https://www.mit.edu/~ecprice/wordlist.10000'

    response = requests.get(url)

    if response.status_code == 200:
        with open('10000_WordList.txt', 'w', encoding='utf-8') as fp:
            fp.write(response.text)
    else:
        print(f'Failed to download word list. Status code: {response.status_code}')

    print(f'Task completed')

# downloading_word_list()

def download_en_us_dictionary():
    base_url = "https://raw.githubusercontent.com/LibreOffice/dictionaries/master/en/"
    files = ["en_US.dic", "en_US.aff"]
    for file in files:
        response = requests.get(base_url + file)
        if response.status_code == 200:
            with open(file, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded {file}")
        else:
            print(f"Failed to download {file}. Status code: {response.status_code}")

download_en_us_dictionary()

Downloaded en_US.dic
Downloaded en_US.aff


In [ ]:
import onnxruntime as ort
print(ort.get_available_providers())


['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [ ]:
# !pip install -q gliner spacy
# !pip install -q polars rapidfuzz pyxlsb fastexcel python-calamine gliner-spacy
!pip install -q optimum[onnxruntime]
# !pip uninstall onnxruntime onnxruntime-gpu -y
# !pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 26.5 MB/s eta 0:00:00


In [ ]:
# !pip show gliner
!pip show spacy

Name: spacy
Version: 3.8.7
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, preshed, pydantic, requests, setuptools, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi, weasel
Required-by: fastai


In [ ]:
!cd /usr/local/lib/python3.11/dist-packages && \
 zip -r /content/gliner.zip gliner

  adding: gliner/ (stored 0%)
  adding: gliner/data_processing/ (stored 0%)
  adding: gliner/data_processing/tokenizer.py (deflated 77%)
  adding: gliner/data_processing/dataset.py (deflated 71%)
  adding: gliner/data_processing/__pycache__/ (stored 0%)
  adding: gliner/data_processing/__pycache__/collator.cpython-311.pyc (deflated 53%)
  adding: gliner/data_processing/__pycache__/dataset.cpython-311.pyc (deflated 47%)
  adding: gliner/data_processing/__pycache__/tokenizer.cpython-311.pyc (deflated 64%)
  adding: gliner/data_processing/__pycache__/utils.cpython-311.pyc (deflated 43%)
  adding: gliner/data_processing/__pycache__/processor.cpython-311.pyc (deflated 62%)
  adding: gliner/data_processing/__pycache__/__init__.cpython-311.pyc (deflated 38%)
  adding: gliner/data_processing/collator.py (deflated 73%)
  adding: gliner/data_processing/processor.py (deflated 80%)
  adding: gliner/data_processing/__init__.py (deflated 45%)
  adding: gliner/data_processing/utils.py (deflated 60%)


In [ ]:
from google.colab import files
files.download("/content/gliner.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from gliner import GLiNER

# model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1", load_onnx_model=False)
# model.save_pretrained("urchade_gliner_medium_v2.1_local")

# !zip -r urchade_gliner_medium_v2.1_local.zip urchade_gliner_medium_v2.1_local

from google.colab import files
files.download("/content/urchade_gliner_medium_v2.1_local.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !python -m spacy download en_core_web_sm
import spacy
from pathlib import Path
import shutil


model_path = Path(spacy.util.get_package_path('en_core_web_sm'))
print(model_path)
shutil.make_archive("en_core_web_sm", 'zip', model_path)

from google.colab import files
files.download("en_core_web_sm.zip")


/usr/local/lib/python3.11/dist-packages/en_core_web_sm


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "valhalla/distilbart-mnli-12-3"

model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.save_pretrained("/content/distilbart_onnx")
tokenizer.save_pretrained("/content/distilbart_onnx")

from google.colab import files
import shutil

shutil.make_archive("/content/distilbart_onnx", 'zip', "/content/distilbart_onnx")

files.download("/content/distilbart_onnx.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q "optimum[onnxruntime]" transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import shutil
from google.colab import files
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

model_id = "valhalla/distilbart-mnli-12-3"
save_dir = "./distilbart_onnx"

model = ORTModelForSequenceClassification.from_pretrained(model_id, export=True)
model.save_pretrained(save_dir)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.save_pretrained(save_dir)
shutil.make_archive(save_dir, 'zip', save_dir)
files.download(save_dir + ".zip")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/modeling_bart.py:232: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.11/dist-packages/transformers/models/bart/modeling_bart.py:239: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.size() != (bsz, 1, tgt_len, src_len):
/usr/local/lib/python3.11/dist-packages/transformers/models/bart/modeling_bart.py:271: TracerWarning: Converting a tensor to a Python boolean might cause the trace to b

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip show optimum

Name: optimum
Version: 1.25.3
Summary: Optimum Library is an extension of the Hugging Face Transformers library, providing a framework to integrate third-party libraries from Hardware Partners and interface with their specific functionality.
Home-page: https://github.com/huggingface/optimum
Author: HuggingFace Inc. Special Ops Team
Author-email: hardware@huggingface.co
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, numpy, packaging, torch, transformers
Required-by: 


In [ ]:
import time
from gliner import GLiNER


model_start = time.time()
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1", load_onnx_model=False)
model_end = time.time()

text = """
Midhun James is working at UST for 1 year
"""

string = """
During the annual leadership summit hosted by Zenith Technologies, Dr. Amelia Carter presented her groundbreaking research on sustainable AI systems. As the Chief Innovation Officer at NeuroGrid Solutions, she’s often consulted by major tech giants like Infinisoft Corp and Helix Data Systems. Alongside her was Jonathan Reyes, the Head of Global Strategy at Orion Analytics, whose presentation on "AI in Defense Systems" drew applause from both military and civilian representatives. Their keynote was followed by a Q&A session moderated by Lena Moritz, a journalist from The Future Daily, who has covered tech industry trends for over a decade.
In attendance was also Rajesh Kumar, CTO of CyberPulse Labs, known for his team's breakthrough work in neural network optimization. Rajesh, who holds security clearance Level 5 under ID CP-9938472A, emphasized the need for ethical frameworks and cross-border regulations. Meanwhile, Emily Sanderson, Project Director at EnviroMind AI, discussed how their systems are now deployed in over 35 countries, aiding in climate change modeling. She provided her contact for follow-ups: emily.s@enviromind.org, and a direct line: +1-415-983-2094.
Behind the scenes, Carlos D'Souza, an independent security consultant from NetSentinel Group, was seen coordinating with Dr. Fiona MacGregor from BioSys Intelligence to ensure secure real-time data exchange during the live demos. Fiona's employee ID badge, tagged BIS-4428391Z, granted her full access to restricted zones during the showcase. Notably, a brief appearance was made by Sarah-Lee Thomson, the former Head of R&D at NeuraLink Global, whose early research in predictive models laid the foundation for many AI startups today.
Lunch was sponsored by QuantumEdge Ventures, a VC firm backing companies like NovaSynth AI and PixelForge Interactive. The event was catered by CloudKitchen Co., and the team lead, Michel Tanaka, made sure every guest had a customized meal option. Later in the day, a panel on AI and mental health featured Dr. Benjamin Osei, a psychologist from MindMap Collective, who shared anonymized case studies supported by data IDs like MMH-20230521-1123 for reproducibility in research.
At the networking gala, Chloe Martinez, known for her work with CivicNet Systems, introduced her recent paper titled “Decentralized Governance and Ethical AI” and exchanged business cards that bore her email: cmartinez@civicnet.com. The evening ended with a fireside chat between Elijah Banks, founder of CodeSight Innovations, and Anita Kapoor, CEO of EcoAI Robotics, discussing the future of AI legislation. Attendees were sent digital summaries via encrypted emails and each delegate’s badge ID, such as DEL-7789210, was logged for security auditing.


"""

labels = ["person", 'organization']

total_start = time.time()
entities = model.predict_entities(string, labels, threshold=0.5)
total_end = time.time()

print(f'Total time: {total_end - total_start:.2f}')
print(f'Model time: {model_end - model_start:.2f}')

for entity in entities:
    print(f"{entity['text']} => {entity['label']}")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gliner/data_processing/processor.py:296: UserWarning: Sentence of length 498 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Total time: 4.50
Model time: 20.55
Zenith Technologies => organization
Dr. Amelia Carter => person
NeuroGrid Solutions => organization
Infinisoft Corp => organization
Helix Data Systems => organization
Jonathan Reyes => person
Orion Analytics => organization
Lena Moritz => person
The Future Daily => organization
Rajesh Kumar => person
CyberPulse Labs => organization
Emily Sanderson => person
EnviroMind AI => organization
Carlos D'Souza => person
NetSentinel Group => organization
Dr. Fiona MacGregor => person
BioSys Intelligence => organization
Sarah-Lee Thomson => person
NeuraLink Global => organization
QuantumEdge Ventures => organization
NovaSynth AI => organization
PixelForge Interactive => organization
CloudKitchen Co. => organization
Michel Tanaka => person
Dr. Benjamin Osei => person
MindMap Collective => organization


In [ ]:
# ONNX implementation of urchade/gliner-medium-v2.1
!pip install transformers accelerate optimum

In [ ]:
!pip install huggingface_hub

from huggingface_hub import snapshot_download

# Download the ONNX GLiNER model repo locally in Colab
model_dir = snapshot_download(repo_id="onnx-community/gliner_medium-v2.1")

print(f"Model downloaded to: {model_dir}")


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/545 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--onnx-community--gliner_medium-v2.1/snapshots/959437589dc623d4c0a93f6e2828213567929cde


In [ ]:
import shutil
from google.colab import files

shutil.make_archive("gliner_onnx_models", 'zip', model_dir)
files.download("gliner_onnx_models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!tar -xzf urchade_gliner_medium_v2.1_local.tar.gz -C /content/


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [ ]:
!pip install -q presidio_analyzer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.3 MB/s eta 0:00:00


In [ ]:
# Json using presidio
import gzip
import json
import pprint
from collections import defaultdict
from presidio_analyzer import AnalyzerEngine

input_data = {
    "name": "Alice Johnson",
    "email": "alice.johnson@example.com",
    "phone": "+14155552671",
    "location": "San Francisco",
    "company": "Microsoft",
    "credit_card": "4111 1111 1111 1111",
    "url": "https://www.linkedin.com/in/alicejohnson",
    "country": "United States",
    "id": "AB1234567",
    "profile": {
        "bio": "Alice Johnson is a software engineer living in San Francisco. Contact her at alice.johnson@example.com or call +14155552671.",
        "social": ["https://twitter.com/alicejohnson", "https://facebook.com/alicejohnson"]
    }
}

fake_data = {}
with gzip.open("faker_data_v3.json.gz", "rt",encoding="utf-8") as f: fake_data_list = json.load(f)
for d in fake_data_list: fake_data.update(d)

entity_mapping = {
    'PERSON': 'names',
    'EMAIL_ADDRESS': 'emails',
    'PHONE_NUMBER': 'phone',
    'LOCATION': 'location',
    'CREDIT_CARD': 'credit',
    'URL': 'url',
    'COUNTRY': 'country',
    'ORG': 'company',
    'ID': 'id',
}

analyzer = AnalyzerEngine()

mappings = defaultdict(dict)

def get_fake_value(entity, original_value):
    """Retrieve a fake value while ensuring consistency."""
    mapped_entity = entity_mapping.get(entity)
    if not mapped_entity or mapped_entity not in fake_data:
        return "[REDACTED]"

    if original_value in mappings[mapped_entity]:
        return mappings[mapped_entity][original_value]

    if fake_data[mapped_entity]:
      fake_value = fake_data[mapped_entity].pop(0)
      fake_data[mapped_entity].append(fake_value)
      # if mapped_entity == 'url':
      #   fake_value = f'https://{fake_value}.com'
      mappings[mapped_entity][original_value] = fake_value
      return fake_value

    return "[REDACTED]"

def anonymize_json(data):
  if isinstance(data, dict):
    return {key: anonymize_json(value) for key, value in data.items()}

  elif isinstance(data, list):
    return [anonymize_json(item) for item in data]

  elif isinstance(data, str):
    results = analyzer.analyze(text=data, entities=list(entity_mapping.keys()), language="en")

    if not results: return data

    results = sorted(results, key=lambda x: x.start, reverse=True)
    for result in results:
        entity_type = result.entity_type
        start, end = result.start, result.end
        sensitive_text = data[start:end].strip().strip(",. ")

        fake_value = get_fake_value(entity_type, sensitive_text)

        data = data[:start] + fake_value + data[end:]

    return data

  else:
      return data


pprint.pprint(f'{anonymize_json(input_data)}\n\n\n')
pprint.pprint(mappings)

("{'name': 'A', 'email': 'chavez-solomonael@example.com', 'phone': "
 "'001-377-755-7593x2399', 'location': 'Port Brett, Delaware Region', "
 "'company': 'Microsoft', 'credit_card': '6528835898351118', 'url': "
 "'roman-rogers', 'country': 'Jefferyburgh, South Dakota', 'id': 'AB1234567', "
 "'profile': {'bio': 'A is a software engineer living in Port Brett, Delaware "
 "Region. Contact her at jacksonley@example.comll 001-377-755-7593x2399.', "
 "'social': ['duran', 'harrell-hawkins']}}\n"
 '\n'
 '\n')
defaultdict(<class 'dict'>,
            {'credit': {'4111 1111 1111 1111': '6528835898351118'},
             'emails': {'alice.johnson@young': 'laramichael@example.com',
                        'alice.johnson@young or ca': 'andrewwiley@example.com'},
             'location': {'San Francisco': 'Port Brett, Delaware Region',
                          'United States': 'Jefferyburgh, South Dakota'},
             'names': {'Alice Johnson': 'A'},
             'phone': {'+14155552671': '001-377-

In [ ]:
!pip install -q gliner presidio_analyzer flair spacy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.5/116.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# JSON masking using GLiNER and REGEX
import re
import json
import time
import gzip, json, pprint
from gliner import GLiNER
from collections import defaultdict

input_data = {
    "name": "Alice Johnson",
    "email": "alicejohnson908@gmail.com",
    "phone": "+976-0509821676",
    "location": "United States",
    "company": "Microsoft",
    "credit_card": "4111 1111 1111 1111",
    "url": "https://www.linkedin.com/in/alicejohnson",
    "country": "United States",
    "id": "AB1234567",
    "profile": {
        "bio": "Alice Johnson is a software engineer living in San Francisco. Contact her at alicejohnson908@gmail.com or call +91-9947155482.",
        "social": ["https://twitter.com/alicejohnson", "https://facebook.com/alicejohnson"]
    }
}

fake_data = {}
with gzip.open("faker_data_v3.json.gz", "rt", encoding="utf-8") as f:
    for d in json.load(f):
        fake_data.update(d)

entity_mapping = {
    'person':'names', 'organization':'company', 'location':'country',
    "email":"emails","mobile number":"phone","url":"url","credit card":"credit"
}

entity_mapping = {
    'person': 'names',
    'organization': 'company',
    'location': 'country'
}

regex_patterns = {
    'emails': re.compile(r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:\.[a-zA-Z]{2,})?\b'),
    'phone': re.compile(r'''
        \b
        (?:\+?(\d{1,3}))?[-.\s]?
        (?:\(?\d{2,4}\)?)?[-.\s]?
        \d{3,5}[-.\s]?\d{4}
        (?:\s*(?:ext|x|extension)\s*\d{1,5})?
        \b
    ''', re.VERBOSE),
    'url': re.compile(r'''
        \b
        (https?://)?
        (?:[-a-zA-Z0-9@:%._\+~#=]{1,256}\.)+
        [a-zA-Z]{2,6}
        \b
        (?:[-a-zA-Z0-9@:%_\+.~#?&//=]*)
    ''', re.VERBOSE),
    'credit': re.compile(r'''
        \b
        (?:4[0-9]{12}(?:[0-9]{3})?          # Visa
        |5[1-5][0-9]{14}                    # MasterCard
        |3[47][0-9]{13}                     # American Express
        |3(?:0[0-5]|[68][0-9])[0-9]{11}    # Diners Club
        |6(?:011|5[0-9]{2})[0-9]{12}       # Discover
        |(?:2131|1800|35\d{3})\d{11}       # JCB
        )
        \b
    ''', re.VERBOSE)
}


model = GLiNER.from_pretrained("urchade/gliner_small-v2.1")
mappings = defaultdict(dict)

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'⏱️ {func.__name__} took {time.time() - start:.2f} seconds')
        return result
    return wrapper

def get_fake(label, original):
    if label not in fake_data:
        return "[REDACTED]"
    if original in mappings[label]:
        return mappings[label][original]

    fake = fake_data[label].pop(0)
    fake_data[label].append(fake)
    if label == 'url':
      fake = f'https://{fake}.co.org'
    mappings[label][original] = fake
    return fake

def is_overlapping(start, end, protected_spans):
    for ps_start, ps_end in protected_spans:
        if max(start, ps_start) < min(end, ps_end):  # overlap condition
            return True
    return False

def anonymize_str(text):
    spans = []
    protected_spans = set()

    for label, pattern in regex_patterns.items():
      for match in re.finditer(pattern, text):
        start, end, orig = match.start(), match.end(), match.group()

        overlap = any(start < e and end > s for (s, e) in protected_spans)
        if overlap:
            continue

        spans.append((start, end, label, orig))
        protected_spans.add((start, end))

    ents = model.predict_entities(text, list(entity_mapping), flat_ner=True)
    for e in ents:
      start, end = e["start"], e["end"]
      if any(start < e_ and end > s_ for (s_, e_) in protected_spans):
            continue

      label = entity_mapping[e["label"].lower()]
      spans.append((start, end, label, text[start:end]))
      protected_spans.add((start, end))

    if not spans:
        return text

    spans.sort(key=lambda x: x[0], reverse=True)
    for start, end, label, orig in spans:
        fake = get_fake(label, orig)
        text = text[:start] + fake + text[end:]

    return text

def anon(obj):
    if isinstance(obj, dict):
        return {k: anon(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [anon(v) for v in obj]
    if isinstance(obj, str):
        return anonymize_str(obj)
    return obj

@time_it
def main():
  with open('dummy.json', 'r') as fp:
    json_file_data = json.load(fp)
  anonymized = anon(json_file_data)
  with open('json_mapping.json', 'w') as fp:
    json.dump(mappings, fp, indent = 4)

if __name__ == '__main__':
  main()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


⏱️ main took 267.26 seconds


In [ ]:
# JSON implementation using Presidio
import re
import json
import time
import gzip, pprint
from presidio_analyzer import AnalyzerEngine
from collections import defaultdict

fake_data = {}
analyzer = AnalyzerEngine()
mappings = defaultdict(dict)

with gzip.open("faker_data_v3.json.gz", "rt", encoding="utf-8") as f:
    for d in json.load(f):
        fake_data.update(d)

entity_mapping = {
    'PERSON': 'names',
    'ORG': 'company',
    'LOCATION': 'country'
}

regex_patterns = {
    'email': re.compile(r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:\.[a-zA-Z]{2,})?\b'),
    'phone': re.compile(r'''
        \b
        (?:\+?(\d{1,3}))?[-.\s]?
        (?:\(?\d{2,4}\)?)?[-.\s]?
        \d{3,5}[-.\s]?\d{4}
        (?:\s*(?:ext|x|extension)\s*\d{1,5})?
        \b
    ''', re.VERBOSE),
    'url': re.compile(r'''
        \b
        (https?://)?
        (?:[-a-zA-Z0-9@:%._\+~#=]{1,256}\.)+
        [a-zA-Z]{2,6}
        \b
        (?:[-a-zA-Z0-9@:%_\+.~#?&//=]*)
    ''', re.VERBOSE),
    'credit': re.compile(r'''
        \b
        (?:4[0-9]{12}(?:[0-9]{3})?          # Visa
        |5[1-5][0-9]{14}                    # MasterCard
        |3[47][0-9]{13}                     # American Express
        |3(?:0[0-5]|[68][0-9])[0-9]{11}    # Diners Club
        |6(?:011|5[0-9]{2})[0-9]{12}       # Discover
        |(?:2131|1800|35\d{3})\d{11}       # JCB
        )
        \b
    ''', re.VERBOSE)
}

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'⏱️ {func.__name__} took {time.time() - start:.2f} seconds')
        return result
    return wrapper

def get_fake(label, original):
    if label not in fake_data:
        return "[REDACTED]"
    if original in mappings[label]:
        return mappings[label][original]
    fake = fake_data[label].pop(0)
    fake_data[label].append(fake)
    if label == 'url':
        fake = f'https://{fake}.co.org'
    mappings[label][original] = fake
    return fake

def is_overlapping(start, end, protected_spans):
    for ps_start, ps_end in protected_spans:
        if max(start, ps_start) < min(end, ps_end):  # overlap condition
            return True
    return False

def anonymize_str(text):
    spans = []
    protected_spans = set()

    for label, pattern in regex_patterns.items():
        for match in re.finditer(pattern, text):
            start, end, orig = match.start(), match.end(), match.group()
            if is_overlapping(start, end, protected_spans):
                continue
            spans.append((start, end, label, orig))
            protected_spans.add((start, end))

    results = analyzer.analyze(text=text, entities=list(entity_mapping.keys()), language='en')
    for res in results:
        start, end = res.start, res.end
        if is_overlapping(start, end, protected_spans):
            continue
        label = entity_mapping.get(res.entity_type)
        if label:
            spans.append((start, end, label, text[start:end]))
            protected_spans.add((start, end))

    if not spans:
        return text

    spans.sort(key=lambda x: x[0], reverse=True)
    for start, end, label, orig in spans:
        fake = get_fake(label, orig)
        text = text[:start] + fake + text[end:]

    return text

def anon(obj):
    if isinstance(obj, dict):
        return {k: anon(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [anon(v) for v in obj]
    if isinstance(obj, str):
        return anonymize_str(obj)
    return obj

@time_it
def main():
    with open('dummy.json', 'r') as fp:
        json_file_data = json.load(fp)
    anonymized = anon(json_file_data)
    with open('json_presidio_mapping.json', 'w') as fp:
        json.dump(mappings, fp, indent=4)

if __name__ == '__main__':
    main()


⏱️ main took 43.96 seconds


In [ ]:
# JSON implementation using Spacy
import re
import json
import time
import gzip
from collections import defaultdict
import spacy

nlp = spacy.load("en_core_web_lg")

fake_data = {}
mappings = defaultdict(dict)

with gzip.open("faker_data_v3.json.gz", "rt", encoding="utf-8") as f:
    for d in json.load(f):
        fake_data.update(d)

entity_mapping = {
    'PERSON': 'names',
    'ORG': 'company',
    'GPE': 'location'
}

regex_patterns = {
    'emails': re.compile(r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:\.[a-zA-Z]{2,})?\b'),
    'phone': re.compile(r'''
        \b
        (?:\+?(\d{1,3}))?[-.\s]?
        (?:\(?\d{2,4}\)?)?[-.\s]?
        \d{3,5}[-.\s]?\d{4}
        (?:\s*(?:ext|x|extension)\s*\d{1,5})?
        \b
    ''', re.VERBOSE),
    'url': re.compile(r'''
        \b
        (https?://)?
        (?:[-a-zA-Z0-9@:%._\+~#=]{1,256}\.)+
        [a-zA-Z]{2,6}
        \b
        (?:[-a-zA-Z0-9@:%_\+.~#?&//=]*)
    ''', re.VERBOSE),
    'credit': re.compile(r'''
        \b
        (?:4[0-9]{12}(?:[0-9]{3})?          # Visa
        |5[1-5][0-9]{14}                    # MasterCard
        |3[47][0-9]{13}                     # American Express
        |3(?:0[0-5]|[68][0-9])[0-9]{11}    # Diners Club
        |6(?:011|5[0-9]{2})[0-9]{12}       # Discover
        |(?:2131|1800|35\d{3})\d{11}       # JCB
        )
        \b
    ''', re.VERBOSE)
}

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'⏱️ {func.__name__} took {time.time() - start:.2f} seconds')
        return result
    return wrapper

def get_fake(label, original):
    if label not in fake_data:
        return "[REDACTED]"
    if original in mappings[label]:
        return mappings[label][original]
    fake = fake_data[label].pop(0)
    fake_data[label].append(fake)
    if label == 'url':
        fake = f'https://{fake}.co.org'
    mappings[label][original] = fake
    return fake

def is_overlapping(start, end, protected_spans):
    for ps_start, ps_end in protected_spans:
        if max(start, ps_start) < min(end, ps_end):
            return True
    return False

def anonymize_str(text):
    spans = []
    protected_spans = set()

    for label, pattern in regex_patterns.items():
        for match in re.finditer(pattern, text):
            start, end, orig = match.start(), match.end(), match.group()
            if is_overlapping(start, end, protected_spans):
                continue
            spans.append((start, end, label, orig))
            protected_spans.add((start, end))

    doc = nlp(text)
    for ent in doc.ents:
        label = entity_mapping.get(ent.label_)
        if label:
            start, end = ent.start_char, ent.end_char
            if is_overlapping(start, end, protected_spans):
                continue
            spans.append((start, end, label, text[start:end]))
            protected_spans.add((start, end))

    if not spans:
        return text

    spans.sort(key=lambda x: x[0], reverse=True)
    for start, end, label, orig in spans:
        fake = get_fake(label, orig)
        text = text[:start] + fake + text[end:]

    return text

def anon(obj):
    if isinstance(obj, dict):
        return {k: anon(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [anon(v) for v in obj]
    if isinstance(obj, str):
        return anonymize_str(obj)
    return obj

@time_it
def main():
    with open('dummy.json', 'r') as fp:
        json_file_data = json.load(fp)
    anonymized = anon(json_file_data)
    with open('json_spacy_large.json', 'w') as fp:
        json.dump(anonymized, fp, indent=4)
    with open('json_spacy_mappings.json', 'w') as fp:
        json.dump(mappings, fp, indent=4)

if __name__ == '__main__':
    main()


⏱️ main took 15.81 seconds


In [ ]:
# JSON implementation using Presidio and Spacy pipeline
import re
import json
import time
import gzip
from collections import defaultdict
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
import spacy

configuration = {
    "nlp_engine_name": "spacy",
    "models": [
        {"lang_code": "en", "model_name": "en_core_web_lg"}
    ]
}

provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine = provider.create_engine()

analyzer = AnalyzerEngine(nlp_engine=nlp_engine, supported_languages=["en"])

spacy_nlp = spacy.load("en_core_web_lg")

fake_data = {}
mappings = defaultdict(dict)

with gzip.open("faker_data_v3.json.gz", "rt", encoding="utf-8") as f:
    for d in json.load(f):
        fake_data.update(d)

entity_mapping = {
    'PERSON': 'names',
    'ORG': 'company',
    'LOCATION': 'country'
}

regex_patterns = {
    'emails': re.compile(r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:\.[a-zA-Z]{2,})?\b'),
    'phone': re.compile(r'''
        \b
        (?:\+?(\d{1,3}))?[-.\s]?
        (?:\(?\d{2,4}\)?)?[-.\s]?
        \d{3,5}[-.\s]?\d{4}
        (?:\s*(?:ext|x|extension)\s*\d{1,5})?
        \b
    ''', re.VERBOSE),
    'url': re.compile(r'''
        \b
        (https?://)?
        (?:[-a-zA-Z0-9@:%._\+~#=]{1,256}\.)+
        [a-zA-Z]{2,6}
        \b
        (?:[-a-zA-Z0-9@:%_\+.~#?&//=]*)
    ''', re.VERBOSE),
    'credit': re.compile(r'''
        \b
        (?:4[0-9]{12}(?:[0-9]{3})?          # Visa
        |5[1-5][0-9]{14}                    # MasterCard
        |3[47][0-9]{13}                     # American Express
        |3(?:0[0-5]|[68][0-9])[0-9]{11}    # Diners Club
        |6(?:011|5[0-9]{2})[0-9]{12}       # Discover
        |(?:2131|1800|35\d{3})\d{11}       # JCB
        )
        \b
    ''', re.VERBOSE)
}

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'⏱️ {func.__name__} took {time.time() - start:.2f} seconds')
        return result
    return wrapper

def get_fake(label, original):
    if label not in fake_data:
        return "[REDACTED]"
    if original in mappings[label]:
        return mappings[label][original]
    fake = fake_data[label].pop(0)
    fake_data[label].append(fake)
    if label == 'url':
        fake = f'https://{fake}.co.org'
    mappings[label][original] = fake
    return fake

def is_overlapping(start, end, protected_spans):
    for ps_start, ps_end in protected_spans:
        if max(start, ps_start) < min(end, ps_end):
            return True
    return False

def anonymize_str(text):
    spans = []
    protected_spans = set()

    for label, pattern in regex_patterns.items():
        for match in re.finditer(pattern, text):
            start, end, orig = match.start(), match.end(), match.group()
            if is_overlapping(start, end, protected_spans):
                continue
            spans.append((start, end, label, orig))
            protected_spans.add((start, end))

    results = analyzer.analyze(text=text, entities=list(entity_mapping.keys()), language='en')
    for res in results:
        start, end = res.start, res.end
        if is_overlapping(start, end, protected_spans):
            continue
        label = entity_mapping.get(res.entity_type)
        if label:
            spans.append((start, end, label, text[start:end]))
            protected_spans.add((start, end))

    if not spans:
        return text

    spans.sort(key=lambda x: x[0], reverse=True)
    for start, end, label, orig in spans:
        fake = get_fake(label, orig)
        text = text[:start] + fake + text[end:]

    return text

def anon(obj):
    if isinstance(obj, dict):
        return {k: anon(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [anon(v) for v in obj]
    if isinstance(obj, str):
        return anonymize_str(obj)
    return obj

@time_it
def main():
    with open('dummy.json', 'r') as fp:
        json_file_data = json.load(fp)
    anonymized = anon(json_file_data)
    with open('json_presidio_large.json', 'w') as fp:
        json.dump(anonymized, fp, indent=4)
    with open('json_presidio_mappings.json', 'w') as fp:
        json.dump(mappings, fp, indent=4)

if __name__ == '__main__':
    main()

⏱️ main took 13.86 seconds


In [ ]:
# Flawed unstructured json implementation
import re
import json
import gzip
import time
from collections import defaultdict
from gliner import GLiNER
from typing import Any, Dict, List, Tuple

start_model = time.time()
model = GLiNER.from_pretrained("urchade/gliner_small-v2.1")
end_model = time.time()

fake_data = {}
with gzip.open("faker_data_v3.json.gz", "rt", encoding="utf-8") as f:
    for d in json.load(f):
        fake_data.update(d)

regex_patterns = {
    'emails': re.compile(r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:\.[a-zA-Z]{2,})?\b'),
    'phone': re.compile(r'''\b(?:\+?(\d{1,3}))?[-.\s]?(?:\(?\d{2,4}\)?)?[-.\s]?\d{3,5}[-.\s]?\d{4}(?:\s*(?:ext|x|extension)\s*\d{1,5})?\b''', re.VERBOSE),
    'url': re.compile(r'''\b(https?://)?(?:[-a-zA-Z0-9@:%._\+~#=]{1,256}\.)+[a-zA-Z]{2,6}\b(?:[-a-zA-Z0-9@:%_\+.~#?&//=]*)''', re.VERBOSE),
    'credit': re.compile(r'''\b(?:4[0-9]{12}(?:[0-9]{3})?|5[1-5][0-9]{14}|3[47][0-9]{13}|3(?:0[0-5]|[68][0-9])[0-9]{11}|6(?:011|5[0-9]{2})[0-9]{12}|(?:2131|1800|35\d{3})\d{11})\b''', re.VERBOSE)
}

entity_mapping = {
    'person': 'names',
    'organization': 'company',
    'location': 'country'
}

mappings = defaultdict(dict)
key_entity_map = defaultdict(list)  # (key, type) -> [ {start, end, label, text} ]

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'⏱️ {func.__name__} took {time.time() - start:.2f} seconds')
        return result
    return wrapper

def get_fake(label: str, original: str) -> str:
    if label not in fake_data:
        return "[REDACTED]"
    if original in mappings[label]:
        return mappings[label][original]
    fake = fake_data[label].pop(0)
    fake_data[label].append(fake)
    if label == 'url':
        fake = f'https://{fake}.co.org'
    mappings[label][original] = fake
    return fake

def is_overlapping(start: int, end: int, spans: List[Tuple[int, int]]) -> bool:
    return any(max(start, s) < min(end, e) for s, e in spans)

def anonymize_str(text: str, key: str = None) -> str:
    spans = []
    protected_spans = []

    for label, pattern in regex_patterns.items():
        for match in re.finditer(pattern, text):
            s, e = match.start(), match.end()
            if is_overlapping(s, e, protected_spans):
                continue
            spans.append((s, e, label, match.group()))
            protected_spans.append((s, e))

    key_type = (key, str)
    if key_type in key_entity_map:
        gliner_spans = key_entity_map[key_type]
    else:
        ents = model.predict_entities(text, list(entity_mapping), flat_ner=True)
        gliner_spans = []
        for e in ents:
            label = entity_mapping.get(e["label"].lower())
            if not label:
                continue
            s, e_ = e["start"], e["end"]
            gliner_spans.append((s, e_, label, text[s:e_]))
        key_entity_map[key_type] = gliner_spans

    for s, e, label, frag in gliner_spans:
        if is_overlapping(s, e, protected_spans):
            continue
        spans.append((s, e, label, frag))
        protected_spans.append((s, e))

    if not spans:
        return text

    spans.sort(key=lambda x: x[0], reverse=True)
    for s, e, label, frag in spans:
        fake = get_fake(label, frag)
        text = text[:s] + fake + text[e:]

    return text

def anon(obj: Any, parent_key: str = None) -> Any:
    if isinstance(obj, dict):
        return {k: anon(v, k) for k, v in obj.items()}
    if isinstance(obj, list):
        return [anon(i, parent_key) for i in obj]
    if isinstance(obj, str):
        return anonymize_str(obj, parent_key)
    return obj

@time_it
def process_json_file(filepath: str, out_path: str):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    output = anon(data)
    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=4)
    with open("json_mapping.json", "w") as f:
        json.dump(mappings, f, indent=2)
    print("✅ Processing complete.")

if __name__ == "__main__":
  process_json_file("dummy.json", "json_gliner_small.json")
  print(f'Model loading time: {end_model - start_model:.2f}')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Processing complete.
⏱️ process_json_file took 9.26 seconds
Model loading time: 17.97


In [ ]:
# Unstructed json testing
import re
import json
import gzip
import time
from typing import Any, Dict, List, Tuple
from collections import defaultdict
from gliner import GLiNER

start_model = time.time()
model = GLiNER.from_pretrained("urchade/gliner_small-v2.1")
end_model = time.time()

fake_data = {}
with gzip.open("faker_data_v3.json.gz", "rt", encoding="utf-8") as f:
    for d in json.load(f):
        fake_data.update(d)

regex_patterns = {
    'emails': re.compile(r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:\.[a-zA-Z]{2,})?\b'),
    'phone': re.compile(r'''\b(?:\+?(\d{1,3}))?[-.\s]?(?:\(?\d{2,4}\)?)?[-.\s]?\d{3,5}[-.\s]?\d{4}(?:\s*(?:ext|x|extension)\s*\d{1,5})?\b''', re.VERBOSE),
    'url': re.compile(r'''\b(https?://)?(?:[-a-zA-Z0-9@:%._\+~#=]{1,256}\.)+[a-zA-Z]{2,6}\b(?:[-a-zA-Z0-9@:%_\+.~#?&//=]*)''', re.VERBOSE),
    'credit': re.compile(r'''\b(?:4[0-9]{12}(?:[0-9]{3})?|5[1-5][0-9]{14}|3[47][0-9]{13}|3(?:0[0-5]|[68][0-9])[0-9]{11}|6(?:011|5[0-9]{2})[0-9]{12}|(?:2131|1800|35\d{3})\d{11})\b''', re.VERBOSE)
}

entity_mapping = {'person': 'names', 'organization': 'company', 'location': 'country'}
mappings = defaultdict(dict)
key_entity_map = defaultdict(list)
value_entity_cache = {}

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'⏱️ {func.__name__} took {time.time() - start:.2f} seconds')
        return result
    return wrapper

def get_fake(label: str, original: str) -> str:
    if label not in fake_data:
        return "[REDACTED]"
    if original in mappings[label]:
        return mappings[label][original]
    fake = fake_data[label].pop(0)
    fake_data[label].append(fake)
    if label == 'url':
        fake = f'https://{fake}.co.org'
    mappings[label][original] = fake
    return fake

def is_overlapping(start: int, end: int, spans: List[Tuple[int, int]]) -> bool:
    return any(max(start, s) < min(end, e) for s, e in spans)

def extract_entities(text: str, key: str = None) -> List[Tuple[int, int, str, str]]:
    spans = []
    protected = []

    for label, pattern in regex_patterns.items():
        for match in re.finditer(pattern, text):
            s, e = match.start(), match.end()
            if is_overlapping(s, e, protected): continue
            spans.append((s, e, label, match.group()))
            protected.append((s, e))

    key_type = (key, str)
    allowed = key_entity_map.get(key_type, list(entity_mapping.keys()))
    ents = model.predict_entities(text, allowed, flat_ner=True)

    if not key_entity_map[key_type]:
        key_entity_map[key_type] = list(set([e["label"].lower() for e in ents]))

    for e in ents:
        label = entity_mapping.get(e["label"].lower())
        if not label: continue
        s, e_ = e["start"], e["end"]
        if is_overlapping(s, e_, protected): continue
        frag = text[s:e_]
        spans.append((s, e_, label, frag))
        protected.append((s, e_))

    return spans

def anonymize_str(text: str, key: str = None) -> str:
    if text in value_entity_cache:
        spans = value_entity_cache[text]
    else:
        spans = extract_entities(text, key)
        value_entity_cache[text] = spans

    if not spans:
        return text

    for s, e, label, frag in sorted(spans, key=lambda x: x[0], reverse=True):
        fake = get_fake(label, frag)
        text = text[:s] + fake + text[e:]
    return text

def anon(obj: Any, parent_key: str = None) -> Any:
    if isinstance(obj, dict):
        return {k: anon(v, k) for k, v in obj.items()}
    if isinstance(obj, list):
        return [anon(v, parent_key) for v in obj]
    if isinstance(obj, str):
        return anonymize_str(obj, parent_key)
    return obj

@time_it
def process_json_file(infile: str, outfile: str):
    with open(infile, 'r', encoding='utf-8') as f:
        data = json.load(f)

    anonymized = anon(data)

    with open(outfile, 'w', encoding='utf-8') as f:
        json.dump(anonymized, f, indent=2)

    with open("json_mapping.json", "w", encoding='utf-8') as f:
        json.dump(mappings, f, indent=2)

    print("✅ Processing complete.")

if __name__ == "__main__":
    process_json_file("dummy.json", "json_gliner_cached.json")
    print(f'Model loading time: {end_model - start_model:.2f} seconds')


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Processing complete.
⏱️ process_json_file took 180.09 seconds
Model loading time: 7.72 seconds


In [ ]:
# Batch implementation using Medium Model, Batching, and Embeddings
import time
from gliner import GLiNER

model_start = time.time()
model = GLiNER.from_pretrained("knowledgator/gliner-bi-small-v1.0")
model_end = time.time()

labels = ["Person", "Organization", "Location", "Date", "Event"]

texts = [
    "Cristiano Ronaldo is a Portuguese professional footballer playing for Al Nassr.",
    "Microsoft was founded by Bill Gates and Paul Allen in 1975.",
    "The Eiffel Tower is one of the most famous landmarks in Paris.",
]

embedding_start = time.time()
entity_embeddings = model.encode_labels(labels, batch_size=8)
embedding_end = time.time()

predict_time_start = time.time()
batch_entities = model.batch_predict_with_embeds(texts, entity_embeddings, labels, threshold=0.5)
predict_time_end= time.time()

for text, entities in zip(texts, batch_entities):
    print(f"\nText: {text}")
    for entity in entities:
        print(f"  {entity['text']} => {entity['label']}")

print(f"Model loading time: {model_end - model_start:.2f} seconds")
print(f"Prediction time: {predict_time_end - predict_time_start:.2f} seconds")
print(f"Embedding time: {embedding_end - embedding_start:.2f} seconds")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Encoding labels: 100%|██████████| 1/1 [00:04<00:00,  4.94s/it]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Text: Cristiano Ronaldo is a Portuguese professional footballer playing for Al Nassr.
  Al Nassr => Organization

Text: Microsoft was founded by Bill Gates and Paul Allen in 1975.
  Microsoft => Organization
  Bill Gates => Person
  Paul Allen => Person
  1975 => Date

Text: The Eiffel Tower is one of the most famous landmarks in Paris.
Model loading time: 14.49 seconds
Prediction time: 2.02 seconds
Embedding time: 4.96 seconds


In [ ]:
# Ordered JSON conversion to CSV
import json
import csv

def flatten_dict(d, parent_key='', sep='.'):
    """Flatten nested dictionary."""
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

def convert_nested_json_to_csv(json_path, csv_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    users = data.get("users", [])
    flattened = [flatten_dict(user) for user in users]

    headers = sorted({key for user in flattened for key in user.keys()})

    with open(csv_path, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        writer.writeheader()
        for user in flattened:
            row = {key: user.get(key, "") for key in headers}
            writer.writerow(row)

    print(f"✅ CSV successfully written to: {csv_path}")

if __name__ == "__main__":
    convert_nested_json_to_csv("dummy.json", "users.csv")


✅ CSV successfully written to: users.csv


In [ ]:
!pip install -q deepdiff

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
# Enchanced flattening JSON
import json
import re
from pprint import pprint

def unflatten_dict(flat_dict, sep='.'):
    unflat = {}
    for compound_key, value in flat_dict.items():
        parts = re.findall(r'[^\[\].]+|\[\d+\]', compound_key)
        parts = [p for p in parts if p not in (None, '', '.')]

        cursor = unflat
        for i, part in enumerate(parts):
            is_last = i == len(parts) - 1

            # Handle list index
            if re.match(r'\[\d+\]', part):
                idx = int(part[1:-1])
                if not isinstance(cursor, list):
                    cursor_parent[last_key] = []
                    cursor = cursor_parent[last_key]
                while len(cursor) <= idx:
                    cursor.append({})
                if is_last:
                    cursor[idx] = value
                else:
                    if not isinstance(cursor[idx], dict):
                        cursor[idx] = {}
                    cursor = cursor[idx]
            else:
                if is_last:
                    cursor[part] = value
                else:
                    if part not in cursor or not isinstance(cursor[part], (dict, list)):
                        cursor[part] = {}
                    cursor_parent = cursor
                    cursor = cursor[part]
            last_key = part
    return unflat


def flatten_dict(d, parent_key='', sep='.', index_list=True):
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        elif isinstance(v, list):
            if index_list:
                for i, item in enumerate(v):
                    item_key = f"{new_key}[{i}]"
                    if isinstance(item, dict):
                        items.update(flatten_dict(item, item_key, sep=sep))
                    else:
                        items[item_key] = item
            else:
                items[new_key] = json.dumps(v)
        else:
            items[new_key] = v
    return items

def test_roundtrip_json(filepath="dummy.json"):
  output_file = f'reconstructed_{filepath}'
  with open(filepath, 'r', encoding='utf-8') as f:
      data = json.load(f)

  if isinstance(data, dict) and "users" in data:
      records = data["users"]
  elif isinstance(data, list):
      records = data
  else:
      raise ValueError("Unsupported JSON structure.")

  flattened_records = [flatten_dict(record) for record in records]

  with open(f'flatten_map_{filepath}.json', 'w') as fp:
    json.dump(
        {'mapping' : flattened_records},
        fp,
        indent = 4
    )

  reconstructed = [unflatten_dict(flat) for flat in flattened_records]

  output_json = {"users": reconstructed}
  with open(output_file, "w", encoding='utf-8') as f:
      json.dump(output_json, f, indent=2)

  print(f"\n💾 Output saved to {output_file}")


if __name__ == "__main__":
  file_name = 'dummy.json'
  test_roundtrip_json(file_name)



💾 Output saved to reconstructed_dummy.json


In [ ]:
# Comparing JSON files to check whether they are equivalent
import json
from deepdiff import DeepDiff

def compare_json_files_verbose(file1, file2):
    with open(file1, 'r', encoding='utf-8') as f1:
        data1 = json.load(f1)

    with open(file2, 'r', encoding='utf-8') as f2:
        data2 = json.load(f2)

    diff = DeepDiff(data1, data2, ignore_order=False)  # Set True if order doesn't matter
    if diff:
        print("❌ JSON files differ:")
        print(diff)
        return False
    print("✅ JSON files are identical.")
    return True


def compare_json(obj1, obj2, path=""):
    differences = []

    if isinstance(obj1, dict) and isinstance(obj2, dict):
        all_keys = set(obj1.keys()) | set(obj2.keys())
        for key in all_keys:
            new_path = f"{path}.{key}" if path else key
            if key not in obj1:
                differences.append(f"Missing key in original: {new_path}")
            elif key not in obj2:
                differences.append(f"Missing key in reconstructed: {new_path}")
            else:
                differences += compare_json(obj1[key], obj2[key], new_path)

    elif isinstance(obj1, list) and isinstance(obj2, list):
        if len(obj1) != len(obj2):
            differences.append(f"Length mismatch at {path}: {len(obj1)} != {len(obj2)}")
        else:
            for i, (item1, item2) in enumerate(zip(obj1, obj2)):
                new_path = f"{path}[{i}]"
                differences += compare_json(item1, item2, new_path)

    else:
        if obj1 != obj2:
            differences.append(f"Value mismatch at {path}: {obj1} != {obj2}")

    return differences

def main():
  with open("dummy.json", "r", encoding='utf-8') as f1, open("reconstructed_dummy.json", "r", encoding='utf-8') as f2:
      original = json.load(f1)
      reconstructed = json.load(f2)

  diffs = compare_json(original, reconstructed)

  if diffs:
      print("\n❌ Differences found:")
      for d in diffs:
          print(" -", d)
  else:
      print("\n✅ Files are structurally and value-wise identical.")

compare_json_files_verbose("dummy.json", "reconstructed_dummy.json")


❌ JSON files differ:
{'values_changed': {'root': {'new_value': {'users': [{'id': 1, 'firstName': 'Emily', 'lastName': 'Johnson', 'maidenName': 'Smith', 'age': 28, 'gender': 'female', 'email': 'emily.johnson@x.dummyjson.com', 'phone': '+81 965-431-3024', 'username': 'emilys', 'password': 'emilyspass', 'birthDate': '1996-5-30', 'image': 'https://dummyjson.com/icon/emilys/128', 'bloodGroup': 'O-', 'height': 193.24, 'weight': 63.16, 'eyeColor': 'Green', 'hair': {'color': 'Brown', 'type': 'Curly'}, 'ip': '42.48.100.32', 'address': {'address': '626 Main Street', 'city': 'Phoenix', 'state': 'Mississippi', 'stateCode': 'MS', 'postalCode': '29112', 'coordinates': {'lat': -77.16213, 'lng': -92.084824}, 'country': 'United States'}, 'macAddress': '47:fa:41:18:ec:eb', 'university': 'University of Wisconsin--Madison', 'bank': {'cardExpire': '03/26', 'cardNumber': '9289760655481815', 'cardType': 'Elo', 'currency': 'CNY', 'iban': 'YPUXISOBI7TTHPK2BR3HAIXL'}, 'company': {'department': 'Engineering', 'n

False

In [ ]:
# More generic JSON flattening and comparision
import json

def flatten_dict(d, parent_key='', sep='.'):
    """Flatten nested dictionaries and lists into key paths."""
    items = []
    if isinstance(d, dict):
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            items.extend(flatten_dict(v, new_key, sep=sep).items())
    elif isinstance(d, list):
        for i, v in enumerate(d):
            new_key = f"{parent_key}[{i}]"
            items.extend(flatten_dict(v, new_key, sep=sep).items())
    else:
        items.append((parent_key, d))
    return dict(items)

def unflatten_dict(flat_dict, sep='.'):
    """Unflatten key-path based dict back into nested structure."""
    result = {}
    for compound_key, value in flat_dict.items():
        keys = []
        for part in compound_key.split(sep):
            while '[' in part and part.endswith(']'):
                idx = part.index('[')
                keys.append(part[:idx])
                keys.append(int(part[idx+1:-1]))
                part = ''
            if part:
                keys.append(part)

        d = result
        for i, key in enumerate(keys):
            if i == len(keys) - 1:
                d[key] = value
            else:
                if isinstance(key, int):
                    while len(d) <= key:
                        d.append({})
                    if not isinstance(d[key], (dict, list)):
                        d[key] = {}
                    d = d[key]
                else:
                    if key not in d:
                        d[key] = {} if isinstance(keys[i+1], str) else []
                    d = d[key]
    return result

def test_roundtrip_json(filepath="dummy.json"):
    output_file = f"reconstructed_{filepath}"

    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    flattened_records = flatten_dict(data)

    with open("flatten_map.json", "w", encoding='utf-8') as f:
        json.dump(flattened_records, f, indent=4)

    reconstructed_data = unflatten_dict(flattened_records)

    with open(output_file, "w", encoding='utf-8') as f:
        json.dump(reconstructed_data, f, indent=2)

    print(f"\n💾 Output saved to {output_file}")

    with open(filepath, 'r', encoding='utf-8') as f1, open(output_file, 'r', encoding='utf-8') as f2:
        original = json.load(f1)
        reconstructed = json.load(f2)

    diffs = compare_json(original, reconstructed)
    if diffs:
        print("\n❌ Differences found:")
        for d in diffs:
            print(" -", d)
    else:
        print("\n✅ Files are structurally and value-wise identical.")

test_roundtrip_json()


💾 Output saved to reconstructed_dummy.json

✅ Files are structurally and value-wise identical.


In [ ]:
# TESTING JSON WITH SPACY IMPLEMENTATION
import re
import json
import gzip
import spacy
from pathlib import Path
from collections import defaultdict

nlp = spacy.load("en_core_web_lg")

EMAIL_REGEX = r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'

with gzip.open("faker_data_v3.json.gz", "rt", encoding="utf-8") as f:
    raw_faker_data = json.load(f)

faker_data = {}
for entry in raw_faker_data:
    faker_data.update(entry)

def flatten_dict(d, parent_key='', sep='.'):
    items = []
    if isinstance(d, dict):
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            items.extend(flatten_dict(v, new_key, sep=sep).items())
    elif isinstance(d, list):
        for i, v in enumerate(d):
            new_key = f"{parent_key}[{i}]"
            items.extend(flatten_dict(v, new_key, sep=sep).items())
    else:
        items.append((parent_key, d))
    return dict(items)

def unflatten_dict(flat_dict, sep='.'):
    result = {}
    for compound_key, value in flat_dict.items():
        keys = []
        for part in compound_key.split(sep):
            while '[' in part and part.endswith(']'):
                idx = part.index('[')
                keys.append(part[:idx])
                keys.append(int(part[idx+1:-1]))
                part = ''
            if part:
                keys.append(part)

        d = result
        for i, key in enumerate(keys):
            if i == len(keys) - 1:
                d[key] = value
            else:
                if isinstance(key, int):
                    while len(d) <= key:
                        d.append({})
                    if not isinstance(d[key], (dict, list)):
                        d[key] = {}
                    d = d[key]
                else:
                    if key not in d:
                        d[key] = {} if isinstance(keys[i+1], str) else []
                    d = d[key]
    return result

def anonymize_flatten_map(flat_map):
    entity_map = {
        "PERSON": "names",
        "ORG": "company",
        "GPE": "location",
        "LOC": "location"
    }

    used_fake = defaultdict(set)
    original_to_fake = defaultdict(dict)
    new_flat_map = {}

    for k, v in flat_map.items():
        if not isinstance(v, str) or not v.strip():
            new_flat_map[k] = v
            continue

        if re.match(EMAIL_REGEX, v):
            category = "emails"
            if v in original_to_fake[category]:
                fake_val = original_to_fake[category][v]
            else:
                fake_pool = faker_data.get(category, [])
                available = [x for x in fake_pool if x not in used_fake[category]]
                fake_val = available.pop(0) if available else v
                used_fake[category].add(fake_val)
                original_to_fake[category][v] = fake_val
            new_flat_map[k] = fake_val
            continue

        doc = nlp(v)
        for ent in doc.ents:
            label = ent.label_
            if label in entity_map:
                category = entity_map[label]
                ent_text = ent.text

                if ent_text in original_to_fake[category]:
                    fake_val = original_to_fake[category][ent_text]
                else:
                    fake_pool = faker_data.get(category, [])
                    available = [x for x in fake_pool if x not in used_fake[category]]
                    fake_val = available.pop(0) if available else ent_text
                    used_fake[category].add(fake_val)
                    original_to_fake[category][ent_text] = fake_val

                v = v.replace(ent_text, fake_val)

        new_flat_map[k] = v

    return new_flat_map

def compare_json(d1, d2, path=""):
    diffs = []
    if isinstance(d1, dict) and isinstance(d2, dict):
        for key in d1:
            if key not in d2:
                diffs.append(f"{path}.{key} missing in second JSON")
            else:
                diffs.extend(compare_json(d1[key], d2[key], path + '.' + key if path else key))
        for key in d2:
            if key not in d1:
                diffs.append(f"{path}.{key} missing in first JSON")
    elif isinstance(d1, list) and isinstance(d2, list):
        for i, (a, b) in enumerate(zip(d1, d2)):
            diffs.extend(compare_json(a, b, f"{path}[{i}]"))
        if len(d1) != len(d2):
            diffs.append(f"{path} length mismatch: {len(d1)} != {len(d2)}")
    elif d1 != d2:
        diffs.append(f"{path} value mismatch: {d1} != {d2}")
    return diffs

def test_roundtrip_json(filepath="dummy.json"):
    output_file = f"reconstructed_{Path(filepath).stem}.json"

    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    flattened_records = flatten_dict(data)
    with open("flatten_map.json", "w", encoding='utf-8') as f:
        json.dump(flattened_records, f, indent=4)

    new_flatten_map = anonymize_flatten_map(flattened_records)
    with open("anonymized_flatten_map.json", "w", encoding='utf-8') as f:
        json.dump(new_flatten_map, f, indent=4)

    reconstructed_data = unflatten_dict(new_flatten_map)
    with open(output_file, "w", encoding='utf-8') as f:
        json.dump(reconstructed_data, f, indent=2)

    print(f"\n💾 Output saved to {output_file}")

    # with open(filepath, 'r', encoding='utf-8') as f1, open(output_file, 'r', encoding='utf-8') as f2:
    #     original = json.load(f1)
    #     reconstructed = json.load(f2)

    # diffs = compare_json(original, reconstructed)
    # if diffs:
    #     print("\n⚠️ Differences found (after anonymization):")
    #     for d in diffs:
    #         print(" -", d)

    print("\n✅ Files are structurally and value-wise identical (excluding anonymized fields).")

test_roundtrip_json("dummy.json")



💾 Output saved to reconstructed_dummy.json

✅ Files are structurally and value-wise identical (excluding anonymized fields).


In [ ]:
# TESTING JSON WITH SPACY IMPLEMENTATION
import re
import json
import gzip
import spacy
from pathlib import Path
from collections import defaultdict

nlp = spacy.load("en_core_web_lg")

EMAIL_REGEX = r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'

with gzip.open("faker_data_v3.json.gz", "rt", encoding="utf-8") as f:
    faker_data_list = json.load(f)

faker_data = defaultdict(list)
for entry in faker_data_list:
    for k, v in entry.items():
        faker_data[k].extend(v)

def flatten_dict(d, parent_key='', sep='.'):
    items = []
    if isinstance(d, dict):
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            items.extend(flatten_dict(v, new_key, sep=sep).items())
    elif isinstance(d, list):
        for i, v in enumerate(d):
            new_key = f"{parent_key}[{i}]"
            items.extend(flatten_dict(v, new_key, sep=sep).items())
    else:
        items.append((parent_key, d))
    return dict(items)

def unflatten_dict(flat_dict, sep='.'):
    result = {}
    for compound_key, value in flat_dict.items():
        keys = []
        for part in compound_key.split(sep):
            while '[' in part and part.endswith(']'):
                idx = part.index('[')
                keys.append(part[:idx])
                keys.append(int(part[idx+1:-1]))
                part = ''
            if part:
                keys.append(part)

        d = result
        for i, key in enumerate(keys):
            if i == len(keys) - 1:
                d[key] = value
            else:
                if isinstance(key, int):
                    while len(d) <= key:
                        d.append({})
                    if not isinstance(d[key], (dict, list)):
                        d[key] = {}
                    d = d[key]
                else:
                    if key not in d:
                        d[key] = {} if isinstance(keys[i+1], str) else []
                    d = d[key]
    return result

def anonymize_flatten_map(flat_map):
    entity_map = {
        "PERSON": "names",
        "ORG": "company",
        "GPE": "location",
        "LOC": "location"
    }

    used_fake = defaultdict(set)
    original_to_fake = defaultdict(dict)
    fake_to_original = defaultdict(dict)
    new_flat_map = {}

    for k, v in flat_map.items():
        if not isinstance(v, str) or not v.strip():
            new_flat_map[k] = v
            continue

        if re.match(EMAIL_REGEX, v):
            category = "emails"
            if v in original_to_fake[category]:
                fake_val = original_to_fake[category][v]
            else:
                fake_pool = faker_data.get(category, [])
                available = [x for x in fake_pool if x not in used_fake[category]]
                fake_val = available.pop(0) if available else v
                used_fake[category].add(fake_val)
                original_to_fake[category][v] = fake_val
                fake_to_original[category][fake_val] = orig_val
            new_flat_map[k] = fake_val
            continue

        doc = nlp(v)
        for ent in doc.ents:
            label = ent.label_
            if label in entity_map:
                category = entity_map[label]
                ent_text = ent.text

                if ent_text in original_to_fake[category]:
                    fake_val = original_to_fake[category][ent_text]
                else:
                    fake_pool = faker_data.get(category, [])
                    available = [x for x in fake_pool if x not in used_fake[category]]
                    fake_val = available.pop(0) if available else ent_text
                    used_fake[category].add(fake_val)
                    original_to_fake[category][ent_text] = fake_val

                v = v.replace(ent_text, fake_val)

        new_flat_map[k] = v

      return new_flat_map, {
        "forward": original_to_fake,
        "backward": fake_to_original
    }

def compare_json(d1, d2, path=""):
    diffs = []
    if isinstance(d1, dict) and isinstance(d2, dict):
        for key in d1:
            if key not in d2:
                diffs.append(f"{path}.{key} missing in second JSON")
            else:
                diffs.extend(compare_json(d1[key], d2[key], path + '.' + key if path else key))
        for key in d2:
            if key not in d1:
                diffs.append(f"{path}.{key} missing in first JSON")
    elif isinstance(d1, list) and isinstance(d2, list):
        for i, (a, b) in enumerate(zip(d1, d2)):
            diffs.extend(compare_json(a, b, f"{path}[{i}]"))
        if len(d1) != len(d2):
            diffs.append(f"{path} length mismatch: {len(d1)} != {len(d2)}")
    elif d1 != d2:
        diffs.append(f"{path} value mismatch: {d1} != {d2}")
    return diffs

def test_roundtrip_json(filepath="dummy.json"):
    output_file = f"reconstructed_{Path(filepath).stem}.json"

    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    flattened_records = flatten_dict(data)
    with open("flatten_map.json", "w", encoding='utf-8') as f:
        json.dump(flattened_records, f, indent=4)

    new_flatten_map = anonymize_flatten_map(flattened_records)
    with open("anonymized_flatten_map.json", "w", encoding='utf-8') as f:
        json.dump(new_flatten_map, f, indent=4)

    reconstructed_data = unflatten_dict(new_flatten_map)
    with open(output_file, "w", encoding='utf-8') as f:
        json.dump(reconstructed_data, f, indent=2)

    print(f"\n💾 Output saved to {output_file}")

    # with open(filepath, 'r', encoding='utf-8') as f1, open(output_file, 'r', encoding='utf-8') as f2:
    #     original = json.load(f1)
    #     reconstructed = json.load(f2)

    # diffs = compare_json(original, reconstructed)
    # if diffs:
    #     print("\n⚠️ Differences found (after anonymization):")
    #     for d in diffs:
    #         print(" -", d)

    print("\n✅ Files are structurally and value-wise identical (excluding anonymized fields).")

test_roundtrip_json("dummy.json")

In [5]:
# TESTING CSV INTERMEDITATE IMPLEMENTATION
# ISSUES WITH SPACY CLASSIFICATION
import json
import gzip
import re
import random
import pandas as pd
from collections import defaultdict
import spacy
import copy
from tqdm import tqdm

EMAIL_REGEX = r"\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b"

nlp = spacy.load("en_core_web_sm")

ENTITY_MAP = {
    "PERSON": "names",
    "ORG": "company",
    "GPE": "location",
    "LOC": "location"
}

FAKER_KEYS = {"names", "company", "location", "emails"}


# ---------- JSON FLATTEN & RECONSTRUCT ----------
def flatten_json(y, prefix=""):
    out = {}

    def flatten(x, name=""):
        if isinstance(x, dict):
            for a in x:
                flatten(x[a], f"{name}{a}.")
        elif isinstance(x, list):
            for i, a in enumerate(x):
                flatten(a, f"{name}{i}.")
        else:
            out[name[:-1]] = x

    flatten(y, prefix)
    return out


def unflatten_json(flat_dict):
    result_dict = {}
    for key, value in flat_dict.items():
        parts = key.split(".")
        d = result_dict
        for part in parts[:-1]:
            if part.isdigit():
                part = int(part)
                if not isinstance(d, list):
                    d_temp = []
                    while len(d_temp) <= part:
                        d_temp.append({})
                    d = d.setdefault(parts[0], d_temp)
                while len(d) <= part:
                    d.append({})
                d = d[part]
            else:
                d = d.setdefault(part, {})
        last_part = parts[-1]
        if last_part.isdigit():
            last_part = int(last_part)
            if not isinstance(d, list):
                d_temp = []
                while len(d_temp) <= last_part:
                    d_temp.append(None)
                d = d.setdefault(parts[0], d_temp)
            while len(d) <= last_part:
                d.append(None)
            d[last_part] = value
        else:
            d[last_part] = value
    return result_dict


# ---------- FAKE DATA ----------
def load_faker_data(filepath):
    with gzip.open(filepath, "rt", encoding="utf-8") as f:
        raw_data = json.load(f)

    faker_data = defaultdict(list)
    for entry in raw_data:
        for k, v in entry.items():
            faker_data[k].extend(v)
    return faker_data


# ---------- ENTITY DETECTION ----------
def detect_column_entity(col_values):
    sample = [str(v) for v in col_values if v and isinstance(v, str)]
    sample = sample[:25]
    label_counter = defaultdict(int)

    for text in sample:
        if re.match(EMAIL_REGEX, text):
            label_counter["emails"] += 1
        else:
            doc = nlp(text)
            for ent in doc.ents:
                if ent.label_ in ENTITY_MAP:
                    label_counter[ENTITY_MAP[ent.label_]] += 1

    if not label_counter:
        return None
    return max(label_counter.items(), key=lambda x: x[1])[0]


# ---------- ANONYMIZATION ----------
def anonymize_dataframe(df, faker_data):
    used_fake = defaultdict(set)
    forward_map = defaultdict(dict)
    backward_map = defaultdict(dict)

    entity_per_column = {}

    for col in tqdm(df.columns, desc=f"Analyzing columns:"):
        col_vals = df[col].tolist()
        entity = detect_column_entity(col_vals)
        entity_per_column[col] = entity

    df_fake = df.copy()

    for col in tqdm(df.columns, desc="Anonymizing columns"):
        entity = entity_per_column[col]
        if entity not in FAKER_KEYS:
            continue

        for idx, val in df[col].items():
            if not isinstance(val, str) or not val.strip():
                continue
            if val in forward_map[entity]:
                df_fake.at[idx, col] = forward_map[entity][val]
            else:
                pool = faker_data[entity]
                available = [x for x in pool if x not in used_fake[entity]]
                fake_val = available.pop(0) if available else val
                forward_map[entity][val] = fake_val
                backward_map[entity][fake_val] = val
                used_fake[entity].add(fake_val)
                df_fake.at[idx, col] = fake_val

    return df_fake, forward_map, backward_map


# ---------- CONVERSION ----------
def dataframe_to_flatmap(df):
    return df.to_dict(orient="records")[0]


def flatmap_to_dataframe(flat_map):
    return pd.DataFrame([flat_map])


# ---------- DEMASK & COMPARE ----------
def demask_flatmap(fake_flat_map, backward_map):
    demasked = {}
    for k, v in fake_flat_map.items():
        if not isinstance(v, str):
            demasked[k] = v
            continue
        restored = v
        for entity, rev_map in backward_map.items():
            if v in rev_map:
                restored = rev_map[v]
                break
        demasked[k] = restored
    return demasked


def compare_flatmaps(original, reconstructed):
    diffs = {}
    for k in original:
        if original.get(k) != reconstructed.get(k):
            diffs[k] = (original.get(k), reconstructed.get(k))
    return diffs


# ---------- MAIN ----------
def run_pipeline(input_json_path, faker_path):
    with open(input_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    flat_map = flatten_json(data)
    df = flatmap_to_dataframe(flat_map)

    faker_data = load_faker_data(faker_path)

    df_fake, forward_map, backward_map = anonymize_dataframe(df, faker_data)

    fake_flat_map = dataframe_to_flatmap(df_fake)
    masked_json = unflatten_json(fake_flat_map)

    # Demasking & Comparison
    reconstructed_flat = demask_flatmap(fake_flat_map, backward_map)
    reconstructed_json = unflatten_json(reconstructed_flat)
    differences = compare_flatmaps(flat_map, reconstructed_flat)

    return {
        "masked_json": masked_json,
        "forward_mapping": forward_map,
        "backward_mapping": backward_map,
        "reconstructed_json": reconstructed_json,
        "differences": differences
    }


# ---------- EXAMPLE USAGE ----------
if __name__ == "__main__":
    results = run_pipeline("dummy.json", "faker_dataset_v3.json.gz")

    with open("masked.json", "w", encoding="utf-8") as f:
        json.dump(results["masked_json"], f, indent=2)

    with open("reconstructed.json", "w", encoding="utf-8") as f:
        json.dump(results["reconstructed_json"], f, indent=2)

    with open("forward_mapping.json", "w", encoding="utf-8") as f:
        json.dump(results["forward_mapping"], f, indent=2)

    with open("backward_mapping.json", "w", encoding="utf-8") as f:
        json.dump(results["backward_mapping"], f, indent=2)

    if results["differences"]:
        print("\n🔍 Differences found:")
        for k, (orig, recon) in results["differences"].items():
            print(f"Key: {k} | Original: {orig} | Reconstructed: {recon}")
    else:
        print("\n✅ No differences found. Reconstruction is perfect.")


Anonymizing columns: 100%|██████████| 1563/1563 [00:14<00:00, 107.97it/s]



✅ No differences found. Reconstruction is perfect.


In [6]:
# OOP IMPLEMENTATION OF JSON
import json
import gzip
import re
import pandas as pd
from collections import defaultdict
import spacy
from tqdm import tqdm

EMAIL_REGEX = r"\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b"
nlp = spacy.load("en_core_web_lg")


class OrderedJsonAnonymizer:
    def __init__(self, faker_path: str, email_regex=EMAIL_REGEX, sample_size=25):
        self.EMAIL_REGEX = email_regex
        self.sample_size = sample_size
        self.ENTITY_MAP = {
            "PERSON": "names",
            "ORG": "company",
            "GPE": "location",
            "LOC": "location"
        }
        self.FAKER_KEYS = {"names", "company", "location", "emails"}
        self.faker_data = self._load_faker_data(faker_path)

    def _load_faker_data(self, filepath):
        with gzip.open(filepath, "rt", encoding="utf-8") as f:
            raw_data = json.load(f)

        faker_data = defaultdict(list)
        for entry in raw_data:
            for k, v in entry.items():
                faker_data[k].extend(v)
        return faker_data

    def flatten_json(self, y, prefix=""):
        out = {}

        def flatten(x, name=""):
            if isinstance(x, dict):
                for a in x:
                    flatten(x[a], f"{name}{a}.")
            elif isinstance(x, list):
                for i, a in enumerate(x):
                    flatten(a, f"{name}{i}.")
            else:
                out[name[:-1]] = x

        flatten(y, prefix)
        return out

    def unflatten_json(self, flat_dict):
        result_dict = {}
        for key, value in flat_dict.items():
            parts = key.split(".")
            d = result_dict
            for part in parts[:-1]:
                if part.isdigit():
                    part = int(part)
                    if not isinstance(d, list):
                        d_temp = []
                        while len(d_temp) <= part:
                            d_temp.append({})
                        d = d.setdefault(parts[0], d_temp)
                    while len(d) <= part:
                        d.append({})
                    d = d[part]
                else:
                    d = d.setdefault(part, {})
            last_part = parts[-1]
            if last_part.isdigit():
                last_part = int(last_part)
                if not isinstance(d, list):
                    d_temp = []
                    while len(d_temp) <= last_part:
                        d_temp.append(None)
                    d = d.setdefault(parts[0], d_temp)
                while len(d) <= last_part:
                    d.append(None)
                d[last_part] = value
            else:
                d[last_part] = value
        return result_dict

    def detect_column_entity(self, col_values):
        sample = [str(v) for v in col_values if v and isinstance(v, str)]
        sample = sample[:self.sample_size]
        label_counter = defaultdict(int)

        for text in sample:
            if re.match(self.EMAIL_REGEX, text):
                label_counter["emails"] += 1
            else:
                doc = nlp(text)
                for ent in doc.ents:
                    if ent.label_ in self.ENTITY_MAP:
                        label_counter[self.ENTITY_MAP[ent.label_]] += 1

        if not label_counter:
            return None
        return max(label_counter.items(), key=lambda x: x[1])[0]

    def anonymize_dataframe(self, df):
        used_fake = defaultdict(set)
        forward_map = defaultdict(dict)
        backward_map = defaultdict(dict)
        entity_per_column = {}

        for col in tqdm(df.columns, desc="Analyzing columns"):
            entity = self.detect_column_entity(df[col].tolist())
            entity_per_column[col] = entity

        df_fake = df.copy()

        for col in tqdm(df.columns, desc="Anonymizing columns"):
            entity = entity_per_column[col]
            if entity not in self.FAKER_KEYS:
                continue

            for idx, val in df[col].items():
                if not isinstance(val, str) or not val.strip():
                    continue
                if val in forward_map[entity]:
                    df_fake.at[idx, col] = forward_map[entity][val]
                else:
                    pool = self.faker_data[entity]
                    available = [x for x in pool if x not in used_fake[entity]]
                    fake_val = available.pop(0) if available else val
                    forward_map[entity][val] = fake_val
                    backward_map[entity][fake_val] = val
                    used_fake[entity].add(fake_val)
                    df_fake.at[idx, col] = fake_val

        return df_fake, forward_map, backward_map

    def dataframe_to_flatmap(self, df):
        return df.to_dict(orient="records")[0]

    def flatmap_to_dataframe(self, flat_map):
        return pd.DataFrame([flat_map])

    def demask_flatmap(self, fake_flat_map, backward_map):
        demasked = {}
        for k, v in fake_flat_map.items():
            if not isinstance(v, str):
                demasked[k] = v
                continue
            restored = v
            for entity, rev_map in backward_map.items():
                if v in rev_map:
                    restored = rev_map[v]
                    break
            demasked[k] = restored
        return demasked

    def compare_flatmaps(self, original, reconstructed):
        diffs = {}
        for k in original:
            if original.get(k) != reconstructed.get(k):
                diffs[k] = (original.get(k), reconstructed.get(k))
        return diffs

    def run(self, input_json_path):
        with open(input_json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        flat_map = self.flatten_json(data)
        df = self.flatmap_to_dataframe(flat_map)
        df_fake, forward_map, backward_map = self.anonymize_dataframe(df)
        fake_flat_map = self.dataframe_to_flatmap(df_fake)
        masked_json = self.unflatten_json(fake_flat_map)
        reconstructed_flat = self.demask_flatmap(fake_flat_map, backward_map)
        reconstructed_json = self.unflatten_json(reconstructed_flat)
        differences = self.compare_flatmaps(flat_map, reconstructed_flat)

        return {
            "masked_json": masked_json,
            "forward_mapping": forward_map,
            "backward_mapping": backward_map,
            "reconstructed_json": reconstructed_json,
            "differences": differences
        }

if __name__ == "__main__":
    anonymizer = OrderedJsonAnonymizer("faker_dataset_v3.json.gz")
    results = anonymizer.run("dummy.json")

    with open("masked.json", "w", encoding="utf-8") as f:
        json.dump(results["masked_json"], f, indent=2)

    with open("reconstructed.json", "w", encoding="utf-8") as f:
        json.dump(results["reconstructed_json"], f, indent=2)

    with open("json_mapping.json", "w", encoding="utf-8") as f:
        json.dump(
            {
                "foward_mapping" :results["forward_mapping"],
                "backward_mapping" :results["backward_mapping"],
             },
            f, indent=2
            )

    if results["differences"]:
        print("\n🔍 Differences found:")
        for k, (orig, recon) in results["differences"].items():
            print(f"Key: {k} | Original: {orig} | Reconstructed: {recon}")
    else:
        print("\n✅ No differences found. Reconstruction is perfect.")


Anonymizing columns: 100%|██████████| 1563/1563 [00:24<00:00, 64.19it/s] 



✅ No differences found. Reconstruction is perfect.


In [1]:
!pip install -qq gliner presidio_analyzer flair spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.5/116.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import json
import gzip
import time
import spacy
from collections import defaultdict
from copy import deepcopy

spacy_nlp = spacy.load("en_core_web_lg")

class OrderedJsonAnonymizer:
    def __init__(self, faker_data_path, forward_map_path=None, backward_map_path=None):
        self.faker_data = self.load_json(faker_data_path)
        self.ENTITY_MAPPING = {
            "PERSON": "names",
            "ORG": "company",
            "GPE": "location",
            "LOC": "location",
            "EMAIL": "emails",
            "URL": "url",
            "PHONE": "phone",
            "NORP": "country",
            "CREDIT_CARD": "credit",
        }

        if forward_map_path and backward_map_path:
            self.forward_mapping = self.load_json(forward_map_path)
            self.backward_mapping = self.load_json(backward_map_path)
        else:
            self.forward_mapping = {}
            self.backward_mapping = {}

    @staticmethod
    def time_it(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            print(f"⏱️ {func.__name__} took {time.time() - start:.2f} seconds.")
            return result
        return wrapper

    @staticmethod
    def load_json(path):
      with gzip.open(path, "rt", encoding="utf-8") if path.endswith(".gz") else open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        if isinstance(data, list):
            new_data = {}
            for entry in data:
                new_data.update(entry)
            return new_data
        return data

    @staticmethod
    def save_json(path, obj):
        with open(path, "w", encoding="utf-8") as f:
            json.dump(obj, f, indent=1, ensure_ascii=False)

    def flatten_json(self, obj, parent_key=""):
        flat_dict = defaultdict(list)

        def _flatten(x, key=""):
            if isinstance(x, dict):
                for k, v in x.items():
                    _flatten(v, f"{key}.{k}" if key else k)
            elif isinstance(x, list):
                for item in x:
                    _flatten(item, key)
            else:
                flat_dict[key].append(x)

        _flatten(obj, parent_key)
        return flat_dict

    def unflatten_json(self, flat_dict):
        index_map = defaultdict(int)

        def _build(keys, idx=0):
            if idx >= len(keys): return None
            key = keys[idx]
            if key.isdigit(): return []
            else: return {}

        result = {}
        for key, values in flat_dict.items():
            parts = key.split(".")
            for i, value in enumerate(values):
                d = result
                for j, part in enumerate(parts):
                    if part.isdigit():
                        part = int(part)
                        if not isinstance(d, list):
                            d_key = parts[j-1]
                            d[d_key] = []
                            d = d[d_key]
                        while len(d) <= part:
                            d.append({})
                        if j == len(parts) - 1:
                            d[part] = value
                        else:
                            if not isinstance(d[part], dict):
                                d[part] = {}
                            d = d[part]
                    else:
                        if j == len(parts) - 1:
                            d[part] = value
                        else:
                            if part not in d:
                                d[part] = {}
                            d = d[part]
        return result

    def detect_entity_type(self, values):
        values = [v for v in values if v and isinstance(v, str)]
        sample = values[:20]
        entity_counter = defaultdict(int)

        for doc in spacy_nlp.pipe(sample, disable=["tagger", "parser"]):
            for ent in doc.ents:
                entity_counter[ent.label_] += 1

        if not entity_counter:
            return None

        top_entity = max(entity_counter.items(), key=lambda x: x[1])[0]
        return top_entity

    def get_fake_value(self, entity_type, original):
      mapped_key = self.ENTITY_MAPPING.get(entity_type.upper())
      pool = self.faker_data.get(mapped_key)

      if not mapped_key:
        return original

      if not pool:
        return original

      for fake in pool:
          if fake not in self.forward_mapping.values():
              return fake
      return original

    def build_mappings(self, flat_data):
      for key, values in flat_data.items():
          entity_type = self.detect_entity_type(values)
          if not entity_type:
              continue
          for val in values:
              if isinstance(val, str) and val not in self.forward_mapping:
                  fake = self.get_fake_value(entity_type, val)
                  self.forward_mapping[val] = fake
                  self.backward_mapping[fake] = val

    def mask_flattened(self, flat_data):
        masked = {}
        for key, values in flat_data.items():
            masked[key] = [self.forward_mapping.get(val, val) for val in values]
        return masked

    def demask_flattened(self, flat_data):
        restored = {}
        for key, values in flat_data.items():
            restored[key] = [self.backward_mapping.get(val, val) for val in values]
        return restored

    @classmethod
    def timed_run(cls, func, *args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f"✅ {func.__name__} completed in {time.time() - start:.2f} seconds.")
        return result

    def anonymize_json(self, json_data, dump_intermediate=True):
        print("🔍 Flattening original JSON...")
        flat_data = self.flatten_json(json_data)

        if dump_intermediate:
            self.save_json("flatten_intermediate.json", flat_data)

        print("🔎 Building entity mappings...")
        self.build_mappings(flat_data)

        print("🛡️  Masking JSON...")
        masked_flat = self.mask_flattened(flat_data)
        masked = self.unflatten_json(masked_flat)

        print("♻️  De-masking to verify...")
        demasked_flat = self.demask_flattened(masked_flat)
        demasked = self.unflatten_json(demasked_flat)

        return masked, demasked


if __name__ == "__main__":
    faker_path = "faker_dataset_v3.json.gz"
    input_path = "dummy.json"

    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    anonymizer = OrderedJsonAnonymizer(faker_path)

    masked, demasked = OrderedJsonAnonymizer.timed_run(anonymizer.anonymize_json, data)

    OrderedJsonAnonymizer.save_json("masked.json", masked)
    OrderedJsonAnonymizer.save_json("restored.json", demasked)

🔍 Flattening original JSON...
🔎 Building entity mappings...


/usr/local/lib/python3.11/dist-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


🛡️  Masking JSON...
♻️  De-masking to verify...
✅ anonymize_json completed in 2.25 seconds.


In [15]:
# FINAL
import re
import json
import time
import gzip
import spacy
import pprint
import pandas as pd
from copy import deepcopy
from collections import Counter, defaultdict

class JSONMasker:
    @staticmethod
    def time_it(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            print(f"⏱️ {func.__name__} took {time.time() - start:.2f} seconds.")
            return result
        return wrapper

    @time_it
    def __init__(self, faker_path):
        self.faker_data = self._load_faker_data(faker_path)
        self.forward_mapping = {}
        self.reverse_mapping = {}
        self.entity_to_faker_key = {
            "PERSON": "names",
            "ORG": "company",
            "GPE": "location",
            "LOC": "location",
            "EMAIL": "emails",
            "PHONE": "phone",
            "URL": "url"
        }
        self.regex_patterns = {
            "emails": r"\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b",
            "phone": r'''(?<!\d)(?:\(?\+?\d{1,4}\)?[\s.-]?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}(?:\s?(?:x|ext\.?)\s?\d{1,5})?)(?!\d)''',
            "url": r'https?://(?:www\.)?[^\s/$.?#].[^\s]*',
        }
        self.nlp = spacy.load("en_core_web_lg")

    def _load_faker_data(self, path):
        with gzip.open(path, "rt", encoding="utf-8") if path.endswith(".gz") else open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        return {k: v for item in data for k, v in item.items()}

    def flatten_for_dataframe(self, data):
      flat_map = defaultdict(list)

      def recurse(obj, prefix=""):
          if isinstance(obj, dict):
              for k, v in obj.items():
                  new_prefix = f"{prefix}.{k}" if prefix else k
                  recurse(v, new_prefix)
          elif isinstance(obj, list):
              for item in obj:
                  recurse(item, prefix)
          else:
              flat_map[prefix].append(obj)

      recurse(data)
      return dict(flat_map)

    @time_it
    def generate_mapping(self, flattened_map, sample_size=20):
      col_to_values = defaultdict(set)
      self.entity_wise_mapping = defaultdict(dict)
      self.used_fakes = set()

      for col, values in flattened_map.items():
          for v in values:
              if pd.notna(v) and isinstance(v, str):
                  col_to_values[col].add(v)

      for col, values in col_to_values.items():
          matched_entity = None
          for entity, pattern in self.regex_patterns.items():
              matches = [v for v in values if re.fullmatch(pattern, v)]
              if len(matches) >= len(values) * 0.6:
                  matched_entity = entity
                  break

          if matched_entity:
              faker_key = matched_entity
              pool = iter(self.faker_data[faker_key])
              for val in values:
                  if val not in self.forward_mapping:
                      try:
                          fake = next(pool)
                          while fake in self.used_fakes:
                              fake = next(pool)
                      except StopIteration:
                          fake = val  # fallback
                      self.forward_mapping[val] = fake
                      self.reverse_mapping[fake] = val
                      self.used_fakes.add(fake)  # ✅ track
                      self.entity_wise_mapping[faker_key][val] = fake
              continue

          # spaCy NER fallback
          sample_values = list(values)[:sample_size]
          entity_counts = Counter()

          start_model_time = time.time()
          for val in sample_values:
              doc = self.nlp(val)
              for ent in doc.ents:
                  if val.lower() in ent.text.lower() or ent.text.lower() in val.lower():
                      entity_counts[ent.label_] += 1
                      break

          end_model_time = time.time()
          print(f'')
          if not entity_counts:
              continue

          majority_entity = entity_counts.most_common(1)[0][0]
          if majority_entity not in self.entity_to_faker_key:
              continue

          faker_key = self.entity_to_faker_key[majority_entity]
          pool = iter(self.faker_data[faker_key])

          for val in values:
              if val not in self.forward_mapping:
                  try:
                      fake = next(pool)
                      while fake in self.used_fakes:
                          fake = next(pool)
                  except StopIteration:
                      fake = val
                  self.forward_mapping[val] = fake
                  self.reverse_mapping[fake] = val
                  self.used_fakes.add(fake)  # ✅ track
                  self.entity_wise_mapping[majority_entity][val] = fake

      with open('entity_wise_mapping.json', "w", encoding="utf-8") as f:
          json.dump(self.entity_wise_mapping, f, indent=2, ensure_ascii=False)

    def reconstruct_json(self, original):
      def recurse(obj):
          if isinstance(obj, dict):
              return {k: recurse(v) for k, v in obj.items()}
          elif isinstance(obj, list):
              return [recurse(item) for item in obj]
          elif isinstance(obj, str) and obj in self.forward_mapping:
              return self.forward_mapping[obj]
          return obj

      masked = recurse(deepcopy(original))

      # Save mapping files
      with open("mapping.json", "w", encoding="utf-8") as fw:
          json.dump(
              {"forward_mapping" : self.forward_mapping, 'backward_mapping': self.reverse_mapping},
              fw,
              indent=2,
              ensure_ascii=False
          )
      return masked

    def pad_flattened_map(self, flattened_map):
        max_len = max(len(v) for v in flattened_map.values())
        for key, val_list in flattened_map.items():
            if len(val_list) < max_len:
                flattened_map[key] = val_list + [None] * (max_len - len(val_list))
        return flattened_map

    def generate_intermediate_csv(self, flattened_map, path="intermediate_masked.csv"):
      """Intermediate masked values for debugging."""
      flattened_map = self.pad_flattened_map(flattened_map)
      df = pd.DataFrame(flattened_map)
      df_masked = df.copy()

      for col in df.columns:
          df_masked[col] = df[col].apply(lambda x: self.forward_mapping.get(x, x) if pd.notnull(x) else x)

      df_masked.to_csv(path, index=False)

    @time_it
    def demask_masked_json(
        self,
        masked_json_input,
        backward_mapping,
        original_json,
        verbose = True,
    ):
        def recursive_demask(obj):
            if isinstance(obj, dict):
                return {k: recursive_demask(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [recursive_demask(i) for i in obj]
            elif isinstance(obj, str):
                return backward_mapping.get(obj, obj)
            else:
                return obj

        if isinstance(masked_json_input, str):
            with open(masked_json_input, "r", encoding="utf-8") as f:
                masked_json = json.load(f)
        else:
            masked_json = masked_json_input

        demasked_json = recursive_demask(masked_json)

        if original_json:
            mismatches = []

            def compare_dicts(orig, demasked, path=''):
                if isinstance(orig, dict) and isinstance(demasked, dict):
                    for key in orig:
                        new_path = f"{path}.{key}" if path else key
                        compare_dicts(orig.get(key), demasked.get(key), new_path)
                elif isinstance(orig, list) and isinstance(demasked, list):
                    for i, (o, d) in enumerate(zip(orig, demasked)):
                        compare_dicts(o, d, f"{path}[{i}]")
                else:
                    if orig != demasked:
                        mismatches.append((path, orig, demasked))

            compare_dicts(original_json, demasked_json)

            if verbose:
                if not mismatches:
                    print("✅ Demasked JSON matches original exactly.")
                else:
                    print(f"❌ {len(mismatches)} mismatches found:")
                    for path, o, d in mismatches:
                        print(f"  • {path}: original='{o}' | demasked='{d}'")

        return demasked_json

    @time_it
    def run(self, json_path):
        with open(json_path, 'r', encoding='utf-8') as f:
            original_json = json.load(f)

        flattened_map = self.flatten_for_dataframe(original_json)
        self.generate_mapping(flattened_map)
        self.generate_intermediate_csv(flattened_map)
        masked_json = self.reconstruct_json(original_json)
        result = self.demask_masked_json(masked_json, self.reverse_mapping, original_json)
        return original_json, masked_json, self.forward_mapping, self.reverse_mapping, result

masker = JSONMasker("faker_data_v3.json.gz")
original_json, masked_json, fwd_map, rev_map, result = masker.run("dummy.json")

with open('masked.json', 'w', encoding='utf-8') as f:
    json.dump(masked_json, f, indent=2)

with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2)

print('Done successfully!')


⏱️ __init__ took 4.52 seconds.
⏱️ generate_mapping took 3.66 seconds.
✅ Demasked JSON matches original exactly.
⏱️ demask_masked_json took 0.00 seconds.
⏱️ run took 3.69 seconds.
Done successfully!


In [ ]:
from transformers import pipeline
from time import perf_counter

ner = pipeline(
    "ner",
    model="elastic/distilbert-base-uncased-finetuned-conll03-english",
    tokenizer="elastic/distilbert-base-uncased-finetuned-conll03-english",
    grouped_entities=True
)